#LOAD DATA

In [ ]:
import pandas as pd

splits = {'train': 'chapter/train-00000-of-00001.parquet', 'test': 'chapter/test-00000-of-00001.parquet', 'validation': 'chapter/validation-00000-of-00001.parquet'}
train_c = pd.read_parquet("hf://datasets/AI-team-UoA/greek_legal_code/" + splits["train"])
val_c = pd.read_parquet("hf://datasets/AI-team-UoA/greek_legal_code/" + splits["validation"])
test_c = pd.read_parquet("hf://datasets/AI-team-UoA/greek_legal_code/" + splits["test"])


splits = {'train': 'subject/train-00000-of-00001.parquet', 'test': 'subject/test-00000-of-00001.parquet', 'validation': 'subject/validation-00000-of-00001.parquet'}
train_s = pd.read_parquet("hf://datasets/AI-team-UoA/greek_legal_code/" + splits["train"])
val_s = pd.read_parquet("hf://datasets/AI-team-UoA/greek_legal_code/" + splits["validation"])
test_s = pd.read_parquet("hf://datasets/AI-team-UoA/greek_legal_code/" + splits["test"])


splits = {'train': 'volume/train-00000-of-00001.parquet', 'test': 'volume/test-00000-of-00001.parquet', 'validation': 'volume/validation-00000-of-00001.parquet'}
train_v = pd.read_parquet("hf://datasets/AI-team-UoA/greek_legal_code/" + splits["train"])
val_v = pd.read_parquet("hf://datasets/AI-team-UoA/greek_legal_code/" + splits["validation"])
test_v = pd.read_parquet("hf://datasets/AI-team-UoA/greek_legal_code/" + splits["test"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# B1 1.

# Volume

#TFIDF

In [ ]:

import pandas as pd
import re
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


# Text Preprocessing


greek_stopwords = ['Α∆ΙΑΚΟΠΑ', 'ΑΙ', 'ΑΚΟΜΑ', 'ΑΚΟΜΗ', 'ΑΚΡΙΒΩΣ', 'ΑΛΗΘΕΙΑ', 'ΑΛΗΘΙΝΑ', 'ΑΛΛΑ', 'ΑΛΛΑΧΟΥ', 'ΑΛΛΕΣ', 'ΑΛΛΗ', 'ΑΛΛΗΝ', 'ΑΛΛΗΣ',
                   'ΑΛΛΙΩΣ', 'ΑΛΛΙΩΤΙΚΑ', 'ΑΛΛΟ', 'ΑΛΛΟΙ', 'ΑΛΛΟΙΩΣ', 'ΑΛΛΟΙΩΤΙΚΑ', 'ΑΛΛΟΝ', 'ΑΛΛΟΣ', 'ΑΛΛΟΤΕ', 'ΑΛΛΟΥ', 'ΑΛΛΟΥΣ', 'ΑΛΛΩΝ', 'ΑΜΑ',
                   'ΑΜΕΣΑ', 'ΑΜΕΣΩΣ', 'ΑΝ', 'ΑΝΑ', 'ΑΝΑΜΕΣΑ', 'ΑΝΑΜΕΤΑΞΥ', 'ΑΝΕΥ', 'ΑΝΤΙ', 'ΑΝΤΙΠΕΡΑ', 'ΑΝΤΙΣ', 'ΑΝΩ', 'ΑΝΩΤΕΡΩ', 'ΑΞΑΦΝΑ', 'ΑΠ',
                   'ΑΠΕΝΑΝΤΙ', 'ΑΠΟ', 'ΑΠΟΨΕ', 'ΑΡΑ', 'ΑΡΑΓΕ', 'ΑΡΓΑ', 'ΑΡΓΟΤΕΡΟ', 'ΑΡΙΣΤΕΡΑ', 'ΑΡΚΕΤΑ', 'ΑΡΧΙΚΑ', 'ΑΣ', 'ΑΥΡΙΟ', 'ΑΥΤΑ', 'ΑΥΤΕΣ',
                   'ΑΥΤΗ', 'ΑΥΤΗΝ', 'ΑΥΤΗΣ', 'ΑΥΤΟ', 'ΑΥΤΟΙ', 'ΑΥΤΟΝ', 'ΑΥΤΟΣ', 'ΑΥΤΟΥ', 'ΑΥΤΟΥΣ', 'ΑΥΤΩΝ', 'ΑΦΟΤΟΥ', 'ΑΦΟΥ', 'ΒΕΒΑΙΑ', 'ΒΕΒΑΙΟΤΑΤΑ',
                   'ΓΙ', 'ΓΙΑ', 'ΓΡΗΓΟΡΑ', 'ΓΥΡΩ', '∆Α', '∆Ε', '∆ΕΙΝΑ', '∆ΕΝ', '∆ΕΞΙΑ', '∆ΗΘΕΝ', '∆ΗΛΑ∆Η', '∆Ι', '∆ΙΑ', '∆ΙΑΡΚΩΣ', '∆ΙΚΑ', '∆ΙΚΟ',
                   '∆ΙΚΟΙ', '∆ΙΚΟΣ', '∆ΙΚΟΥ', '∆ΙΚΟΥΣ', '∆ΙΟΛΟΥ', '∆ΙΠΛΑ', '∆ΙΧΩΣ', 'ΕΑΝ', 'ΕΑΥΤΟ', 'ΕΑΥΤΟΝ', 'ΕΑΥΤΟΥ', 'ΕΑΥΤΟΥΣ', 'ΕΑΥΤΩΝ',
                   'ΕΓΚΑΙΡΑ', 'ΕΓΚΑΙΡΩΣ', 'ΕΓΩ', 'Ε∆Ω', 'ΕΙ∆ΕΜΗ', 'ΕΙΘΕ', 'ΕΙΜΑΙ', 'ΕΙΜΑΣΤΕ', 'ΕΙΝΑΙ', 'ΕΙΣ', 'ΕΙΣΑΙ', 'ΕΙΣΑΣΤΕ', 'ΕΙΣΤΕ', 'ΕΙΤΕ',
                   'ΕΙΧΑ', 'ΕΙΧΑΜΕ', 'ΕΙΧΑΝ', 'ΕΙΧΑΤΕ', 'ΕΙΧΕ', 'ΕΙΧΕΣ', 'ΕΚΑΣΤΑ', 'ΕΚΑΣΤΕΣ', 'ΕΚΑΣΤΗ', 'ΕΚΑΣΤΗΝ', 'ΕΚΑΣΤΗΣ', 'ΕΚΑΣΤΟ', 'ΕΚΑΣΤΟΙ',
                   'ΕΚΑΣΤΟΝ', 'ΕΚΑΣΤΟΣ', 'ΕΚΑΣΤΟΥ', 'ΕΚΑΣΤΟΥΣ', 'ΕΚΑΣΤΩΝ', 'ΕΚΕΙ', 'ΕΚΕΙΝΑ', 'ΕΚΕΙΝΕΣ', 'ΕΚΕΙΝΗ', 'ΕΚΕΙΝΗΝ', 'ΕΚΕΙΝΗΣ', 'ΕΚΕΙΝΟ',
                   'ΕΚΕΙΝΟΙ', 'ΕΚΕΙΝΟΝ', 'ΕΚΕΙΝΟΣ', 'ΕΚΕΙΝΟΥ', 'ΕΚΕΙΝΟΥΣ', 'ΕΚΕΙΝΩΝ', 'ΕΚΤΟΣ', 'ΕΜΑΣ', 'ΕΜΕΙΣ', 'ΕΜΕΝΑ', 'ΕΜΠΡΟΣ', 'ΕΝ', 'ΕΝΑ',
                   'ΕΝΑΝ', 'ΕΝΑΣ', 'ΕΝΟΣ', 'ΕΝΤΕΛΩΣ', 'ΕΝΤΟΣ', 'ΕΝΤΩΜΕΤΑΞΥ', 'ΕΝΩ', 'ΕΞ', 'ΕΞΑΦΝΑ', 'ΕΞΗΣ', 'ΕΞΙΣΟΥ', 'ΕΞΩ', 'ΕΠΑΝΩ', 'ΕΠΕΙ∆Η',
                   'ΕΠΕΙΤΑ', 'ΕΠΙ', 'ΕΠΙΣΗΣ', 'ΕΠΟΜΕΝΩΣ', 'ΕΣΑΣ', 'ΕΣΕΙΣ', 'ΕΣΕΝΑ', 'ΕΣΤΩ', 'ΕΣΥ', 'ΕΤΕΡΑ', 'ΕΤΕΡΑΙ', 'ΕΤΕΡΑΣ', 'ΕΤΕΡΕΣ', 'ΕΤΕΡΗ',
                   'ΕΤΕΡΗΣ', 'ΕΤΕΡΟ', 'ΕΤΕΡΟΙ', 'ΕΤΕΡΟΝ', 'ΕΤΕΡΟΣ', 'ΕΤΕΡΟΥ', 'ΕΤΕΡΟΥΣ', 'ΕΤΕΡΩΝ', 'ΕΤΟΥΤΑ', 'ΕΤΟΥΤΕΣ', 'ΕΤΟΥΤΗ', 'ΕΤΟΥΤΗΝ',
                   'ΕΤΟΥΤΗΣ', 'ΕΤΟΥΤΟ', 'ΕΤΟΥΤΟΙ', 'ΕΤΟΥΤΟΝ', 'ΕΤΟΥΤΟΣ', 'ΕΤΟΥΤΟΥ', 'ΕΤΟΥΤΟΥΣ', 'ΕΤΟΥΤΩΝ', 'ΕΤΣΙ', 'ΕΥΓΕ', 'ΕΥΘΥΣ', 'ΕΥΤΥΧΩΣ',
                   'ΕΦΕΞΗΣ', 'ΕΧΕΙ', 'ΕΧΕΙΣ', 'ΕΧΕΤΕ', 'ΕΧΘΕΣ', 'ΕΧΟΜΕ', 'ΕΧΟΥΜΕ', 'ΕΧΟΥΝ', 'ΕΧΤΕΣ', 'ΕΧΩ', 'ΕΩΣ', 'Η', 'Η∆Η', 'ΗΜΑΣΤΑΝ', 'ΗΜΑΣΤΕ',
                   'ΗΜΟΥΝ', 'ΗΣΑΣΤΑΝ', 'ΗΣΑΣΤΕ', 'ΗΣΟΥΝ', 'ΗΤΑΝ', 'ΗΤΑΝΕ', 'ΗΤΟΙ', 'ΗΤΤΟΝ', 'ΘΑ', 'Ι', 'Ι∆ΙΑ', 'Ι∆ΙΑΝ', 'Ι∆ΙΑΣ', 'Ι∆ΙΕΣ', 'Ι∆ΙΟ',
                   'Ι∆ΙΟΙ', 'Ι∆ΙΟΝ', 'Ι∆ΙΟΣ', 'Ι∆ΙΟΥ', 'Ι∆ΙΟΥΣ', 'Ι∆ΙΩΝ', 'Ι∆ΙΩΣ', 'ΙΙ', 'ΙΙΙ', 'ΙΣΑΜΕ', 'ΙΣΙΑ', 'ΙΣΩΣ', 'ΚΑΘΕ', 'ΚΑΘΕΜΙΑ',
                   'ΚΑΘΕΜΙΑΣ', 'ΚΑΘΕΝΑ', 'ΚΑΘΕΝΑΣ', 'ΚΑΘΕΝΟΣ', 'ΚΑΘΕΤΙ', 'ΚΑΘΟΛΟΥ', 'ΚΑΘΩΣ', 'ΚΑΙ', 'ΚΑΚΑ', 'ΚΑΚΩΣ', 'ΚΑΛΑ', 'ΚΑΛΩΣ', 'ΚΑΜΙΑ',
                   'ΚΑΜΙΑΝ', 'ΚΑΜΙΑΣ', 'ΚΑΜΠΟΣΑ', 'ΚΑΜΠΟΣΕΣ', 'ΚΑΜΠΟΣΗ', 'ΚΑΜΠΟΣΗΝ', 'ΚΑΜΠΟΣΗΣ', 'ΚΑΜΠΟΣΟ', 'ΚΑΜΠΟΣΟΙ', 'ΚΑΜΠΟΣΟΝ', 'ΚΑΜΠΟΣΟΣ',
                   'ΚΑΜΠΟΣΟΥ', 'ΚΑΜΠΟΣΟΥΣ', 'ΚΑΜΠΟΣΩΝ', 'ΚΑΝΕΙΣ', 'ΚΑΝΕΝ', 'ΚΑΝΕΝΑ', 'ΚΑΝΕΝΑΝ', 'ΚΑΝΕΝΑΣ', 'ΚΑΝΕΝΟΣ', 'ΚΑΠΟΙΑ', 'ΚΑΠΟΙΑΝ', 'ΚΑΠΟΙΑΣ',
                   'ΚΑΠΟΙΕΣ', 'ΚΑΠΟΙΟ', 'ΚΑΠΟΙΟΙ', 'ΚΑΠΟΙΟΝ', 'ΚΑΠΟΙΟΣ', 'ΚΑΠΟΙΟΥ', 'ΚΑΠΟΙΟΥΣ', 'ΚΑΠΟΙΩΝ', 'ΚΑΠΟΤΕ', 'ΚΑΠΟΥ', 'ΚΑΠΩΣ', 'ΚΑΤ', 'ΚΑΤΑ',
                   'ΚΑΤΙ', 'ΚΑΤΙΤΙ', 'ΚΑΤΟΠΙΝ', 'ΚΑΤΩ', 'ΚΙΟΛΑΣ', 'ΚΛΠ', 'ΚΟΝΤΑ', 'ΚΤΛ', 'ΚΥΡΙΩΣ', 'ΛΙΓΑΚΙ', 'ΛΙΓΟ', 'ΛΙΓΩΤΕΡΟ', 'ΛΟΓΩ', 'ΛΟΙΠΑ',
                   'ΛΟΙΠΟΝ', 'ΜΑ', 'ΜΑΖΙ', 'ΜΑΚΑΡΙ', 'ΜΑΚΡΥΑ', 'ΜΑΛΙΣΤΑ', 'ΜΑΛΛΟΝ', 'ΜΑΣ', 'ΜΕ', 'ΜΕΘΑΥΡΙΟ', 'ΜΕΙΟΝ', 'ΜΕΛΕΙ', 'ΜΕΛΛΕΤΑΙ', 'ΜΕΜΙΑΣ',
                   'ΜΕΝ', 'ΜΕΡΙΚΑ', 'ΜΕΡΙΚΕΣ', 'ΜΕΡΙΚΟΙ', 'ΜΕΡΙΚΟΥΣ', 'ΜΕΡΙΚΩΝ', 'ΜΕΣΑ', 'ΜΕΤ', 'ΜΕΤΑ', 'ΜΕΤΑΞΥ', 'ΜΕΧΡΙ', 'ΜΗ', 'ΜΗ∆Ε', 'ΜΗΝ',
                   'ΜΗΠΩΣ', 'ΜΗΤΕ', 'ΜΙΑ', 'ΜΙΑΝ', 'ΜΙΑΣ', 'ΜΟΛΙΣ', 'ΜΟΛΟΝΟΤΙ', 'ΜΟΝΑΧΑ', 'ΜΟΝΕΣ', 'ΜΟΝΗ', 'ΜΟΝΗΝ', 'ΜΟΝΗΣ', 'ΜΟΝΟ', 'ΜΟΝΟΙ',
                   'ΜΟΝΟΜΙΑΣ', 'ΜΟΝΟΣ', 'ΜΟΝΟΥ', 'ΜΟΝΟΥΣ', 'ΜΟΝΩΝ', 'ΜΟΥ', 'ΜΠΟΡΕΙ', 'ΜΠΟΡΟΥΝ', 'ΜΠΡΑΒΟ', 'ΜΠΡΟΣ', 'ΝΑ', 'ΝΑΙ', 'ΝΩΡΙΣ', 'ΞΑΝΑ',
                   'ΞΑΦΝΙΚΑ', 'Ο', 'ΟΙ', 'ΟΛΑ', 'ΟΛΕΣ', 'ΟΛΗ', 'ΟΛΗΝ', 'ΟΛΗΣ', 'ΟΛΟ', 'ΟΛΟΓΥΡΑ', 'ΟΛΟΙ', 'ΟΛΟΝ', 'ΟΛΟΝΕΝ', 'ΟΛΟΣ', 'ΟΛΟΤΕΛΑ', 'ΟΛΟΥ',
                   'ΟΛΟΥΣ', 'ΟΛΩΝ', 'ΟΛΩΣ', 'ΟΛΩΣ∆ΙΟΛΟΥ', 'ΟΜΩΣ', 'ΟΠΟΙΑ', 'ΟΠΟΙΑ∆ΗΠΟΤΕ', 'ΟΠΟΙΑΝ', 'ΟΠΟΙΑΝ∆ΗΠΟΤΕ', 'ΟΠΟΙΑΣ', 'ΟΠΟΙΑΣ∆ΗΠΟΤΕ',
                   'ΟΠΟΙ∆ΗΠΟΤΕ', 'ΟΠΟΙΕΣ', 'ΟΠΟΙΕΣ∆ΗΠΟΤΕ', 'ΟΠΟΙΟ', 'ΟΠΟΙΟ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΙ', 'ΟΠΟΙΟΝ', 'ΟΠΟΙΟΝ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΣ', 'ΟΠΟΙΟΣ∆ΗΠΟΤΕ',
                   'ΟΠΟΙΟΥ', 'ΟΠΟΙΟΥ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΥΣ', 'ΟΠΟΙΟΥΣ∆ΗΠΟΤΕ', 'ΟΠΟΙΩΝ', 'ΟΠΟΙΩΝ∆ΗΠΟΤΕ', 'ΟΠΟΤΕ', 'ΟΠΟΤΕ∆ΗΠΟΤΕ', 'ΟΠΟΥ', 'ΟΠΟΥ∆ΗΠΟΤΕ',
                   'ΟΠΩΣ', 'ΟΡΙΣΜΕΝΑ', 'ΟΡΙΣΜΕΝΕΣ', 'ΟΡΙΣΜΕΝΩΝ', 'ΟΡΙΣΜΕΝΩΣ', 'ΟΣΑ', 'ΟΣΑ∆ΗΠΟΤΕ', 'ΟΣΕΣ', 'ΟΣΕΣ∆ΗΠΟΤΕ', 'ΟΣΗ', 'ΟΣΗ∆ΗΠΟΤΕ', 'ΟΣΗΝ',
                   'ΟΣΗΝ∆ΗΠΟΤΕ', 'ΟΣΗΣ', 'ΟΣΗΣ∆ΗΠΟΤΕ', 'ΟΣΟ', 'ΟΣΟ∆ΗΠΟΤΕ', 'ΟΣΟΙ', 'ΟΣΟΙ∆ΗΠΟΤΕ', 'ΟΣΟΝ', 'ΟΣΟΝ∆ΗΠΟΤΕ', 'ΟΣΟΣ', 'ΟΣΟΣ∆ΗΠΟΤΕ', 'ΟΣΟΥ',
                   'ΟΣΟΥ∆ΗΠΟΤΕ', 'ΟΣΟΥΣ', 'ΟΣΟΥΣ∆ΗΠΟΤΕ', 'ΟΣΩΝ', 'ΟΣΩΝ∆ΗΠΟΤΕ', 'ΟΤΑΝ', 'ΟΤΙ', 'ΟΤΙ∆ΗΠΟΤΕ', 'ΟΤΟΥ', 'ΟΥ', 'ΟΥ∆Ε', 'ΟΥΤΕ', 'ΟΧΙ',
                   'ΠΑΛΙ', 'ΠΑΝΤΟΤΕ', 'ΠΑΝΤΟΥ', 'ΠΑΝΤΩΣ', 'ΠΑΡΑ', 'ΠΕΡΑ', 'ΠΕΡΙ', 'ΠΕΡΙΠΟΥ', 'ΠΕΡΙΣΣΟΤΕΡΟ', 'ΠΕΡΣΙ', 'ΠΕΡΥΣΙ', 'ΠΙΑ', 'ΠΙΘΑΝΟΝ',
                   'ΠΙΟ', 'ΠΙΣΩ', 'ΠΛΑΙ', 'ΠΛΕΟΝ', 'ΠΛΗΝ', 'ΠΟΙΑ', 'ΠΟΙΑΝ', 'ΠΟΙΑΣ', 'ΠΟΙΕΣ', 'ΠΟΙΟ', 'ΠΟΙΟΙ', 'ΠΟΙΟΝ', 'ΠΟΙΟΣ', 'ΠΟΙΟΥ', 'ΠΟΙΟΥΣ',
                   'ΠΟΙΩΝ', 'ΠΟΛΥ', 'ΠΟΣΕΣ', 'ΠΟΣΗ', 'ΠΟΣΗΝ', 'ΠΟΣΗΣ', 'ΠΟΣΟΙ', 'ΠΟΣΟΣ', 'ΠΟΣΟΥΣ', 'ΠΟΤΕ', 'ΠΟΥ', 'ΠΟΥΘΕ', 'ΠΟΥΘΕΝΑ', 'ΠΡΕΠΕΙ',
                   'ΠΡΙΝ', 'ΠΡΟ', 'ΠΡΟΚΕΙΜΕΝΟΥ', 'ΠΡΟΚΕΙΤΑΙ', 'ΠΡΟΠΕΡΣΙ', 'ΠΡΟΣ', 'ΠΡΟΤΟΥ', 'ΠΡΟΧΘΕΣ', 'ΠΡΟΧΤΕΣ', 'ΠΡΩΤΥΤΕΡΑ', 'ΠΩΣ', 'ΣΑΝ', 'ΣΑΣ',
                   'ΣΕ', 'ΣΕΙΣ', 'ΣΗΜΕΡΑ', 'ΣΙΓΑ', 'ΣΟΥ', 'ΣΤΑ', 'ΣΤΗ', 'ΣΤΗΝ', 'ΣΤΗΣ', 'ΣΤΙΣ', 'ΣΤΟ', 'ΣΤΟΝ', 'ΣΤΟΥ', 'ΣΤΟΥΣ', 'ΣΤΩΝ', 'ΣΥΓΧΡΟΝΩΣ',
                   'ΣΥΝ', 'ΣΥΝΑΜΑ', 'ΣΥΝΕΠΩΣ', 'ΣΥΝΗΘΩΣ', 'ΣΥΧΝΑ', 'ΣΥΧΝΑΣ', 'ΣΥΧΝΕΣ', 'ΣΥΧΝΗ', 'ΣΥΧΝΗΝ', 'ΣΥΧΝΗΣ', 'ΣΥΧΝΟ', 'ΣΥΧΝΟΙ', 'ΣΥΧΝΟΝ',
                   'ΣΥΧΝΟΣ', 'ΣΥΧΝΟΥ', 'ΣΥΧΝΟΥ', 'ΣΥΧΝΟΥΣ', 'ΣΥΧΝΩΝ', 'ΣΥΧΝΩΣ', 'ΣΧΕ∆ΟΝ', 'ΣΩΣΤΑ', 'ΤΑ', 'ΤΑ∆Ε', 'ΤΑΥΤΑ', 'ΤΑΥΤΕΣ', 'ΤΑΥΤΗ', 'ΤΑΥΤΗΝ',
                   'ΤΑΥΤΗΣ', 'ΤΑΥΤΟ,ΤΑΥΤΟΝ', 'ΤΑΥΤΟΣ', 'ΤΑΥΤΟΥ', 'ΤΑΥΤΩΝ', 'ΤΑΧΑ', 'ΤΑΧΑΤΕ', 'ΤΕΛΙΚΑ', 'ΤΕΛΙΚΩΣ', 'ΤΕΣ', 'ΤΕΤΟΙΑ', 'ΤΕΤΟΙΑΝ',
                   'ΤΕΤΟΙΑΣ', 'ΤΕΤΟΙΕΣ', 'ΤΕΤΟΙΟ', 'ΤΕΤΟΙΟΙ', 'ΤΕΤΟΙΟΝ', 'ΤΕΤΟΙΟΣ', 'ΤΕΤΟΙΟΥ', 'ΤΕΤΟΙΟΥΣ', 'ΤΕΤΟΙΩΝ', 'ΤΗ', 'ΤΗΝ', 'ΤΗΣ', 'ΤΙ',
                   'ΤΙΠΟΤΑ', 'ΤΙΠΟΤΕ', 'ΤΙΣ', 'ΤΟ', 'ΤΟΙ', 'ΤΟΝ', 'ΤΟΣ', 'ΤΟΣΑ', 'ΤΟΣΕΣ', 'ΤΟΣΗ', 'ΤΟΣΗΝ', 'ΤΟΣΗΣ', 'ΤΟΣΟ', 'ΤΟΣΟΙ', 'ΤΟΣΟΝ', 'ΤΟΣΟΣ',
                   'ΤΟΣΟΥ', 'ΤΟΣΟΥΣ', 'ΤΟΣΩΝ', 'ΤΟΤΕ', 'ΤΟΥ', 'ΤΟΥΛΑΧΙΣΤΟ', 'ΤΟΥΛΑΧΙΣΤΟΝ', 'ΤΟΥΣ', 'ΤΟΥΤΑ', 'ΤΟΥΤΕΣ', 'ΤΟΥΤΗ', 'ΤΟΥΤΗΝ', 'ΤΟΥΤΗΣ',
                   'ΤΟΥΤΟ', 'ΤΟΥΤΟΙ', 'ΤΟΥΤΟΙΣ', 'ΤΟΥΤΟΝ', 'ΤΟΥΤΟΣ', 'ΤΟΥΤΟΥ', 'ΤΟΥΤΟΥΣ', 'ΤΟΥΤΩΝ', 'ΤΥΧΟΝ', 'ΤΩΝ', 'ΤΩΡΑ', 'ΥΠ', 'ΥΠΕΡ', 'ΥΠΟ',
                   'ΥΠΟΨΗ', 'ΥΠΟΨΙΝ', 'ΥΣΤΕΡΑ', 'ΦΕΤΟΣ', 'ΧΑΜΗΛΑ', 'ΧΘΕΣ', 'ΧΤΕΣ', 'ΧΩΡΙΣ', 'ΧΩΡΙΣΤΑ', 'ΨΗΛΑ', 'Ω', 'ΩΡΑΙΑ', 'ΩΣ', 'ΩΣΑΝ', 'ΩΣΟΤΟΥ',
                   'ΩΣΠΟΥ', 'ΩΣΤΕ', 'ΩΣΤΟΣΟ', 'ΩΧ']


def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# Prepare data
X_train = train_v["text"]
y_train = train_v["label"]
X_val = val_v["text"]
y_val = val_v["label"]
X_test = test_v["text"]
y_test = test_v["label"]


for df in [train_v]:
    df["text"] = df["text"].astype(str).apply(clean_text)



val_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words=greek_stopwords,
        lowercase=True,
        ngram_range=(1, 2),   # unigrams and bigrams
        min_df=2,
        max_df=0.95
    )),
    ('clf', LinearSVC())
])

val_pipeline.fit(X_train,y_train)

# -----------------------------
# Evaluate on Validation Set
# -----------------------------
y_pred_val = val_pipeline.predict(X_val)
print("\nTest Results:\n")
print(classification_report(y_val, y_pred_val))


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'εαν', 'εαυτο', 'εαυτον', 'εαυτου', 'εαυτους', 'εαυτων', 'εγκαιρα', 'εγκαιρως', 'εγω', 'ει', 'ειθε', 'ειμαι', 'ειμαστε', 'εινα', 'ειναι', 'εις', 


Test Results:

              precision    recall  f1-score   support

           0       0.85      0.86      0.86       187
           1       0.85      0.92      0.88       389
           2       0.94      0.82      0.88        79
           3       0.84      0.85      0.84       244
           4       0.87      0.92      0.90       346
           5       0.84      0.80      0.82       108
           6       0.94      0.88      0.91       224
           7       0.93      0.69      0.79       111
           8       0.86      0.71      0.78        91
           9       0.90      0.76      0.82        71
          10       0.95      0.89      0.92       160
          11       0.84      0.87      0.86       184
          12       0.92      0.83      0.87        83
          13       0.89      0.91      0.90       152
          14       0.91      0.90      0.90       169
          15       0.88      0.80      0.84       155
          16       0.85      0.80      0.83       102
          1

TEST DATASET VOLUME TFIDF

In [ ]:
final_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words=greek_stopwords,
        lowercase=True,
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.95
    )),
    ('clf', LinearSVC())
])

final_pipeline.fit(X_train,y_train)

# Evaluate on Test Set

y_pred_test = final_pipeline.predict(X_test)
print("\nTest Results:\n")
print(classification_report(y_test, y_pred_test))

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'εαν', 'εαυτο', 'εαυτον', 'εαυτου', 'εαυτους', 'εαυτων', 'εγκαιρα', 'εγκαιρως', 'εγω', 'ει', 'ειθε', 'ειμαι', 'ειμαστε', 'εινα', 'ειναι', 'εις', 


Test Results:

              precision    recall  f1-score   support

           0       0.81      0.84      0.83       187
           1       0.84      0.91      0.88       389
           2       0.86      0.79      0.83        78
           3       0.88      0.84      0.85       243
           4       0.84      0.91      0.87       346
           5       0.81      0.76      0.78       109
           6       0.94      0.88      0.91       225
           7       0.90      0.78      0.83       110
           8       0.94      0.79      0.86        92
           9       0.81      0.77      0.79        71
          10       0.95      0.86      0.90       160
          11       0.83      0.86      0.85       183
          12       0.96      0.86      0.90        83
          13       0.85      0.90      0.87       153
          14       0.92      0.80      0.86       170
          15       0.88      0.83      0.85       155
          16       0.83      0.85      0.84       102
          1

# BOW (Bag of words)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_pipeline = Pipeline([
    ('bow', CountVectorizer(
        ngram_range=(1, 2),  # unigrams + bigrams
        min_df=2,
        max_df=0.95,
        stop_words=greek_stopwords
    )),
    ('clf', LinearSVC())
])
bow_pipeline.fit(X_train,y_train)


# Evaluate

y_pred_bow = bow_pipeline.predict(X_val)
print("\n📊 Test Results with BoW:\n")
print(classification_report(y_val, y_pred_bow))

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'εαν', 'εαυτο', 'εαυτον', 'εαυτου', 'εαυτους', 'εαυτων', 'εγκαιρα', 'εγκαιρως', 'εγω', 'ει', 'ειθε', 'ειμαι', 'ειμαστε', 'εινα', 'ειναι', 'εις', 


📊 Test Results with BoW:

              precision    recall  f1-score   support

           0       0.78      0.76      0.77       187
           1       0.80      0.86      0.83       389
           2       0.77      0.75      0.76        79
           3       0.79      0.79      0.79       244
           4       0.81      0.81      0.81       346
           5       0.67      0.73      0.70       108
           6       0.89      0.83      0.86       224
           7       0.78      0.57      0.66       111
           8       0.74      0.60      0.67        91
           9       0.69      0.63      0.66        71
          10       0.87      0.87      0.87       160
          11       0.81      0.77      0.79       184
          12       0.88      0.69      0.77        83
          13       0.82      0.80      0.81       152
          14       0.83      0.80      0.82       169
          15       0.77      0.76      0.77       155
          16       0.81      0.74      0.77       102


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_pipeline = Pipeline([
    ('bow', CountVectorizer(
        ngram_range=(1, 1),  # unigrams + bigrams
        min_df=2,            # πολύ σπάνιες λέξεις
        max_df=0.95,          # αγνoεί πολύ συχνές λέξεις
        stop_words=greek_stopwords
    )),
    ('clf', LinearSVC(C=0.8645797044664848))
])
bow_pipeline.fit(X_train,y_train)


# Evaluate

y_pred_bow = bow_pipeline.predict(X_val)
print("\n📊 Test Results with BoW:\n")
print(classification_report(y_val, y_pred_bow))

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'εαν', 'εαυτο', 'εαυτον', 'εαυτου', 'εαυτους', 'εαυτων', 'εγκαιρα', 'εγκαιρως', 'εγω', 'ει', 'ειθε', 'ειμαι', 'ειμαστε', 'εινα', 'ειναι', 'εις', 


📊 Test Results with BoW:

              precision    recall  f1-score   support

           0       0.76      0.75      0.75       187
           1       0.81      0.86      0.83       389
           2       0.74      0.78      0.76        79
           3       0.76      0.77      0.76       244
           4       0.82      0.82      0.82       346
           5       0.71      0.77      0.74       108
           6       0.84      0.85      0.84       224
           7       0.72      0.61      0.66       111
           8       0.66      0.60      0.63        91
           9       0.71      0.69      0.70        71
          10       0.83      0.91      0.87       160
          11       0.75      0.75      0.75       184
          12       0.82      0.73      0.78        83
          13       0.82      0.79      0.80       152
          14       0.82      0.79      0.81       169
          15       0.75      0.78      0.77       155
          16       0.74      0.68      0.71       102


TEST DATASET VOLUME BAG OF WORDS

In [ ]:
bow_final_pipeline = Pipeline([
    ('bow', CountVectorizer(
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.95,
        stop_words=greek_stopwords
    )),
    ('clf', LinearSVC())
])
bow_final_pipeline.fit(X_train,y_train)


# Evaluate

y_pred_bow_final = bow_final_pipeline.predict(X_test)
print("\n📊 Test Results with BoW:\n")
print(classification_report(y_test, y_pred_bow_final))

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'εαν', 'εαυτο', 'εαυτον', 'εαυτου', 'εαυτους', 'εαυτων', 'εγκαιρα', 'εγκαιρως', 'εγω', 'ει', 'ειθε', 'ειμαι', 'ειμαστε', 'εινα', 'ειναι', 'εις', 


📊 Test Results with BoW:

              precision    recall  f1-score   support

           0       0.73      0.74      0.74       187
           1       0.78      0.86      0.82       389
           2       0.73      0.73      0.73        78
           3       0.75      0.76      0.76       243
           4       0.78      0.83      0.80       346
           5       0.71      0.72      0.71       109
           6       0.85      0.80      0.82       225
           7       0.80      0.71      0.75       110
           8       0.82      0.70      0.75        92
           9       0.70      0.69      0.70        71
          10       0.87      0.82      0.84       160
          11       0.79      0.76      0.78       183
          12       0.88      0.81      0.84        83
          13       0.76      0.80      0.78       153
          14       0.78      0.72      0.75       170
          15       0.77      0.77      0.77       155
          16       0.73      0.75      0.74       102


# Chapter

# TFIDF

In [ ]:


import pandas as pd
import re
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

# Text Preprocessing

greek_stopwords = ['Α∆ΙΑΚΟΠΑ', 'ΑΙ', 'ΑΚΟΜΑ', 'ΑΚΟΜΗ', 'ΑΚΡΙΒΩΣ', 'ΑΛΗΘΕΙΑ', 'ΑΛΗΘΙΝΑ', 'ΑΛΛΑ', 'ΑΛΛΑΧΟΥ', 'ΑΛΛΕΣ', 'ΑΛΛΗ', 'ΑΛΛΗΝ', 'ΑΛΛΗΣ',
                   'ΑΛΛΙΩΣ', 'ΑΛΛΙΩΤΙΚΑ', 'ΑΛΛΟ', 'ΑΛΛΟΙ', 'ΑΛΛΟΙΩΣ', 'ΑΛΛΟΙΩΤΙΚΑ', 'ΑΛΛΟΝ', 'ΑΛΛΟΣ', 'ΑΛΛΟΤΕ', 'ΑΛΛΟΥ', 'ΑΛΛΟΥΣ', 'ΑΛΛΩΝ', 'ΑΜΑ',
                   'ΑΜΕΣΑ', 'ΑΜΕΣΩΣ', 'ΑΝ', 'ΑΝΑ', 'ΑΝΑΜΕΣΑ', 'ΑΝΑΜΕΤΑΞΥ', 'ΑΝΕΥ', 'ΑΝΤΙ', 'ΑΝΤΙΠΕΡΑ', 'ΑΝΤΙΣ', 'ΑΝΩ', 'ΑΝΩΤΕΡΩ', 'ΑΞΑΦΝΑ', 'ΑΠ',
                   'ΑΠΕΝΑΝΤΙ', 'ΑΠΟ', 'ΑΠΟΨΕ', 'ΑΡΑ', 'ΑΡΑΓΕ', 'ΑΡΓΑ', 'ΑΡΓΟΤΕΡΟ', 'ΑΡΙΣΤΕΡΑ', 'ΑΡΚΕΤΑ', 'ΑΡΧΙΚΑ', 'ΑΣ', 'ΑΥΡΙΟ', 'ΑΥΤΑ', 'ΑΥΤΕΣ',
                   'ΑΥΤΗ', 'ΑΥΤΗΝ', 'ΑΥΤΗΣ', 'ΑΥΤΟ', 'ΑΥΤΟΙ', 'ΑΥΤΟΝ', 'ΑΥΤΟΣ', 'ΑΥΤΟΥ', 'ΑΥΤΟΥΣ', 'ΑΥΤΩΝ', 'ΑΦΟΤΟΥ', 'ΑΦΟΥ', 'ΒΕΒΑΙΑ', 'ΒΕΒΑΙΟΤΑΤΑ',
                   'ΓΙ', 'ΓΙΑ', 'ΓΡΗΓΟΡΑ', 'ΓΥΡΩ', '∆Α', '∆Ε', '∆ΕΙΝΑ', '∆ΕΝ', '∆ΕΞΙΑ', '∆ΗΘΕΝ', '∆ΗΛΑ∆Η', '∆Ι', '∆ΙΑ', '∆ΙΑΡΚΩΣ', '∆ΙΚΑ', '∆ΙΚΟ',
                   '∆ΙΚΟΙ', '∆ΙΚΟΣ', '∆ΙΚΟΥ', '∆ΙΚΟΥΣ', '∆ΙΟΛΟΥ', '∆ΙΠΛΑ', '∆ΙΧΩΣ', 'ΕΑΝ', 'ΕΑΥΤΟ', 'ΕΑΥΤΟΝ', 'ΕΑΥΤΟΥ', 'ΕΑΥΤΟΥΣ', 'ΕΑΥΤΩΝ',
                   'ΕΓΚΑΙΡΑ', 'ΕΓΚΑΙΡΩΣ', 'ΕΓΩ', 'Ε∆Ω', 'ΕΙ∆ΕΜΗ', 'ΕΙΘΕ', 'ΕΙΜΑΙ', 'ΕΙΜΑΣΤΕ', 'ΕΙΝΑΙ', 'ΕΙΣ', 'ΕΙΣΑΙ', 'ΕΙΣΑΣΤΕ', 'ΕΙΣΤΕ', 'ΕΙΤΕ',
                   'ΕΙΧΑ', 'ΕΙΧΑΜΕ', 'ΕΙΧΑΝ', 'ΕΙΧΑΤΕ', 'ΕΙΧΕ', 'ΕΙΧΕΣ', 'ΕΚΑΣΤΑ', 'ΕΚΑΣΤΕΣ', 'ΕΚΑΣΤΗ', 'ΕΚΑΣΤΗΝ', 'ΕΚΑΣΤΗΣ', 'ΕΚΑΣΤΟ', 'ΕΚΑΣΤΟΙ',
                   'ΕΚΑΣΤΟΝ', 'ΕΚΑΣΤΟΣ', 'ΕΚΑΣΤΟΥ', 'ΕΚΑΣΤΟΥΣ', 'ΕΚΑΣΤΩΝ', 'ΕΚΕΙ', 'ΕΚΕΙΝΑ', 'ΕΚΕΙΝΕΣ', 'ΕΚΕΙΝΗ', 'ΕΚΕΙΝΗΝ', 'ΕΚΕΙΝΗΣ', 'ΕΚΕΙΝΟ',
                   'ΕΚΕΙΝΟΙ', 'ΕΚΕΙΝΟΝ', 'ΕΚΕΙΝΟΣ', 'ΕΚΕΙΝΟΥ', 'ΕΚΕΙΝΟΥΣ', 'ΕΚΕΙΝΩΝ', 'ΕΚΤΟΣ', 'ΕΜΑΣ', 'ΕΜΕΙΣ', 'ΕΜΕΝΑ', 'ΕΜΠΡΟΣ', 'ΕΝ', 'ΕΝΑ',
                   'ΕΝΑΝ', 'ΕΝΑΣ', 'ΕΝΟΣ', 'ΕΝΤΕΛΩΣ', 'ΕΝΤΟΣ', 'ΕΝΤΩΜΕΤΑΞΥ', 'ΕΝΩ', 'ΕΞ', 'ΕΞΑΦΝΑ', 'ΕΞΗΣ', 'ΕΞΙΣΟΥ', 'ΕΞΩ', 'ΕΠΑΝΩ', 'ΕΠΕΙ∆Η',
                   'ΕΠΕΙΤΑ', 'ΕΠΙ', 'ΕΠΙΣΗΣ', 'ΕΠΟΜΕΝΩΣ', 'ΕΣΑΣ', 'ΕΣΕΙΣ', 'ΕΣΕΝΑ', 'ΕΣΤΩ', 'ΕΣΥ', 'ΕΤΕΡΑ', 'ΕΤΕΡΑΙ', 'ΕΤΕΡΑΣ', 'ΕΤΕΡΕΣ', 'ΕΤΕΡΗ',
                   'ΕΤΕΡΗΣ', 'ΕΤΕΡΟ', 'ΕΤΕΡΟΙ', 'ΕΤΕΡΟΝ', 'ΕΤΕΡΟΣ', 'ΕΤΕΡΟΥ', 'ΕΤΕΡΟΥΣ', 'ΕΤΕΡΩΝ', 'ΕΤΟΥΤΑ', 'ΕΤΟΥΤΕΣ', 'ΕΤΟΥΤΗ', 'ΕΤΟΥΤΗΝ',
                   'ΕΤΟΥΤΗΣ', 'ΕΤΟΥΤΟ', 'ΕΤΟΥΤΟΙ', 'ΕΤΟΥΤΟΝ', 'ΕΤΟΥΤΟΣ', 'ΕΤΟΥΤΟΥ', 'ΕΤΟΥΤΟΥΣ', 'ΕΤΟΥΤΩΝ', 'ΕΤΣΙ', 'ΕΥΓΕ', 'ΕΥΘΥΣ', 'ΕΥΤΥΧΩΣ',
                   'ΕΦΕΞΗΣ', 'ΕΧΕΙ', 'ΕΧΕΙΣ', 'ΕΧΕΤΕ', 'ΕΧΘΕΣ', 'ΕΧΟΜΕ', 'ΕΧΟΥΜΕ', 'ΕΧΟΥΝ', 'ΕΧΤΕΣ', 'ΕΧΩ', 'ΕΩΣ', 'Η', 'Η∆Η', 'ΗΜΑΣΤΑΝ', 'ΗΜΑΣΤΕ',
                   'ΗΜΟΥΝ', 'ΗΣΑΣΤΑΝ', 'ΗΣΑΣΤΕ', 'ΗΣΟΥΝ', 'ΗΤΑΝ', 'ΗΤΑΝΕ', 'ΗΤΟΙ', 'ΗΤΤΟΝ', 'ΘΑ', 'Ι', 'Ι∆ΙΑ', 'Ι∆ΙΑΝ', 'Ι∆ΙΑΣ', 'Ι∆ΙΕΣ', 'Ι∆ΙΟ',
                   'Ι∆ΙΟΙ', 'Ι∆ΙΟΝ', 'Ι∆ΙΟΣ', 'Ι∆ΙΟΥ', 'Ι∆ΙΟΥΣ', 'Ι∆ΙΩΝ', 'Ι∆ΙΩΣ', 'ΙΙ', 'ΙΙΙ', 'ΙΣΑΜΕ', 'ΙΣΙΑ', 'ΙΣΩΣ', 'ΚΑΘΕ', 'ΚΑΘΕΜΙΑ',
                   'ΚΑΘΕΜΙΑΣ', 'ΚΑΘΕΝΑ', 'ΚΑΘΕΝΑΣ', 'ΚΑΘΕΝΟΣ', 'ΚΑΘΕΤΙ', 'ΚΑΘΟΛΟΥ', 'ΚΑΘΩΣ', 'ΚΑΙ', 'ΚΑΚΑ', 'ΚΑΚΩΣ', 'ΚΑΛΑ', 'ΚΑΛΩΣ', 'ΚΑΜΙΑ',
                   'ΚΑΜΙΑΝ', 'ΚΑΜΙΑΣ', 'ΚΑΜΠΟΣΑ', 'ΚΑΜΠΟΣΕΣ', 'ΚΑΜΠΟΣΗ', 'ΚΑΜΠΟΣΗΝ', 'ΚΑΜΠΟΣΗΣ', 'ΚΑΜΠΟΣΟ', 'ΚΑΜΠΟΣΟΙ', 'ΚΑΜΠΟΣΟΝ', 'ΚΑΜΠΟΣΟΣ',
                   'ΚΑΜΠΟΣΟΥ', 'ΚΑΜΠΟΣΟΥΣ', 'ΚΑΜΠΟΣΩΝ', 'ΚΑΝΕΙΣ', 'ΚΑΝΕΝ', 'ΚΑΝΕΝΑ', 'ΚΑΝΕΝΑΝ', 'ΚΑΝΕΝΑΣ', 'ΚΑΝΕΝΟΣ', 'ΚΑΠΟΙΑ', 'ΚΑΠΟΙΑΝ', 'ΚΑΠΟΙΑΣ',
                   'ΚΑΠΟΙΕΣ', 'ΚΑΠΟΙΟ', 'ΚΑΠΟΙΟΙ', 'ΚΑΠΟΙΟΝ', 'ΚΑΠΟΙΟΣ', 'ΚΑΠΟΙΟΥ', 'ΚΑΠΟΙΟΥΣ', 'ΚΑΠΟΙΩΝ', 'ΚΑΠΟΤΕ', 'ΚΑΠΟΥ', 'ΚΑΠΩΣ', 'ΚΑΤ', 'ΚΑΤΑ',
                   'ΚΑΤΙ', 'ΚΑΤΙΤΙ', 'ΚΑΤΟΠΙΝ', 'ΚΑΤΩ', 'ΚΙΟΛΑΣ', 'ΚΛΠ', 'ΚΟΝΤΑ', 'ΚΤΛ', 'ΚΥΡΙΩΣ', 'ΛΙΓΑΚΙ', 'ΛΙΓΟ', 'ΛΙΓΩΤΕΡΟ', 'ΛΟΓΩ', 'ΛΟΙΠΑ',
                   'ΛΟΙΠΟΝ', 'ΜΑ', 'ΜΑΖΙ', 'ΜΑΚΑΡΙ', 'ΜΑΚΡΥΑ', 'ΜΑΛΙΣΤΑ', 'ΜΑΛΛΟΝ', 'ΜΑΣ', 'ΜΕ', 'ΜΕΘΑΥΡΙΟ', 'ΜΕΙΟΝ', 'ΜΕΛΕΙ', 'ΜΕΛΛΕΤΑΙ', 'ΜΕΜΙΑΣ',
                   'ΜΕΝ', 'ΜΕΡΙΚΑ', 'ΜΕΡΙΚΕΣ', 'ΜΕΡΙΚΟΙ', 'ΜΕΡΙΚΟΥΣ', 'ΜΕΡΙΚΩΝ', 'ΜΕΣΑ', 'ΜΕΤ', 'ΜΕΤΑ', 'ΜΕΤΑΞΥ', 'ΜΕΧΡΙ', 'ΜΗ', 'ΜΗ∆Ε', 'ΜΗΝ',
                   'ΜΗΠΩΣ', 'ΜΗΤΕ', 'ΜΙΑ', 'ΜΙΑΝ', 'ΜΙΑΣ', 'ΜΟΛΙΣ', 'ΜΟΛΟΝΟΤΙ', 'ΜΟΝΑΧΑ', 'ΜΟΝΕΣ', 'ΜΟΝΗ', 'ΜΟΝΗΝ', 'ΜΟΝΗΣ', 'ΜΟΝΟ', 'ΜΟΝΟΙ',
                   'ΜΟΝΟΜΙΑΣ', 'ΜΟΝΟΣ', 'ΜΟΝΟΥ', 'ΜΟΝΟΥΣ', 'ΜΟΝΩΝ', 'ΜΟΥ', 'ΜΠΟΡΕΙ', 'ΜΠΟΡΟΥΝ', 'ΜΠΡΑΒΟ', 'ΜΠΡΟΣ', 'ΝΑ', 'ΝΑΙ', 'ΝΩΡΙΣ', 'ΞΑΝΑ',
                   'ΞΑΦΝΙΚΑ', 'Ο', 'ΟΙ', 'ΟΛΑ', 'ΟΛΕΣ', 'ΟΛΗ', 'ΟΛΗΝ', 'ΟΛΗΣ', 'ΟΛΟ', 'ΟΛΟΓΥΡΑ', 'ΟΛΟΙ', 'ΟΛΟΝ', 'ΟΛΟΝΕΝ', 'ΟΛΟΣ', 'ΟΛΟΤΕΛΑ', 'ΟΛΟΥ',
                   'ΟΛΟΥΣ', 'ΟΛΩΝ', 'ΟΛΩΣ', 'ΟΛΩΣ∆ΙΟΛΟΥ', 'ΟΜΩΣ', 'ΟΠΟΙΑ', 'ΟΠΟΙΑ∆ΗΠΟΤΕ', 'ΟΠΟΙΑΝ', 'ΟΠΟΙΑΝ∆ΗΠΟΤΕ', 'ΟΠΟΙΑΣ', 'ΟΠΟΙΑΣ∆ΗΠΟΤΕ',
                   'ΟΠΟΙ∆ΗΠΟΤΕ', 'ΟΠΟΙΕΣ', 'ΟΠΟΙΕΣ∆ΗΠΟΤΕ', 'ΟΠΟΙΟ', 'ΟΠΟΙΟ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΙ', 'ΟΠΟΙΟΝ', 'ΟΠΟΙΟΝ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΣ', 'ΟΠΟΙΟΣ∆ΗΠΟΤΕ',
                   'ΟΠΟΙΟΥ', 'ΟΠΟΙΟΥ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΥΣ', 'ΟΠΟΙΟΥΣ∆ΗΠΟΤΕ', 'ΟΠΟΙΩΝ', 'ΟΠΟΙΩΝ∆ΗΠΟΤΕ', 'ΟΠΟΤΕ', 'ΟΠΟΤΕ∆ΗΠΟΤΕ', 'ΟΠΟΥ', 'ΟΠΟΥ∆ΗΠΟΤΕ',
                   'ΟΠΩΣ', 'ΟΡΙΣΜΕΝΑ', 'ΟΡΙΣΜΕΝΕΣ', 'ΟΡΙΣΜΕΝΩΝ', 'ΟΡΙΣΜΕΝΩΣ', 'ΟΣΑ', 'ΟΣΑ∆ΗΠΟΤΕ', 'ΟΣΕΣ', 'ΟΣΕΣ∆ΗΠΟΤΕ', 'ΟΣΗ', 'ΟΣΗ∆ΗΠΟΤΕ', 'ΟΣΗΝ',
                   'ΟΣΗΝ∆ΗΠΟΤΕ', 'ΟΣΗΣ', 'ΟΣΗΣ∆ΗΠΟΤΕ', 'ΟΣΟ', 'ΟΣΟ∆ΗΠΟΤΕ', 'ΟΣΟΙ', 'ΟΣΟΙ∆ΗΠΟΤΕ', 'ΟΣΟΝ', 'ΟΣΟΝ∆ΗΠΟΤΕ', 'ΟΣΟΣ', 'ΟΣΟΣ∆ΗΠΟΤΕ', 'ΟΣΟΥ',
                   'ΟΣΟΥ∆ΗΠΟΤΕ', 'ΟΣΟΥΣ', 'ΟΣΟΥΣ∆ΗΠΟΤΕ', 'ΟΣΩΝ', 'ΟΣΩΝ∆ΗΠΟΤΕ', 'ΟΤΑΝ', 'ΟΤΙ', 'ΟΤΙ∆ΗΠΟΤΕ', 'ΟΤΟΥ', 'ΟΥ', 'ΟΥ∆Ε', 'ΟΥΤΕ', 'ΟΧΙ',
                   'ΠΑΛΙ', 'ΠΑΝΤΟΤΕ', 'ΠΑΝΤΟΥ', 'ΠΑΝΤΩΣ', 'ΠΑΡΑ', 'ΠΕΡΑ', 'ΠΕΡΙ', 'ΠΕΡΙΠΟΥ', 'ΠΕΡΙΣΣΟΤΕΡΟ', 'ΠΕΡΣΙ', 'ΠΕΡΥΣΙ', 'ΠΙΑ', 'ΠΙΘΑΝΟΝ',
                   'ΠΙΟ', 'ΠΙΣΩ', 'ΠΛΑΙ', 'ΠΛΕΟΝ', 'ΠΛΗΝ', 'ΠΟΙΑ', 'ΠΟΙΑΝ', 'ΠΟΙΑΣ', 'ΠΟΙΕΣ', 'ΠΟΙΟ', 'ΠΟΙΟΙ', 'ΠΟΙΟΝ', 'ΠΟΙΟΣ', 'ΠΟΙΟΥ', 'ΠΟΙΟΥΣ',
                   'ΠΟΙΩΝ', 'ΠΟΛΥ', 'ΠΟΣΕΣ', 'ΠΟΣΗ', 'ΠΟΣΗΝ', 'ΠΟΣΗΣ', 'ΠΟΣΟΙ', 'ΠΟΣΟΣ', 'ΠΟΣΟΥΣ', 'ΠΟΤΕ', 'ΠΟΥ', 'ΠΟΥΘΕ', 'ΠΟΥΘΕΝΑ', 'ΠΡΕΠΕΙ',
                   'ΠΡΙΝ', 'ΠΡΟ', 'ΠΡΟΚΕΙΜΕΝΟΥ', 'ΠΡΟΚΕΙΤΑΙ', 'ΠΡΟΠΕΡΣΙ', 'ΠΡΟΣ', 'ΠΡΟΤΟΥ', 'ΠΡΟΧΘΕΣ', 'ΠΡΟΧΤΕΣ', 'ΠΡΩΤΥΤΕΡΑ', 'ΠΩΣ', 'ΣΑΝ', 'ΣΑΣ',
                   'ΣΕ', 'ΣΕΙΣ', 'ΣΗΜΕΡΑ', 'ΣΙΓΑ', 'ΣΟΥ', 'ΣΤΑ', 'ΣΤΗ', 'ΣΤΗΝ', 'ΣΤΗΣ', 'ΣΤΙΣ', 'ΣΤΟ', 'ΣΤΟΝ', 'ΣΤΟΥ', 'ΣΤΟΥΣ', 'ΣΤΩΝ', 'ΣΥΓΧΡΟΝΩΣ',
                   'ΣΥΝ', 'ΣΥΝΑΜΑ', 'ΣΥΝΕΠΩΣ', 'ΣΥΝΗΘΩΣ', 'ΣΥΧΝΑ', 'ΣΥΧΝΑΣ', 'ΣΥΧΝΕΣ', 'ΣΥΧΝΗ', 'ΣΥΧΝΗΝ', 'ΣΥΧΝΗΣ', 'ΣΥΧΝΟ', 'ΣΥΧΝΟΙ', 'ΣΥΧΝΟΝ',
                   'ΣΥΧΝΟΣ', 'ΣΥΧΝΟΥ', 'ΣΥΧΝΟΥ', 'ΣΥΧΝΟΥΣ', 'ΣΥΧΝΩΝ', 'ΣΥΧΝΩΣ', 'ΣΧΕ∆ΟΝ', 'ΣΩΣΤΑ', 'ΤΑ', 'ΤΑ∆Ε', 'ΤΑΥΤΑ', 'ΤΑΥΤΕΣ', 'ΤΑΥΤΗ', 'ΤΑΥΤΗΝ',
                   'ΤΑΥΤΗΣ', 'ΤΑΥΤΟ,ΤΑΥΤΟΝ', 'ΤΑΥΤΟΣ', 'ΤΑΥΤΟΥ', 'ΤΑΥΤΩΝ', 'ΤΑΧΑ', 'ΤΑΧΑΤΕ', 'ΤΕΛΙΚΑ', 'ΤΕΛΙΚΩΣ', 'ΤΕΣ', 'ΤΕΤΟΙΑ', 'ΤΕΤΟΙΑΝ',
                   'ΤΕΤΟΙΑΣ', 'ΤΕΤΟΙΕΣ', 'ΤΕΤΟΙΟ', 'ΤΕΤΟΙΟΙ', 'ΤΕΤΟΙΟΝ', 'ΤΕΤΟΙΟΣ', 'ΤΕΤΟΙΟΥ', 'ΤΕΤΟΙΟΥΣ', 'ΤΕΤΟΙΩΝ', 'ΤΗ', 'ΤΗΝ', 'ΤΗΣ', 'ΤΙ',
                   'ΤΙΠΟΤΑ', 'ΤΙΠΟΤΕ', 'ΤΙΣ', 'ΤΟ', 'ΤΟΙ', 'ΤΟΝ', 'ΤΟΣ', 'ΤΟΣΑ', 'ΤΟΣΕΣ', 'ΤΟΣΗ', 'ΤΟΣΗΝ', 'ΤΟΣΗΣ', 'ΤΟΣΟ', 'ΤΟΣΟΙ', 'ΤΟΣΟΝ', 'ΤΟΣΟΣ',
                   'ΤΟΣΟΥ', 'ΤΟΣΟΥΣ', 'ΤΟΣΩΝ', 'ΤΟΤΕ', 'ΤΟΥ', 'ΤΟΥΛΑΧΙΣΤΟ', 'ΤΟΥΛΑΧΙΣΤΟΝ', 'ΤΟΥΣ', 'ΤΟΥΤΑ', 'ΤΟΥΤΕΣ', 'ΤΟΥΤΗ', 'ΤΟΥΤΗΝ', 'ΤΟΥΤΗΣ',
                   'ΤΟΥΤΟ', 'ΤΟΥΤΟΙ', 'ΤΟΥΤΟΙΣ', 'ΤΟΥΤΟΝ', 'ΤΟΥΤΟΣ', 'ΤΟΥΤΟΥ', 'ΤΟΥΤΟΥΣ', 'ΤΟΥΤΩΝ', 'ΤΥΧΟΝ', 'ΤΩΝ', 'ΤΩΡΑ', 'ΥΠ', 'ΥΠΕΡ', 'ΥΠΟ',
                   'ΥΠΟΨΗ', 'ΥΠΟΨΙΝ', 'ΥΣΤΕΡΑ', 'ΦΕΤΟΣ', 'ΧΑΜΗΛΑ', 'ΧΘΕΣ', 'ΧΤΕΣ', 'ΧΩΡΙΣ', 'ΧΩΡΙΣΤΑ', 'ΨΗΛΑ', 'Ω', 'ΩΡΑΙΑ', 'ΩΣ', 'ΩΣΑΝ', 'ΩΣΟΤΟΥ',
                   'ΩΣΠΟΥ', 'ΩΣΤΕ', 'ΩΣΤΟΣΟ', 'ΩΧ']


def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# Prepare data
X_train = train_c["text"]
y_train = train_c["label"]
X_val = val_c["text"]
y_val = val_c["label"]
X_test = test_c["text"]
y_test = test_c["label"]

for df in [train_c]:
    df["text"] = df["text"].astype(str).apply(clean_text)


val_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words=greek_stopwords,
        lowercase=True,
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.95
    )),
    ('clf', LinearSVC())
])

val_pipeline.fit(X_train,y_train)

# Evaluate on Validation Set

y_pred_val = val_pipeline.predict(X_val)
print("\nTest Results:\n")
print(classification_report(y_val, y_pred_val))


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'εαν', 'εαυτο', 'εαυτον', 'εαυτου', 'εαυτους', 'εαυτων', 'εγκαιρα', 'εγκαιρως', 'εγω', 'ει', 'ειθε', 'ειμαι', 'ειμαστε', 'εινα', 'ειναι', 'εις', 


Test Results:

              precision    recall  f1-score   support

           0       0.92      0.94      0.93        49
           1       0.77      0.81      0.79        21
           2       1.00      1.00      1.00        12
           3       0.83      0.79      0.81        19
           4       1.00      1.00      1.00         1
           5       0.80      0.86      0.83        28
           6       0.72      0.68      0.70        31
           7       0.62      0.59      0.60        22
           8       0.76      0.79      0.77        67
           9       0.67      0.67      0.67         3
          10       1.00      0.62      0.77         8
          11       0.93      0.81      0.86        31
          12       0.00      0.00      0.00         1
          14       0.68      0.78      0.72        27
          15       0.71      0.83      0.77         6
          16       0.40      0.67      0.50         3
          17       0.97      0.95      0.96        40
          1

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [ ]:


import pandas as pd
import re
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


# Text Preprocessing


greek_stopwords = ['Α∆ΙΑΚΟΠΑ', 'ΑΙ', 'ΑΚΟΜΑ', 'ΑΚΟΜΗ', 'ΑΚΡΙΒΩΣ', 'ΑΛΗΘΕΙΑ', 'ΑΛΗΘΙΝΑ', 'ΑΛΛΑ', 'ΑΛΛΑΧΟΥ', 'ΑΛΛΕΣ', 'ΑΛΛΗ', 'ΑΛΛΗΝ', 'ΑΛΛΗΣ',
                   'ΑΛΛΙΩΣ', 'ΑΛΛΙΩΤΙΚΑ', 'ΑΛΛΟ', 'ΑΛΛΟΙ', 'ΑΛΛΟΙΩΣ', 'ΑΛΛΟΙΩΤΙΚΑ', 'ΑΛΛΟΝ', 'ΑΛΛΟΣ', 'ΑΛΛΟΤΕ', 'ΑΛΛΟΥ', 'ΑΛΛΟΥΣ', 'ΑΛΛΩΝ', 'ΑΜΑ',
                   'ΑΜΕΣΑ', 'ΑΜΕΣΩΣ', 'ΑΝ', 'ΑΝΑ', 'ΑΝΑΜΕΣΑ', 'ΑΝΑΜΕΤΑΞΥ', 'ΑΝΕΥ', 'ΑΝΤΙ', 'ΑΝΤΙΠΕΡΑ', 'ΑΝΤΙΣ', 'ΑΝΩ', 'ΑΝΩΤΕΡΩ', 'ΑΞΑΦΝΑ', 'ΑΠ',
                   'ΑΠΕΝΑΝΤΙ', 'ΑΠΟ', 'ΑΠΟΨΕ', 'ΑΡΑ', 'ΑΡΑΓΕ', 'ΑΡΓΑ', 'ΑΡΓΟΤΕΡΟ', 'ΑΡΙΣΤΕΡΑ', 'ΑΡΚΕΤΑ', 'ΑΡΧΙΚΑ', 'ΑΣ', 'ΑΥΡΙΟ', 'ΑΥΤΑ', 'ΑΥΤΕΣ',
                   'ΑΥΤΗ', 'ΑΥΤΗΝ', 'ΑΥΤΗΣ', 'ΑΥΤΟ', 'ΑΥΤΟΙ', 'ΑΥΤΟΝ', 'ΑΥΤΟΣ', 'ΑΥΤΟΥ', 'ΑΥΤΟΥΣ', 'ΑΥΤΩΝ', 'ΑΦΟΤΟΥ', 'ΑΦΟΥ', 'ΒΕΒΑΙΑ', 'ΒΕΒΑΙΟΤΑΤΑ',
                   'ΓΙ', 'ΓΙΑ', 'ΓΡΗΓΟΡΑ', 'ΓΥΡΩ', '∆Α', '∆Ε', '∆ΕΙΝΑ', '∆ΕΝ', '∆ΕΞΙΑ', '∆ΗΘΕΝ', '∆ΗΛΑ∆Η', '∆Ι', '∆ΙΑ', '∆ΙΑΡΚΩΣ', '∆ΙΚΑ', '∆ΙΚΟ',
                   '∆ΙΚΟΙ', '∆ΙΚΟΣ', '∆ΙΚΟΥ', '∆ΙΚΟΥΣ', '∆ΙΟΛΟΥ', '∆ΙΠΛΑ', '∆ΙΧΩΣ', 'ΕΑΝ', 'ΕΑΥΤΟ', 'ΕΑΥΤΟΝ', 'ΕΑΥΤΟΥ', 'ΕΑΥΤΟΥΣ', 'ΕΑΥΤΩΝ',
                   'ΕΓΚΑΙΡΑ', 'ΕΓΚΑΙΡΩΣ', 'ΕΓΩ', 'Ε∆Ω', 'ΕΙ∆ΕΜΗ', 'ΕΙΘΕ', 'ΕΙΜΑΙ', 'ΕΙΜΑΣΤΕ', 'ΕΙΝΑΙ', 'ΕΙΣ', 'ΕΙΣΑΙ', 'ΕΙΣΑΣΤΕ', 'ΕΙΣΤΕ', 'ΕΙΤΕ',
                   'ΕΙΧΑ', 'ΕΙΧΑΜΕ', 'ΕΙΧΑΝ', 'ΕΙΧΑΤΕ', 'ΕΙΧΕ', 'ΕΙΧΕΣ', 'ΕΚΑΣΤΑ', 'ΕΚΑΣΤΕΣ', 'ΕΚΑΣΤΗ', 'ΕΚΑΣΤΗΝ', 'ΕΚΑΣΤΗΣ', 'ΕΚΑΣΤΟ', 'ΕΚΑΣΤΟΙ',
                   'ΕΚΑΣΤΟΝ', 'ΕΚΑΣΤΟΣ', 'ΕΚΑΣΤΟΥ', 'ΕΚΑΣΤΟΥΣ', 'ΕΚΑΣΤΩΝ', 'ΕΚΕΙ', 'ΕΚΕΙΝΑ', 'ΕΚΕΙΝΕΣ', 'ΕΚΕΙΝΗ', 'ΕΚΕΙΝΗΝ', 'ΕΚΕΙΝΗΣ', 'ΕΚΕΙΝΟ',
                   'ΕΚΕΙΝΟΙ', 'ΕΚΕΙΝΟΝ', 'ΕΚΕΙΝΟΣ', 'ΕΚΕΙΝΟΥ', 'ΕΚΕΙΝΟΥΣ', 'ΕΚΕΙΝΩΝ', 'ΕΚΤΟΣ', 'ΕΜΑΣ', 'ΕΜΕΙΣ', 'ΕΜΕΝΑ', 'ΕΜΠΡΟΣ', 'ΕΝ', 'ΕΝΑ',
                   'ΕΝΑΝ', 'ΕΝΑΣ', 'ΕΝΟΣ', 'ΕΝΤΕΛΩΣ', 'ΕΝΤΟΣ', 'ΕΝΤΩΜΕΤΑΞΥ', 'ΕΝΩ', 'ΕΞ', 'ΕΞΑΦΝΑ', 'ΕΞΗΣ', 'ΕΞΙΣΟΥ', 'ΕΞΩ', 'ΕΠΑΝΩ', 'ΕΠΕΙ∆Η',
                   'ΕΠΕΙΤΑ', 'ΕΠΙ', 'ΕΠΙΣΗΣ', 'ΕΠΟΜΕΝΩΣ', 'ΕΣΑΣ', 'ΕΣΕΙΣ', 'ΕΣΕΝΑ', 'ΕΣΤΩ', 'ΕΣΥ', 'ΕΤΕΡΑ', 'ΕΤΕΡΑΙ', 'ΕΤΕΡΑΣ', 'ΕΤΕΡΕΣ', 'ΕΤΕΡΗ',
                   'ΕΤΕΡΗΣ', 'ΕΤΕΡΟ', 'ΕΤΕΡΟΙ', 'ΕΤΕΡΟΝ', 'ΕΤΕΡΟΣ', 'ΕΤΕΡΟΥ', 'ΕΤΕΡΟΥΣ', 'ΕΤΕΡΩΝ', 'ΕΤΟΥΤΑ', 'ΕΤΟΥΤΕΣ', 'ΕΤΟΥΤΗ', 'ΕΤΟΥΤΗΝ',
                   'ΕΤΟΥΤΗΣ', 'ΕΤΟΥΤΟ', 'ΕΤΟΥΤΟΙ', 'ΕΤΟΥΤΟΝ', 'ΕΤΟΥΤΟΣ', 'ΕΤΟΥΤΟΥ', 'ΕΤΟΥΤΟΥΣ', 'ΕΤΟΥΤΩΝ', 'ΕΤΣΙ', 'ΕΥΓΕ', 'ΕΥΘΥΣ', 'ΕΥΤΥΧΩΣ',
                   'ΕΦΕΞΗΣ', 'ΕΧΕΙ', 'ΕΧΕΙΣ', 'ΕΧΕΤΕ', 'ΕΧΘΕΣ', 'ΕΧΟΜΕ', 'ΕΧΟΥΜΕ', 'ΕΧΟΥΝ', 'ΕΧΤΕΣ', 'ΕΧΩ', 'ΕΩΣ', 'Η', 'Η∆Η', 'ΗΜΑΣΤΑΝ', 'ΗΜΑΣΤΕ',
                   'ΗΜΟΥΝ', 'ΗΣΑΣΤΑΝ', 'ΗΣΑΣΤΕ', 'ΗΣΟΥΝ', 'ΗΤΑΝ', 'ΗΤΑΝΕ', 'ΗΤΟΙ', 'ΗΤΤΟΝ', 'ΘΑ', 'Ι', 'Ι∆ΙΑ', 'Ι∆ΙΑΝ', 'Ι∆ΙΑΣ', 'Ι∆ΙΕΣ', 'Ι∆ΙΟ',
                   'Ι∆ΙΟΙ', 'Ι∆ΙΟΝ', 'Ι∆ΙΟΣ', 'Ι∆ΙΟΥ', 'Ι∆ΙΟΥΣ', 'Ι∆ΙΩΝ', 'Ι∆ΙΩΣ', 'ΙΙ', 'ΙΙΙ', 'ΙΣΑΜΕ', 'ΙΣΙΑ', 'ΙΣΩΣ', 'ΚΑΘΕ', 'ΚΑΘΕΜΙΑ',
                   'ΚΑΘΕΜΙΑΣ', 'ΚΑΘΕΝΑ', 'ΚΑΘΕΝΑΣ', 'ΚΑΘΕΝΟΣ', 'ΚΑΘΕΤΙ', 'ΚΑΘΟΛΟΥ', 'ΚΑΘΩΣ', 'ΚΑΙ', 'ΚΑΚΑ', 'ΚΑΚΩΣ', 'ΚΑΛΑ', 'ΚΑΛΩΣ', 'ΚΑΜΙΑ',
                   'ΚΑΜΙΑΝ', 'ΚΑΜΙΑΣ', 'ΚΑΜΠΟΣΑ', 'ΚΑΜΠΟΣΕΣ', 'ΚΑΜΠΟΣΗ', 'ΚΑΜΠΟΣΗΝ', 'ΚΑΜΠΟΣΗΣ', 'ΚΑΜΠΟΣΟ', 'ΚΑΜΠΟΣΟΙ', 'ΚΑΜΠΟΣΟΝ', 'ΚΑΜΠΟΣΟΣ',
                   'ΚΑΜΠΟΣΟΥ', 'ΚΑΜΠΟΣΟΥΣ', 'ΚΑΜΠΟΣΩΝ', 'ΚΑΝΕΙΣ', 'ΚΑΝΕΝ', 'ΚΑΝΕΝΑ', 'ΚΑΝΕΝΑΝ', 'ΚΑΝΕΝΑΣ', 'ΚΑΝΕΝΟΣ', 'ΚΑΠΟΙΑ', 'ΚΑΠΟΙΑΝ', 'ΚΑΠΟΙΑΣ',
                   'ΚΑΠΟΙΕΣ', 'ΚΑΠΟΙΟ', 'ΚΑΠΟΙΟΙ', 'ΚΑΠΟΙΟΝ', 'ΚΑΠΟΙΟΣ', 'ΚΑΠΟΙΟΥ', 'ΚΑΠΟΙΟΥΣ', 'ΚΑΠΟΙΩΝ', 'ΚΑΠΟΤΕ', 'ΚΑΠΟΥ', 'ΚΑΠΩΣ', 'ΚΑΤ', 'ΚΑΤΑ',
                   'ΚΑΤΙ', 'ΚΑΤΙΤΙ', 'ΚΑΤΟΠΙΝ', 'ΚΑΤΩ', 'ΚΙΟΛΑΣ', 'ΚΛΠ', 'ΚΟΝΤΑ', 'ΚΤΛ', 'ΚΥΡΙΩΣ', 'ΛΙΓΑΚΙ', 'ΛΙΓΟ', 'ΛΙΓΩΤΕΡΟ', 'ΛΟΓΩ', 'ΛΟΙΠΑ',
                   'ΛΟΙΠΟΝ', 'ΜΑ', 'ΜΑΖΙ', 'ΜΑΚΑΡΙ', 'ΜΑΚΡΥΑ', 'ΜΑΛΙΣΤΑ', 'ΜΑΛΛΟΝ', 'ΜΑΣ', 'ΜΕ', 'ΜΕΘΑΥΡΙΟ', 'ΜΕΙΟΝ', 'ΜΕΛΕΙ', 'ΜΕΛΛΕΤΑΙ', 'ΜΕΜΙΑΣ',
                   'ΜΕΝ', 'ΜΕΡΙΚΑ', 'ΜΕΡΙΚΕΣ', 'ΜΕΡΙΚΟΙ', 'ΜΕΡΙΚΟΥΣ', 'ΜΕΡΙΚΩΝ', 'ΜΕΣΑ', 'ΜΕΤ', 'ΜΕΤΑ', 'ΜΕΤΑΞΥ', 'ΜΕΧΡΙ', 'ΜΗ', 'ΜΗ∆Ε', 'ΜΗΝ',
                   'ΜΗΠΩΣ', 'ΜΗΤΕ', 'ΜΙΑ', 'ΜΙΑΝ', 'ΜΙΑΣ', 'ΜΟΛΙΣ', 'ΜΟΛΟΝΟΤΙ', 'ΜΟΝΑΧΑ', 'ΜΟΝΕΣ', 'ΜΟΝΗ', 'ΜΟΝΗΝ', 'ΜΟΝΗΣ', 'ΜΟΝΟ', 'ΜΟΝΟΙ',
                   'ΜΟΝΟΜΙΑΣ', 'ΜΟΝΟΣ', 'ΜΟΝΟΥ', 'ΜΟΝΟΥΣ', 'ΜΟΝΩΝ', 'ΜΟΥ', 'ΜΠΟΡΕΙ', 'ΜΠΟΡΟΥΝ', 'ΜΠΡΑΒΟ', 'ΜΠΡΟΣ', 'ΝΑ', 'ΝΑΙ', 'ΝΩΡΙΣ', 'ΞΑΝΑ',
                   'ΞΑΦΝΙΚΑ', 'Ο', 'ΟΙ', 'ΟΛΑ', 'ΟΛΕΣ', 'ΟΛΗ', 'ΟΛΗΝ', 'ΟΛΗΣ', 'ΟΛΟ', 'ΟΛΟΓΥΡΑ', 'ΟΛΟΙ', 'ΟΛΟΝ', 'ΟΛΟΝΕΝ', 'ΟΛΟΣ', 'ΟΛΟΤΕΛΑ', 'ΟΛΟΥ',
                   'ΟΛΟΥΣ', 'ΟΛΩΝ', 'ΟΛΩΣ', 'ΟΛΩΣ∆ΙΟΛΟΥ', 'ΟΜΩΣ', 'ΟΠΟΙΑ', 'ΟΠΟΙΑ∆ΗΠΟΤΕ', 'ΟΠΟΙΑΝ', 'ΟΠΟΙΑΝ∆ΗΠΟΤΕ', 'ΟΠΟΙΑΣ', 'ΟΠΟΙΑΣ∆ΗΠΟΤΕ',
                   'ΟΠΟΙ∆ΗΠΟΤΕ', 'ΟΠΟΙΕΣ', 'ΟΠΟΙΕΣ∆ΗΠΟΤΕ', 'ΟΠΟΙΟ', 'ΟΠΟΙΟ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΙ', 'ΟΠΟΙΟΝ', 'ΟΠΟΙΟΝ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΣ', 'ΟΠΟΙΟΣ∆ΗΠΟΤΕ',
                   'ΟΠΟΙΟΥ', 'ΟΠΟΙΟΥ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΥΣ', 'ΟΠΟΙΟΥΣ∆ΗΠΟΤΕ', 'ΟΠΟΙΩΝ', 'ΟΠΟΙΩΝ∆ΗΠΟΤΕ', 'ΟΠΟΤΕ', 'ΟΠΟΤΕ∆ΗΠΟΤΕ', 'ΟΠΟΥ', 'ΟΠΟΥ∆ΗΠΟΤΕ',
                   'ΟΠΩΣ', 'ΟΡΙΣΜΕΝΑ', 'ΟΡΙΣΜΕΝΕΣ', 'ΟΡΙΣΜΕΝΩΝ', 'ΟΡΙΣΜΕΝΩΣ', 'ΟΣΑ', 'ΟΣΑ∆ΗΠΟΤΕ', 'ΟΣΕΣ', 'ΟΣΕΣ∆ΗΠΟΤΕ', 'ΟΣΗ', 'ΟΣΗ∆ΗΠΟΤΕ', 'ΟΣΗΝ',
                   'ΟΣΗΝ∆ΗΠΟΤΕ', 'ΟΣΗΣ', 'ΟΣΗΣ∆ΗΠΟΤΕ', 'ΟΣΟ', 'ΟΣΟ∆ΗΠΟΤΕ', 'ΟΣΟΙ', 'ΟΣΟΙ∆ΗΠΟΤΕ', 'ΟΣΟΝ', 'ΟΣΟΝ∆ΗΠΟΤΕ', 'ΟΣΟΣ', 'ΟΣΟΣ∆ΗΠΟΤΕ', 'ΟΣΟΥ',
                   'ΟΣΟΥ∆ΗΠΟΤΕ', 'ΟΣΟΥΣ', 'ΟΣΟΥΣ∆ΗΠΟΤΕ', 'ΟΣΩΝ', 'ΟΣΩΝ∆ΗΠΟΤΕ', 'ΟΤΑΝ', 'ΟΤΙ', 'ΟΤΙ∆ΗΠΟΤΕ', 'ΟΤΟΥ', 'ΟΥ', 'ΟΥ∆Ε', 'ΟΥΤΕ', 'ΟΧΙ',
                   'ΠΑΛΙ', 'ΠΑΝΤΟΤΕ', 'ΠΑΝΤΟΥ', 'ΠΑΝΤΩΣ', 'ΠΑΡΑ', 'ΠΕΡΑ', 'ΠΕΡΙ', 'ΠΕΡΙΠΟΥ', 'ΠΕΡΙΣΣΟΤΕΡΟ', 'ΠΕΡΣΙ', 'ΠΕΡΥΣΙ', 'ΠΙΑ', 'ΠΙΘΑΝΟΝ',
                   'ΠΙΟ', 'ΠΙΣΩ', 'ΠΛΑΙ', 'ΠΛΕΟΝ', 'ΠΛΗΝ', 'ΠΟΙΑ', 'ΠΟΙΑΝ', 'ΠΟΙΑΣ', 'ΠΟΙΕΣ', 'ΠΟΙΟ', 'ΠΟΙΟΙ', 'ΠΟΙΟΝ', 'ΠΟΙΟΣ', 'ΠΟΙΟΥ', 'ΠΟΙΟΥΣ',
                   'ΠΟΙΩΝ', 'ΠΟΛΥ', 'ΠΟΣΕΣ', 'ΠΟΣΗ', 'ΠΟΣΗΝ', 'ΠΟΣΗΣ', 'ΠΟΣΟΙ', 'ΠΟΣΟΣ', 'ΠΟΣΟΥΣ', 'ΠΟΤΕ', 'ΠΟΥ', 'ΠΟΥΘΕ', 'ΠΟΥΘΕΝΑ', 'ΠΡΕΠΕΙ',
                   'ΠΡΙΝ', 'ΠΡΟ', 'ΠΡΟΚΕΙΜΕΝΟΥ', 'ΠΡΟΚΕΙΤΑΙ', 'ΠΡΟΠΕΡΣΙ', 'ΠΡΟΣ', 'ΠΡΟΤΟΥ', 'ΠΡΟΧΘΕΣ', 'ΠΡΟΧΤΕΣ', 'ΠΡΩΤΥΤΕΡΑ', 'ΠΩΣ', 'ΣΑΝ', 'ΣΑΣ',
                   'ΣΕ', 'ΣΕΙΣ', 'ΣΗΜΕΡΑ', 'ΣΙΓΑ', 'ΣΟΥ', 'ΣΤΑ', 'ΣΤΗ', 'ΣΤΗΝ', 'ΣΤΗΣ', 'ΣΤΙΣ', 'ΣΤΟ', 'ΣΤΟΝ', 'ΣΤΟΥ', 'ΣΤΟΥΣ', 'ΣΤΩΝ', 'ΣΥΓΧΡΟΝΩΣ',
                   'ΣΥΝ', 'ΣΥΝΑΜΑ', 'ΣΥΝΕΠΩΣ', 'ΣΥΝΗΘΩΣ', 'ΣΥΧΝΑ', 'ΣΥΧΝΑΣ', 'ΣΥΧΝΕΣ', 'ΣΥΧΝΗ', 'ΣΥΧΝΗΝ', 'ΣΥΧΝΗΣ', 'ΣΥΧΝΟ', 'ΣΥΧΝΟΙ', 'ΣΥΧΝΟΝ',
                   'ΣΥΧΝΟΣ', 'ΣΥΧΝΟΥ', 'ΣΥΧΝΟΥ', 'ΣΥΧΝΟΥΣ', 'ΣΥΧΝΩΝ', 'ΣΥΧΝΩΣ', 'ΣΧΕ∆ΟΝ', 'ΣΩΣΤΑ', 'ΤΑ', 'ΤΑ∆Ε', 'ΤΑΥΤΑ', 'ΤΑΥΤΕΣ', 'ΤΑΥΤΗ', 'ΤΑΥΤΗΝ',
                   'ΤΑΥΤΗΣ', 'ΤΑΥΤΟ,ΤΑΥΤΟΝ', 'ΤΑΥΤΟΣ', 'ΤΑΥΤΟΥ', 'ΤΑΥΤΩΝ', 'ΤΑΧΑ', 'ΤΑΧΑΤΕ', 'ΤΕΛΙΚΑ', 'ΤΕΛΙΚΩΣ', 'ΤΕΣ', 'ΤΕΤΟΙΑ', 'ΤΕΤΟΙΑΝ',
                   'ΤΕΤΟΙΑΣ', 'ΤΕΤΟΙΕΣ', 'ΤΕΤΟΙΟ', 'ΤΕΤΟΙΟΙ', 'ΤΕΤΟΙΟΝ', 'ΤΕΤΟΙΟΣ', 'ΤΕΤΟΙΟΥ', 'ΤΕΤΟΙΟΥΣ', 'ΤΕΤΟΙΩΝ', 'ΤΗ', 'ΤΗΝ', 'ΤΗΣ', 'ΤΙ',
                   'ΤΙΠΟΤΑ', 'ΤΙΠΟΤΕ', 'ΤΙΣ', 'ΤΟ', 'ΤΟΙ', 'ΤΟΝ', 'ΤΟΣ', 'ΤΟΣΑ', 'ΤΟΣΕΣ', 'ΤΟΣΗ', 'ΤΟΣΗΝ', 'ΤΟΣΗΣ', 'ΤΟΣΟ', 'ΤΟΣΟΙ', 'ΤΟΣΟΝ', 'ΤΟΣΟΣ',
                   'ΤΟΣΟΥ', 'ΤΟΣΟΥΣ', 'ΤΟΣΩΝ', 'ΤΟΤΕ', 'ΤΟΥ', 'ΤΟΥΛΑΧΙΣΤΟ', 'ΤΟΥΛΑΧΙΣΤΟΝ', 'ΤΟΥΣ', 'ΤΟΥΤΑ', 'ΤΟΥΤΕΣ', 'ΤΟΥΤΗ', 'ΤΟΥΤΗΝ', 'ΤΟΥΤΗΣ',
                   'ΤΟΥΤΟ', 'ΤΟΥΤΟΙ', 'ΤΟΥΤΟΙΣ', 'ΤΟΥΤΟΝ', 'ΤΟΥΤΟΣ', 'ΤΟΥΤΟΥ', 'ΤΟΥΤΟΥΣ', 'ΤΟΥΤΩΝ', 'ΤΥΧΟΝ', 'ΤΩΝ', 'ΤΩΡΑ', 'ΥΠ', 'ΥΠΕΡ', 'ΥΠΟ',
                   'ΥΠΟΨΗ', 'ΥΠΟΨΙΝ', 'ΥΣΤΕΡΑ', 'ΦΕΤΟΣ', 'ΧΑΜΗΛΑ', 'ΧΘΕΣ', 'ΧΤΕΣ', 'ΧΩΡΙΣ', 'ΧΩΡΙΣΤΑ', 'ΨΗΛΑ', 'Ω', 'ΩΡΑΙΑ', 'ΩΣ', 'ΩΣΑΝ', 'ΩΣΟΤΟΥ',
                   'ΩΣΠΟΥ', 'ΩΣΤΕ', 'ΩΣΤΟΣΟ', 'ΩΧ']


def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


# Prepare data
X_train = train_c["text"]
y_train = train_c["label"]
X_val = val_c["text"]
y_val = val_c["label"]
X_test = test_c["text"]
y_test = test_c["label"]

for df in [train_c]:
    df["text"] = df["text"].astype(str).apply(clean_text)


val_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words=greek_stopwords,
        lowercase=True,
        ngram_range=(1, 2),   # unigrams and bigrams
        min_df=2,             # ignore very rare terms
        max_df=0.95           # ignore too common terms
    )),
    ('clf', LinearSVC(class_weight='balanced'))
])

val_pipeline.fit(X_train,y_train)

# -----------------------------
# Evaluate on Validation Set
# -----------------------------
y_pred_val = val_pipeline.predict(X_val)
print("\nTest Results:\n")
print(classification_report(y_val, y_pred_val))


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'εαν', 'εαυτο', 'εαυτον', 'εαυτου', 'εαυτους', 'εαυτων', 'εγκαιρα', 'εγκαιρως', 'εγω', 'ει', 'ειθε', 'ειμαι', 'ειμαστε', 'εινα', 'ειναι', 'εις', 


Test Results:

              precision    recall  f1-score   support

           0       0.96      0.92      0.94        49
           1       0.71      0.81      0.76        21
           2       1.00      1.00      1.00        12
           3       0.75      0.79      0.77        19
           4       1.00      1.00      1.00         1
           5       0.80      0.86      0.83        28
           6       0.72      0.68      0.70        31
           7       0.59      0.59      0.59        22
           8       0.76      0.78      0.77        67
           9       0.50      0.67      0.57         3
          10       0.86      0.75      0.80         8
          11       0.87      0.87      0.87        31
          12       0.00      0.00      0.00         1
          14       0.66      0.78      0.71        27
          15       0.62      0.83      0.71         6
          16       0.33      0.67      0.44         3
          17       0.97      0.93      0.95        40
          1

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

TEST Chapter TFIDF

In [ ]:
final_chapter_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words=greek_stopwords,
        lowercase=True,
        ngram_range=(1, 2),   # unigrams and bigrams
        min_df=2,             # ignore very rare terms
        max_df=0.95           # ignore too common terms
    )),
    ('clf', LinearSVC(class_weight='balanced'))
])

final_chapter_pipeline.fit(X_train,y_train)

# Evaluate on Test Set

y_pred_chapter_test = final_chapter_pipeline.predict(X_test)
print("\nTest Results:\n")
print(classification_report(y_test, y_pred_chapter_test))

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'εαν', 'εαυτο', 'εαυτον', 'εαυτου', 'εαυτους', 'εαυτων', 'εγκαιρα', 'εγκαιρως', 'εγω', 'ει', 'ειθε', 'ειμαι', 'ειμαστε', 'εινα', 'ειναι', 'εις', 


Test Results:

              precision    recall  f1-score   support

           0       0.89      0.91      0.90        43
           1       0.75      0.82      0.78        22
           2       0.56      0.83      0.67         6
           3       0.68      0.81      0.74        16
           4       1.00      0.50      0.67         2
           5       0.95      0.84      0.89        25
           6       0.47      0.69      0.56        13
           7       0.70      0.63      0.67        30
           8       0.90      0.81      0.85        57
           9       0.33      1.00      0.50         2
          10       1.00      0.71      0.83         7
          11       0.98      0.93      0.95        44
          12       0.00      0.00      0.00         2
          13       1.00      0.50      0.67         2
          14       0.84      0.91      0.87        34
          15       0.67      1.00      0.80         2
          16       0.43      1.00      0.60         3
          1

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

# BOW (bag of words)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_pipeline = Pipeline([
    ('bow', CountVectorizer(
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.95,
        stop_words=greek_stopwords
    )),
    ('clf', LinearSVC(class_weight='balanced'))
])
bow_pipeline.fit(X_train,y_train)

# Evaluate on Validation Set

y_pred_bow = bow_pipeline.predict(X_val)
print("\n📊 Test Results with BoW:\n")
print(classification_report(y_val, y_pred_bow))

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'εαν', 'εαυτο', 'εαυτον', 'εαυτου', 'εαυτους', 'εαυτων', 'εγκαιρα', 'εγκαιρως', 'εγω', 'ει', 'ειθε', 'ειμαι', 'ειμαστε', 'εινα', 'ειναι', 'εις', 


📊 Test Results with BoW:

              precision    recall  f1-score   support

           0       0.75      0.82      0.78        49
           1       0.60      0.57      0.59        21
           2       0.89      0.67      0.76        12
           3       0.75      0.63      0.69        19
           4       0.00      0.00      0.00         1
           5       0.84      0.75      0.79        28
           6       0.65      0.55      0.60        31
           7       0.52      0.55      0.53        22
           8       0.69      0.75      0.72        67
           9       0.33      0.33      0.33         3
          10       1.00      0.38      0.55         8
          11       0.92      0.74      0.82        31
          12       0.00      0.00      0.00         1
          14       0.60      0.67      0.63        27
          15       0.50      0.67      0.57         6
          16       0.17      0.33      0.22         3
          17       0.88      0.90      0.89        40


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_pipeline = Pipeline([
    ('bow', CountVectorizer(
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.95,
        stop_words=greek_stopwords
    )),
    ('clf', LinearSVC())
])
bow_pipeline.fit(X_train,y_train)

# Evaluate

y_pred_bow = bow_pipeline.predict(X_val)
print("\n📊 Test Results with BoW:\n")
print(classification_report(y_val, y_pred_bow))


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'εαν', 'εαυτο', 'εαυτον', 'εαυτου', 'εαυτους', 'εαυτων', 'εγκαιρα', 'εγκαιρως', 'εγω', 'ει', 'ειθε', 'ειμαι', 'ειμαστε', 'εινα', 'ειναι', 'εις', 


📊 Test Results with BoW:

              precision    recall  f1-score   support

           0       0.74      0.82      0.78        49
           1       0.63      0.57      0.60        21
           2       0.89      0.67      0.76        12
           3       0.71      0.63      0.67        19
           4       0.00      0.00      0.00         1
           5       0.84      0.75      0.79        28
           6       0.65      0.55      0.60        31
           7       0.55      0.55      0.55        22
           8       0.68      0.75      0.71        67
           9       0.50      0.33      0.40         3
          10       1.00      0.38      0.55         8
          11       0.92      0.74      0.82        31
          12       0.00      0.00      0.00         1
          14       0.56      0.67      0.61        27
          15       0.57      0.67      0.62         6
          16       0.17      0.33      0.22         3
          17       0.88      0.90      0.89        40


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

TEST CHAPTER BAG OF WORDS

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_final_pipeline = Pipeline([
    ('bow', CountVectorizer(
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.95,
        stop_words=greek_stopwords
    )),
    ('clf', LinearSVC())
])
bow_final_pipeline.fit(X_train,y_train)


# Evaluate on Test Set

y_pred_final_bow = bow_final_pipeline.predict(X_test)
print("\n📊 Test Results with BoW:\n")
print(classification_report(y_test, y_pred_final_bow))


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'εαν', 'εαυτο', 'εαυτον', 'εαυτου', 'εαυτους', 'εαυτων', 'εγκαιρα', 'εγκαιρως', 'εγω', 'ει', 'ειθε', 'ειμαι', 'ειμαστε', 'εινα', 'ειναι', 'εις', 


📊 Test Results with BoW:

              precision    recall  f1-score   support

           0       0.72      0.79      0.76        43
           1       0.55      0.73      0.63        22
           2       0.50      0.50      0.50         6
           3       0.53      0.56      0.55        16
           4       0.00      0.00      0.00         2
           5       0.67      0.56      0.61        25
           6       0.48      0.77      0.59        13
           7       0.74      0.57      0.64        30
           8       0.74      0.61      0.67        57
           9       0.40      1.00      0.57         2
          10       0.83      0.71      0.77         7
          11       0.89      0.91      0.90        44
          12       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         2
          14       0.65      0.76      0.70        34
          15       1.00      0.50      0.67         2
          16       0.38      1.00      0.55         3


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

# Subject


#TFIDF

In [ ]:
import pandas as pd
import re
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


# Text Preprocessing

def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# Prepare Data

X_train = train_s["text"]
y_train = train_s["label"]
X_val = val_s["text"]
y_val = val_s["label"]
X_test = test_s["text"]
y_test = test_s["label"]

for df in [train_s]:
    df["text"] = df["text"].astype(str).apply(clean_text)


# Define and Fit Pipeline

val_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words=greek_stopwords,
        lowercase=True,
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.95
    )),
    ('clf', LinearSVC(class_weight='balanced'))
])

val_pipeline.fit(X_train, y_train)


# Evaluate on Validation Set

print("\n📊 Validation Results (subject level):\n")
y_pred_val = val_pipeline.predict(X_val)
print(classification_report(y_val, y_pred_val, digits=4))




/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'εαν', 'εαυτο', 'εαυτον', 'εαυτου', 'εαυτους', 'εαυτων', 'εγκαιρα', 'εγκαιρως', 'εγω', 'ει', 'ειθε', 'ειμαι', 'ειμαστε', 'εινα', 'ειναι', 'εις', 


📊 Validation Results (subject level):

              precision    recall  f1-score   support

           0     1.0000    0.6364    0.7778        11
           3     0.3333    1.0000    0.5000         1
           4     0.0000    0.0000    0.0000         1
           5     0.0000    0.0000    0.0000         2
           8     0.8182    0.7500    0.7826        12
           9     1.0000    0.7143    0.8333         7
          10     0.0000    0.0000    0.0000         0
          11     0.3333    0.5000    0.4000         2
          12     0.0000    0.0000    0.0000         1
          13     0.8333    1.0000    0.9091        10
          14     1.0000    1.0000    1.0000         1
          15     0.0000    0.0000    0.0000         2
          16     1.0000    1.0000    1.0000         2
          17     1.0000    1.0000    1.0000         9
          19     0.2500    0.2500    0.2500         4
          20     0.7500    1.0000    0.8571         3
          21     0.0000    0.0000    0.00

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [ ]:
#! pip install imbalanced-learn
import pandas as pd
import re
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
greek_stopwords = ['Α∆ΙΑΚΟΠΑ', 'ΑΙ', 'ΑΚΟΜΑ', 'ΑΚΟΜΗ', 'ΑΚΡΙΒΩΣ', 'ΑΛΗΘΕΙΑ', 'ΑΛΗΘΙΝΑ', 'ΑΛΛΑ', 'ΑΛΛΑΧΟΥ', 'ΑΛΛΕΣ', 'ΑΛΛΗ', 'ΑΛΛΗΝ', 'ΑΛΛΗΣ',
                   'ΑΛΛΙΩΣ', 'ΑΛΛΙΩΤΙΚΑ', 'ΑΛΛΟ', 'ΑΛΛΟΙ', 'ΑΛΛΟΙΩΣ', 'ΑΛΛΟΙΩΤΙΚΑ', 'ΑΛΛΟΝ', 'ΑΛΛΟΣ', 'ΑΛΛΟΤΕ', 'ΑΛΛΟΥ', 'ΑΛΛΟΥΣ', 'ΑΛΛΩΝ', 'ΑΜΑ',
                   'ΑΜΕΣΑ', 'ΑΜΕΣΩΣ', 'ΑΝ', 'ΑΝΑ', 'ΑΝΑΜΕΣΑ', 'ΑΝΑΜΕΤΑΞΥ', 'ΑΝΕΥ', 'ΑΝΤΙ', 'ΑΝΤΙΠΕΡΑ', 'ΑΝΤΙΣ', 'ΑΝΩ', 'ΑΝΩΤΕΡΩ', 'ΑΞΑΦΝΑ', 'ΑΠ',
                   'ΑΠΕΝΑΝΤΙ', 'ΑΠΟ', 'ΑΠΟΨΕ', 'ΑΡΑ', 'ΑΡΑΓΕ', 'ΑΡΓΑ', 'ΑΡΓΟΤΕΡΟ', 'ΑΡΙΣΤΕΡΑ', 'ΑΡΚΕΤΑ', 'ΑΡΧΙΚΑ', 'ΑΣ', 'ΑΥΡΙΟ', 'ΑΥΤΑ', 'ΑΥΤΕΣ',
                   'ΑΥΤΗ', 'ΑΥΤΗΝ', 'ΑΥΤΗΣ', 'ΑΥΤΟ', 'ΑΥΤΟΙ', 'ΑΥΤΟΝ', 'ΑΥΤΟΣ', 'ΑΥΤΟΥ', 'ΑΥΤΟΥΣ', 'ΑΥΤΩΝ', 'ΑΦΟΤΟΥ', 'ΑΦΟΥ', 'ΒΕΒΑΙΑ', 'ΒΕΒΑΙΟΤΑΤΑ',
                   'ΓΙ', 'ΓΙΑ', 'ΓΡΗΓΟΡΑ', 'ΓΥΡΩ', '∆Α', '∆Ε', '∆ΕΙΝΑ', '∆ΕΝ', '∆ΕΞΙΑ', '∆ΗΘΕΝ', '∆ΗΛΑ∆Η', '∆Ι', '∆ΙΑ', '∆ΙΑΡΚΩΣ', '∆ΙΚΑ', '∆ΙΚΟ',
                   '∆ΙΚΟΙ', '∆ΙΚΟΣ', '∆ΙΚΟΥ', '∆ΙΚΟΥΣ', '∆ΙΟΛΟΥ', '∆ΙΠΛΑ', '∆ΙΧΩΣ', 'ΕΑΝ', 'ΕΑΥΤΟ', 'ΕΑΥΤΟΝ', 'ΕΑΥΤΟΥ', 'ΕΑΥΤΟΥΣ', 'ΕΑΥΤΩΝ',
                   'ΕΓΚΑΙΡΑ', 'ΕΓΚΑΙΡΩΣ', 'ΕΓΩ', 'Ε∆Ω', 'ΕΙ∆ΕΜΗ', 'ΕΙΘΕ', 'ΕΙΜΑΙ', 'ΕΙΜΑΣΤΕ', 'ΕΙΝΑΙ', 'ΕΙΣ', 'ΕΙΣΑΙ', 'ΕΙΣΑΣΤΕ', 'ΕΙΣΤΕ', 'ΕΙΤΕ',
                   'ΕΙΧΑ', 'ΕΙΧΑΜΕ', 'ΕΙΧΑΝ', 'ΕΙΧΑΤΕ', 'ΕΙΧΕ', 'ΕΙΧΕΣ', 'ΕΚΑΣΤΑ', 'ΕΚΑΣΤΕΣ', 'ΕΚΑΣΤΗ', 'ΕΚΑΣΤΗΝ', 'ΕΚΑΣΤΗΣ', 'ΕΚΑΣΤΟ', 'ΕΚΑΣΤΟΙ',
                   'ΕΚΑΣΤΟΝ', 'ΕΚΑΣΤΟΣ', 'ΕΚΑΣΤΟΥ', 'ΕΚΑΣΤΟΥΣ', 'ΕΚΑΣΤΩΝ', 'ΕΚΕΙ', 'ΕΚΕΙΝΑ', 'ΕΚΕΙΝΕΣ', 'ΕΚΕΙΝΗ', 'ΕΚΕΙΝΗΝ', 'ΕΚΕΙΝΗΣ', 'ΕΚΕΙΝΟ',
                   'ΕΚΕΙΝΟΙ', 'ΕΚΕΙΝΟΝ', 'ΕΚΕΙΝΟΣ', 'ΕΚΕΙΝΟΥ', 'ΕΚΕΙΝΟΥΣ', 'ΕΚΕΙΝΩΝ', 'ΕΚΤΟΣ', 'ΕΜΑΣ', 'ΕΜΕΙΣ', 'ΕΜΕΝΑ', 'ΕΜΠΡΟΣ', 'ΕΝ', 'ΕΝΑ',
                   'ΕΝΑΝ', 'ΕΝΑΣ', 'ΕΝΟΣ', 'ΕΝΤΕΛΩΣ', 'ΕΝΤΟΣ', 'ΕΝΤΩΜΕΤΑΞΥ', 'ΕΝΩ', 'ΕΞ', 'ΕΞΑΦΝΑ', 'ΕΞΗΣ', 'ΕΞΙΣΟΥ', 'ΕΞΩ', 'ΕΠΑΝΩ', 'ΕΠΕΙ∆Η',
                   'ΕΠΕΙΤΑ', 'ΕΠΙ', 'ΕΠΙΣΗΣ', 'ΕΠΟΜΕΝΩΣ', 'ΕΣΑΣ', 'ΕΣΕΙΣ', 'ΕΣΕΝΑ', 'ΕΣΤΩ', 'ΕΣΥ', 'ΕΤΕΡΑ', 'ΕΤΕΡΑΙ', 'ΕΤΕΡΑΣ', 'ΕΤΕΡΕΣ', 'ΕΤΕΡΗ',
                   'ΕΤΕΡΗΣ', 'ΕΤΕΡΟ', 'ΕΤΕΡΟΙ', 'ΕΤΕΡΟΝ', 'ΕΤΕΡΟΣ', 'ΕΤΕΡΟΥ', 'ΕΤΕΡΟΥΣ', 'ΕΤΕΡΩΝ', 'ΕΤΟΥΤΑ', 'ΕΤΟΥΤΕΣ', 'ΕΤΟΥΤΗ', 'ΕΤΟΥΤΗΝ',
                   'ΕΤΟΥΤΗΣ', 'ΕΤΟΥΤΟ', 'ΕΤΟΥΤΟΙ', 'ΕΤΟΥΤΟΝ', 'ΕΤΟΥΤΟΣ', 'ΕΤΟΥΤΟΥ', 'ΕΤΟΥΤΟΥΣ', 'ΕΤΟΥΤΩΝ', 'ΕΤΣΙ', 'ΕΥΓΕ', 'ΕΥΘΥΣ', 'ΕΥΤΥΧΩΣ',
                   'ΕΦΕΞΗΣ', 'ΕΧΕΙ', 'ΕΧΕΙΣ', 'ΕΧΕΤΕ', 'ΕΧΘΕΣ', 'ΕΧΟΜΕ', 'ΕΧΟΥΜΕ', 'ΕΧΟΥΝ', 'ΕΧΤΕΣ', 'ΕΧΩ', 'ΕΩΣ', 'Η', 'Η∆Η', 'ΗΜΑΣΤΑΝ', 'ΗΜΑΣΤΕ',
                   'ΗΜΟΥΝ', 'ΗΣΑΣΤΑΝ', 'ΗΣΑΣΤΕ', 'ΗΣΟΥΝ', 'ΗΤΑΝ', 'ΗΤΑΝΕ', 'ΗΤΟΙ', 'ΗΤΤΟΝ', 'ΘΑ', 'Ι', 'Ι∆ΙΑ', 'Ι∆ΙΑΝ', 'Ι∆ΙΑΣ', 'Ι∆ΙΕΣ', 'Ι∆ΙΟ',
                   'Ι∆ΙΟΙ', 'Ι∆ΙΟΝ', 'Ι∆ΙΟΣ', 'Ι∆ΙΟΥ', 'Ι∆ΙΟΥΣ', 'Ι∆ΙΩΝ', 'Ι∆ΙΩΣ', 'ΙΙ', 'ΙΙΙ', 'ΙΣΑΜΕ', 'ΙΣΙΑ', 'ΙΣΩΣ', 'ΚΑΘΕ', 'ΚΑΘΕΜΙΑ',
                   'ΚΑΘΕΜΙΑΣ', 'ΚΑΘΕΝΑ', 'ΚΑΘΕΝΑΣ', 'ΚΑΘΕΝΟΣ', 'ΚΑΘΕΤΙ', 'ΚΑΘΟΛΟΥ', 'ΚΑΘΩΣ', 'ΚΑΙ', 'ΚΑΚΑ', 'ΚΑΚΩΣ', 'ΚΑΛΑ', 'ΚΑΛΩΣ', 'ΚΑΜΙΑ',
                   'ΚΑΜΙΑΝ', 'ΚΑΜΙΑΣ', 'ΚΑΜΠΟΣΑ', 'ΚΑΜΠΟΣΕΣ', 'ΚΑΜΠΟΣΗ', 'ΚΑΜΠΟΣΗΝ', 'ΚΑΜΠΟΣΗΣ', 'ΚΑΜΠΟΣΟ', 'ΚΑΜΠΟΣΟΙ', 'ΚΑΜΠΟΣΟΝ', 'ΚΑΜΠΟΣΟΣ',
                   'ΚΑΜΠΟΣΟΥ', 'ΚΑΜΠΟΣΟΥΣ', 'ΚΑΜΠΟΣΩΝ', 'ΚΑΝΕΙΣ', 'ΚΑΝΕΝ', 'ΚΑΝΕΝΑ', 'ΚΑΝΕΝΑΝ', 'ΚΑΝΕΝΑΣ', 'ΚΑΝΕΝΟΣ', 'ΚΑΠΟΙΑ', 'ΚΑΠΟΙΑΝ', 'ΚΑΠΟΙΑΣ',
                   'ΚΑΠΟΙΕΣ', 'ΚΑΠΟΙΟ', 'ΚΑΠΟΙΟΙ', 'ΚΑΠΟΙΟΝ', 'ΚΑΠΟΙΟΣ', 'ΚΑΠΟΙΟΥ', 'ΚΑΠΟΙΟΥΣ', 'ΚΑΠΟΙΩΝ', 'ΚΑΠΟΤΕ', 'ΚΑΠΟΥ', 'ΚΑΠΩΣ', 'ΚΑΤ', 'ΚΑΤΑ',
                   'ΚΑΤΙ', 'ΚΑΤΙΤΙ', 'ΚΑΤΟΠΙΝ', 'ΚΑΤΩ', 'ΚΙΟΛΑΣ', 'ΚΛΠ', 'ΚΟΝΤΑ', 'ΚΤΛ', 'ΚΥΡΙΩΣ', 'ΛΙΓΑΚΙ', 'ΛΙΓΟ', 'ΛΙΓΩΤΕΡΟ', 'ΛΟΓΩ', 'ΛΟΙΠΑ',
                   'ΛΟΙΠΟΝ', 'ΜΑ', 'ΜΑΖΙ', 'ΜΑΚΑΡΙ', 'ΜΑΚΡΥΑ', 'ΜΑΛΙΣΤΑ', 'ΜΑΛΛΟΝ', 'ΜΑΣ', 'ΜΕ', 'ΜΕΘΑΥΡΙΟ', 'ΜΕΙΟΝ', 'ΜΕΛΕΙ', 'ΜΕΛΛΕΤΑΙ', 'ΜΕΜΙΑΣ',
                   'ΜΕΝ', 'ΜΕΡΙΚΑ', 'ΜΕΡΙΚΕΣ', 'ΜΕΡΙΚΟΙ', 'ΜΕΡΙΚΟΥΣ', 'ΜΕΡΙΚΩΝ', 'ΜΕΣΑ', 'ΜΕΤ', 'ΜΕΤΑ', 'ΜΕΤΑΞΥ', 'ΜΕΧΡΙ', 'ΜΗ', 'ΜΗ∆Ε', 'ΜΗΝ',
                   'ΜΗΠΩΣ', 'ΜΗΤΕ', 'ΜΙΑ', 'ΜΙΑΝ', 'ΜΙΑΣ', 'ΜΟΛΙΣ', 'ΜΟΛΟΝΟΤΙ', 'ΜΟΝΑΧΑ', 'ΜΟΝΕΣ', 'ΜΟΝΗ', 'ΜΟΝΗΝ', 'ΜΟΝΗΣ', 'ΜΟΝΟ', 'ΜΟΝΟΙ',
                   'ΜΟΝΟΜΙΑΣ', 'ΜΟΝΟΣ', 'ΜΟΝΟΥ', 'ΜΟΝΟΥΣ', 'ΜΟΝΩΝ', 'ΜΟΥ', 'ΜΠΟΡΕΙ', 'ΜΠΟΡΟΥΝ', 'ΜΠΡΑΒΟ', 'ΜΠΡΟΣ', 'ΝΑ', 'ΝΑΙ', 'ΝΩΡΙΣ', 'ΞΑΝΑ',
                   'ΞΑΦΝΙΚΑ', 'Ο', 'ΟΙ', 'ΟΛΑ', 'ΟΛΕΣ', 'ΟΛΗ', 'ΟΛΗΝ', 'ΟΛΗΣ', 'ΟΛΟ', 'ΟΛΟΓΥΡΑ', 'ΟΛΟΙ', 'ΟΛΟΝ', 'ΟΛΟΝΕΝ', 'ΟΛΟΣ', 'ΟΛΟΤΕΛΑ', 'ΟΛΟΥ',
                   'ΟΛΟΥΣ', 'ΟΛΩΝ', 'ΟΛΩΣ', 'ΟΛΩΣ∆ΙΟΛΟΥ', 'ΟΜΩΣ', 'ΟΠΟΙΑ', 'ΟΠΟΙΑ∆ΗΠΟΤΕ', 'ΟΠΟΙΑΝ', 'ΟΠΟΙΑΝ∆ΗΠΟΤΕ', 'ΟΠΟΙΑΣ', 'ΟΠΟΙΑΣ∆ΗΠΟΤΕ',
                   'ΟΠΟΙ∆ΗΠΟΤΕ', 'ΟΠΟΙΕΣ', 'ΟΠΟΙΕΣ∆ΗΠΟΤΕ', 'ΟΠΟΙΟ', 'ΟΠΟΙΟ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΙ', 'ΟΠΟΙΟΝ', 'ΟΠΟΙΟΝ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΣ', 'ΟΠΟΙΟΣ∆ΗΠΟΤΕ',
                   'ΟΠΟΙΟΥ', 'ΟΠΟΙΟΥ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΥΣ', 'ΟΠΟΙΟΥΣ∆ΗΠΟΤΕ', 'ΟΠΟΙΩΝ', 'ΟΠΟΙΩΝ∆ΗΠΟΤΕ', 'ΟΠΟΤΕ', 'ΟΠΟΤΕ∆ΗΠΟΤΕ', 'ΟΠΟΥ', 'ΟΠΟΥ∆ΗΠΟΤΕ',
                   'ΟΠΩΣ', 'ΟΡΙΣΜΕΝΑ', 'ΟΡΙΣΜΕΝΕΣ', 'ΟΡΙΣΜΕΝΩΝ', 'ΟΡΙΣΜΕΝΩΣ', 'ΟΣΑ', 'ΟΣΑ∆ΗΠΟΤΕ', 'ΟΣΕΣ', 'ΟΣΕΣ∆ΗΠΟΤΕ', 'ΟΣΗ', 'ΟΣΗ∆ΗΠΟΤΕ', 'ΟΣΗΝ',
                   'ΟΣΗΝ∆ΗΠΟΤΕ', 'ΟΣΗΣ', 'ΟΣΗΣ∆ΗΠΟΤΕ', 'ΟΣΟ', 'ΟΣΟ∆ΗΠΟΤΕ', 'ΟΣΟΙ', 'ΟΣΟΙ∆ΗΠΟΤΕ', 'ΟΣΟΝ', 'ΟΣΟΝ∆ΗΠΟΤΕ', 'ΟΣΟΣ', 'ΟΣΟΣ∆ΗΠΟΤΕ', 'ΟΣΟΥ',
                   'ΟΣΟΥ∆ΗΠΟΤΕ', 'ΟΣΟΥΣ', 'ΟΣΟΥΣ∆ΗΠΟΤΕ', 'ΟΣΩΝ', 'ΟΣΩΝ∆ΗΠΟΤΕ', 'ΟΤΑΝ', 'ΟΤΙ', 'ΟΤΙ∆ΗΠΟΤΕ', 'ΟΤΟΥ', 'ΟΥ', 'ΟΥ∆Ε', 'ΟΥΤΕ', 'ΟΧΙ',
                   'ΠΑΛΙ', 'ΠΑΝΤΟΤΕ', 'ΠΑΝΤΟΥ', 'ΠΑΝΤΩΣ', 'ΠΑΡΑ', 'ΠΕΡΑ', 'ΠΕΡΙ', 'ΠΕΡΙΠΟΥ', 'ΠΕΡΙΣΣΟΤΕΡΟ', 'ΠΕΡΣΙ', 'ΠΕΡΥΣΙ', 'ΠΙΑ', 'ΠΙΘΑΝΟΝ',
                   'ΠΙΟ', 'ΠΙΣΩ', 'ΠΛΑΙ', 'ΠΛΕΟΝ', 'ΠΛΗΝ', 'ΠΟΙΑ', 'ΠΟΙΑΝ', 'ΠΟΙΑΣ', 'ΠΟΙΕΣ', 'ΠΟΙΟ', 'ΠΟΙΟΙ', 'ΠΟΙΟΝ', 'ΠΟΙΟΣ', 'ΠΟΙΟΥ', 'ΠΟΙΟΥΣ',
                   'ΠΟΙΩΝ', 'ΠΟΛΥ', 'ΠΟΣΕΣ', 'ΠΟΣΗ', 'ΠΟΣΗΝ', 'ΠΟΣΗΣ', 'ΠΟΣΟΙ', 'ΠΟΣΟΣ', 'ΠΟΣΟΥΣ', 'ΠΟΤΕ', 'ΠΟΥ', 'ΠΟΥΘΕ', 'ΠΟΥΘΕΝΑ', 'ΠΡΕΠΕΙ',
                   'ΠΡΙΝ', 'ΠΡΟ', 'ΠΡΟΚΕΙΜΕΝΟΥ', 'ΠΡΟΚΕΙΤΑΙ', 'ΠΡΟΠΕΡΣΙ', 'ΠΡΟΣ', 'ΠΡΟΤΟΥ', 'ΠΡΟΧΘΕΣ', 'ΠΡΟΧΤΕΣ', 'ΠΡΩΤΥΤΕΡΑ', 'ΠΩΣ', 'ΣΑΝ', 'ΣΑΣ',
                   'ΣΕ', 'ΣΕΙΣ', 'ΣΗΜΕΡΑ', 'ΣΙΓΑ', 'ΣΟΥ', 'ΣΤΑ', 'ΣΤΗ', 'ΣΤΗΝ', 'ΣΤΗΣ', 'ΣΤΙΣ', 'ΣΤΟ', 'ΣΤΟΝ', 'ΣΤΟΥ', 'ΣΤΟΥΣ', 'ΣΤΩΝ', 'ΣΥΓΧΡΟΝΩΣ',
                   'ΣΥΝ', 'ΣΥΝΑΜΑ', 'ΣΥΝΕΠΩΣ', 'ΣΥΝΗΘΩΣ', 'ΣΥΧΝΑ', 'ΣΥΧΝΑΣ', 'ΣΥΧΝΕΣ', 'ΣΥΧΝΗ', 'ΣΥΧΝΗΝ', 'ΣΥΧΝΗΣ', 'ΣΥΧΝΟ', 'ΣΥΧΝΟΙ', 'ΣΥΧΝΟΝ',
                   'ΣΥΧΝΟΣ', 'ΣΥΧΝΟΥ', 'ΣΥΧΝΟΥ', 'ΣΥΧΝΟΥΣ', 'ΣΥΧΝΩΝ', 'ΣΥΧΝΩΣ', 'ΣΧΕ∆ΟΝ', 'ΣΩΣΤΑ', 'ΤΑ', 'ΤΑ∆Ε', 'ΤΑΥΤΑ', 'ΤΑΥΤΕΣ', 'ΤΑΥΤΗ', 'ΤΑΥΤΗΝ',
                   'ΤΑΥΤΗΣ', 'ΤΑΥΤΟ,ΤΑΥΤΟΝ', 'ΤΑΥΤΟΣ', 'ΤΑΥΤΟΥ', 'ΤΑΥΤΩΝ', 'ΤΑΧΑ', 'ΤΑΧΑΤΕ', 'ΤΕΛΙΚΑ', 'ΤΕΛΙΚΩΣ', 'ΤΕΣ', 'ΤΕΤΟΙΑ', 'ΤΕΤΟΙΑΝ',
                   'ΤΕΤΟΙΑΣ', 'ΤΕΤΟΙΕΣ', 'ΤΕΤΟΙΟ', 'ΤΕΤΟΙΟΙ', 'ΤΕΤΟΙΟΝ', 'ΤΕΤΟΙΟΣ', 'ΤΕΤΟΙΟΥ', 'ΤΕΤΟΙΟΥΣ', 'ΤΕΤΟΙΩΝ', 'ΤΗ', 'ΤΗΝ', 'ΤΗΣ', 'ΤΙ',
                   'ΤΙΠΟΤΑ', 'ΤΙΠΟΤΕ', 'ΤΙΣ', 'ΤΟ', 'ΤΟΙ', 'ΤΟΝ', 'ΤΟΣ', 'ΤΟΣΑ', 'ΤΟΣΕΣ', 'ΤΟΣΗ', 'ΤΟΣΗΝ', 'ΤΟΣΗΣ', 'ΤΟΣΟ', 'ΤΟΣΟΙ', 'ΤΟΣΟΝ', 'ΤΟΣΟΣ',
                   'ΤΟΣΟΥ', 'ΤΟΣΟΥΣ', 'ΤΟΣΩΝ', 'ΤΟΤΕ', 'ΤΟΥ', 'ΤΟΥΛΑΧΙΣΤΟ', 'ΤΟΥΛΑΧΙΣΤΟΝ', 'ΤΟΥΣ', 'ΤΟΥΤΑ', 'ΤΟΥΤΕΣ', 'ΤΟΥΤΗ', 'ΤΟΥΤΗΝ', 'ΤΟΥΤΗΣ',
                   'ΤΟΥΤΟ', 'ΤΟΥΤΟΙ', 'ΤΟΥΤΟΙΣ', 'ΤΟΥΤΟΝ', 'ΤΟΥΤΟΣ', 'ΤΟΥΤΟΥ', 'ΤΟΥΤΟΥΣ', 'ΤΟΥΤΩΝ', 'ΤΥΧΟΝ', 'ΤΩΝ', 'ΤΩΡΑ', 'ΥΠ', 'ΥΠΕΡ', 'ΥΠΟ',
                   'ΥΠΟΨΗ', 'ΥΠΟΨΙΝ', 'ΥΣΤΕΡΑ', 'ΦΕΤΟΣ', 'ΧΑΜΗΛΑ', 'ΧΘΕΣ', 'ΧΤΕΣ', 'ΧΩΡΙΣ', 'ΧΩΡΙΣΤΑ', 'ΨΗΛΑ', 'Ω', 'ΩΡΑΙΑ', 'ΩΣ', 'ΩΣΑΝ', 'ΩΣΟΤΟΥ',
                   'ΩΣΠΟΥ', 'ΩΣΤΕ', 'ΩΣΤΟΣΟ', 'ΩΧ']

# Text Preprocessing

def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# Prepare Data

X_train = train_s["text"]
y_train = train_s["label"]
X_val = val_s["text"]
y_val = val_s["label"]
X_test = test_s["text"]
y_test = test_s["label"]

for df in [train_s]:
    df["text"] = df["text"].astype(str).apply(clean_text)


# Compute Sample Weights

from collections import Counter

# Count των labels
label_counts = Counter(y_train)
under_10_classes = [label for label, count in label_counts.items() if count < 10]

# Fit

vectorizer = TfidfVectorizer(
    stop_words=greek_stopwords,
    lowercase=True,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95
)
# Fit & transform τα κείμενα
X_train_vec = vectorizer.fit_transform(X_train)


sampling_strategy = {label: 20 for label in under_10_classes}

# Oversampler
ros = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train_vec, y_train)
print(X_resampled.shape[0])
print(y_resampled.shape[0])
# Μοντέλο
clf = LinearSVC()

# Fit στο resampled data
clf.fit(X_resampled, y_resampled)

# Validation
X_val_vec = vectorizer.transform(X_val)
y_pred_val = clf.predict(X_val_vec)

# Αποτελέσματα
print(classification_report(y_val, y_pred_val, digits=4))


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'εαν', 'εαυτο', 'εαυτον', 'εαυτου', 'εαυτους', 'εαυτων', 'εγκαιρα', 'εγκαιρως', 'εγω', 'ει', 'ειθε', 'ειμαι', 'ειμαστε', 'εινα', 'ειναι', 'εις', 

50945
50945
              precision    recall  f1-score   support

           0     1.0000    0.4545    0.6250        11
           3     0.5000    1.0000    0.6667         1
           4     0.0000    0.0000    0.0000         1
           5     0.0000    0.0000    0.0000         2
           8     0.8182    0.7500    0.7826        12
           9     1.0000    0.7143    0.8333         7
          10     0.0000    0.0000    0.0000         0
          11     0.3333    0.5000    0.4000         2
          12     0.0000    0.0000    0.0000         1
          13     1.0000    0.9000    0.9474        10
          14     1.0000    1.0000    1.0000         1
          15     0.0000    0.0000    0.0000         2
          16     0.6667    1.0000    0.8000         2
          17     1.0000    1.0000    1.0000         9
          19     0.0000    0.0000    0.0000         4
          20     0.5000    0.3333    0.4000         3
          21     0.0000    0.0000    0.0000         1
          23   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

'\nval_pipeline = Pipeline([\n    (\'tfidf\', TfidfVectorizer(\n        stop_words=greek_stopwords,\n        lowercase=True,\n        ngram_range=(1, 2),\n        min_df=2,\n        max_df=0.95\n    )),\n    (\'over\', RandomOverSampler(random_state=42)),\n    (\'clf\', LinearSVC())\n])\n\nval_pipeline.fit(X_train, y_train)\n\n# -----------------------------\n# Evaluate\n# -----------------------------\nprint("\n📊 Validation Results (subject level):\n")\ny_pred_val = val_pipeline.predict(X_val)\nprint(classification_report(y_val, y_pred_val, digits=4))\n\n\n'

TEST subject TFIDF

In [ ]:
# Validation
X_test_vec = vectorizer.transform(X_test)
y_pred_test = clf.predict(X_test_vec)

# Αποτελέσματα
print(classification_report(y_test, y_pred_test, digits=4))

              precision    recall  f1-score   support

           0     0.8000    1.0000    0.8889         8
           4     0.0000    0.0000    0.0000         1
           5     1.0000    0.3333    0.5000         3
           7     0.0000    0.0000    0.0000         0
           8     0.7333    1.0000    0.8462        11
           9     0.6667    0.5000    0.5714         4
          10     0.5000    0.3333    0.4000         3
          11     1.0000    0.6667    0.8000         3
          12     0.0000    0.0000    0.0000         1
          13     1.0000    1.0000    1.0000         4
          14     1.0000    1.0000    1.0000         1
          16     0.0000    0.0000    0.0000         0
          17     0.5000    0.5000    0.5000         2
          19     0.0000    0.0000    0.0000         3
          20     1.0000    0.5000    0.6667         4
          22     0.5000    1.0000    0.6667         1
          23     0.0000    0.0000    0.0000         1
          24     1.0000    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

# BOW (bag of words)

In [ ]:
import pandas as pd
import re
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
greek_stopwords = ['Α∆ΙΑΚΟΠΑ', 'ΑΙ', 'ΑΚΟΜΑ', 'ΑΚΟΜΗ', 'ΑΚΡΙΒΩΣ', 'ΑΛΗΘΕΙΑ', 'ΑΛΗΘΙΝΑ', 'ΑΛΛΑ', 'ΑΛΛΑΧΟΥ', 'ΑΛΛΕΣ', 'ΑΛΛΗ', 'ΑΛΛΗΝ', 'ΑΛΛΗΣ',
                   'ΑΛΛΙΩΣ', 'ΑΛΛΙΩΤΙΚΑ', 'ΑΛΛΟ', 'ΑΛΛΟΙ', 'ΑΛΛΟΙΩΣ', 'ΑΛΛΟΙΩΤΙΚΑ', 'ΑΛΛΟΝ', 'ΑΛΛΟΣ', 'ΑΛΛΟΤΕ', 'ΑΛΛΟΥ', 'ΑΛΛΟΥΣ', 'ΑΛΛΩΝ', 'ΑΜΑ',
                   'ΑΜΕΣΑ', 'ΑΜΕΣΩΣ', 'ΑΝ', 'ΑΝΑ', 'ΑΝΑΜΕΣΑ', 'ΑΝΑΜΕΤΑΞΥ', 'ΑΝΕΥ', 'ΑΝΤΙ', 'ΑΝΤΙΠΕΡΑ', 'ΑΝΤΙΣ', 'ΑΝΩ', 'ΑΝΩΤΕΡΩ', 'ΑΞΑΦΝΑ', 'ΑΠ',
                   'ΑΠΕΝΑΝΤΙ', 'ΑΠΟ', 'ΑΠΟΨΕ', 'ΑΡΑ', 'ΑΡΑΓΕ', 'ΑΡΓΑ', 'ΑΡΓΟΤΕΡΟ', 'ΑΡΙΣΤΕΡΑ', 'ΑΡΚΕΤΑ', 'ΑΡΧΙΚΑ', 'ΑΣ', 'ΑΥΡΙΟ', 'ΑΥΤΑ', 'ΑΥΤΕΣ',
                   'ΑΥΤΗ', 'ΑΥΤΗΝ', 'ΑΥΤΗΣ', 'ΑΥΤΟ', 'ΑΥΤΟΙ', 'ΑΥΤΟΝ', 'ΑΥΤΟΣ', 'ΑΥΤΟΥ', 'ΑΥΤΟΥΣ', 'ΑΥΤΩΝ', 'ΑΦΟΤΟΥ', 'ΑΦΟΥ', 'ΒΕΒΑΙΑ', 'ΒΕΒΑΙΟΤΑΤΑ',
                   'ΓΙ', 'ΓΙΑ', 'ΓΡΗΓΟΡΑ', 'ΓΥΡΩ', '∆Α', '∆Ε', '∆ΕΙΝΑ', '∆ΕΝ', '∆ΕΞΙΑ', '∆ΗΘΕΝ', '∆ΗΛΑ∆Η', '∆Ι', '∆ΙΑ', '∆ΙΑΡΚΩΣ', '∆ΙΚΑ', '∆ΙΚΟ',
                   '∆ΙΚΟΙ', '∆ΙΚΟΣ', '∆ΙΚΟΥ', '∆ΙΚΟΥΣ', '∆ΙΟΛΟΥ', '∆ΙΠΛΑ', '∆ΙΧΩΣ', 'ΕΑΝ', 'ΕΑΥΤΟ', 'ΕΑΥΤΟΝ', 'ΕΑΥΤΟΥ', 'ΕΑΥΤΟΥΣ', 'ΕΑΥΤΩΝ',
                   'ΕΓΚΑΙΡΑ', 'ΕΓΚΑΙΡΩΣ', 'ΕΓΩ', 'Ε∆Ω', 'ΕΙ∆ΕΜΗ', 'ΕΙΘΕ', 'ΕΙΜΑΙ', 'ΕΙΜΑΣΤΕ', 'ΕΙΝΑΙ', 'ΕΙΣ', 'ΕΙΣΑΙ', 'ΕΙΣΑΣΤΕ', 'ΕΙΣΤΕ', 'ΕΙΤΕ',
                   'ΕΙΧΑ', 'ΕΙΧΑΜΕ', 'ΕΙΧΑΝ', 'ΕΙΧΑΤΕ', 'ΕΙΧΕ', 'ΕΙΧΕΣ', 'ΕΚΑΣΤΑ', 'ΕΚΑΣΤΕΣ', 'ΕΚΑΣΤΗ', 'ΕΚΑΣΤΗΝ', 'ΕΚΑΣΤΗΣ', 'ΕΚΑΣΤΟ', 'ΕΚΑΣΤΟΙ',
                   'ΕΚΑΣΤΟΝ', 'ΕΚΑΣΤΟΣ', 'ΕΚΑΣΤΟΥ', 'ΕΚΑΣΤΟΥΣ', 'ΕΚΑΣΤΩΝ', 'ΕΚΕΙ', 'ΕΚΕΙΝΑ', 'ΕΚΕΙΝΕΣ', 'ΕΚΕΙΝΗ', 'ΕΚΕΙΝΗΝ', 'ΕΚΕΙΝΗΣ', 'ΕΚΕΙΝΟ',
                   'ΕΚΕΙΝΟΙ', 'ΕΚΕΙΝΟΝ', 'ΕΚΕΙΝΟΣ', 'ΕΚΕΙΝΟΥ', 'ΕΚΕΙΝΟΥΣ', 'ΕΚΕΙΝΩΝ', 'ΕΚΤΟΣ', 'ΕΜΑΣ', 'ΕΜΕΙΣ', 'ΕΜΕΝΑ', 'ΕΜΠΡΟΣ', 'ΕΝ', 'ΕΝΑ',
                   'ΕΝΑΝ', 'ΕΝΑΣ', 'ΕΝΟΣ', 'ΕΝΤΕΛΩΣ', 'ΕΝΤΟΣ', 'ΕΝΤΩΜΕΤΑΞΥ', 'ΕΝΩ', 'ΕΞ', 'ΕΞΑΦΝΑ', 'ΕΞΗΣ', 'ΕΞΙΣΟΥ', 'ΕΞΩ', 'ΕΠΑΝΩ', 'ΕΠΕΙ∆Η',
                   'ΕΠΕΙΤΑ', 'ΕΠΙ', 'ΕΠΙΣΗΣ', 'ΕΠΟΜΕΝΩΣ', 'ΕΣΑΣ', 'ΕΣΕΙΣ', 'ΕΣΕΝΑ', 'ΕΣΤΩ', 'ΕΣΥ', 'ΕΤΕΡΑ', 'ΕΤΕΡΑΙ', 'ΕΤΕΡΑΣ', 'ΕΤΕΡΕΣ', 'ΕΤΕΡΗ',
                   'ΕΤΕΡΗΣ', 'ΕΤΕΡΟ', 'ΕΤΕΡΟΙ', 'ΕΤΕΡΟΝ', 'ΕΤΕΡΟΣ', 'ΕΤΕΡΟΥ', 'ΕΤΕΡΟΥΣ', 'ΕΤΕΡΩΝ', 'ΕΤΟΥΤΑ', 'ΕΤΟΥΤΕΣ', 'ΕΤΟΥΤΗ', 'ΕΤΟΥΤΗΝ',
                   'ΕΤΟΥΤΗΣ', 'ΕΤΟΥΤΟ', 'ΕΤΟΥΤΟΙ', 'ΕΤΟΥΤΟΝ', 'ΕΤΟΥΤΟΣ', 'ΕΤΟΥΤΟΥ', 'ΕΤΟΥΤΟΥΣ', 'ΕΤΟΥΤΩΝ', 'ΕΤΣΙ', 'ΕΥΓΕ', 'ΕΥΘΥΣ', 'ΕΥΤΥΧΩΣ',
                   'ΕΦΕΞΗΣ', 'ΕΧΕΙ', 'ΕΧΕΙΣ', 'ΕΧΕΤΕ', 'ΕΧΘΕΣ', 'ΕΧΟΜΕ', 'ΕΧΟΥΜΕ', 'ΕΧΟΥΝ', 'ΕΧΤΕΣ', 'ΕΧΩ', 'ΕΩΣ', 'Η', 'Η∆Η', 'ΗΜΑΣΤΑΝ', 'ΗΜΑΣΤΕ',
                   'ΗΜΟΥΝ', 'ΗΣΑΣΤΑΝ', 'ΗΣΑΣΤΕ', 'ΗΣΟΥΝ', 'ΗΤΑΝ', 'ΗΤΑΝΕ', 'ΗΤΟΙ', 'ΗΤΤΟΝ', 'ΘΑ', 'Ι', 'Ι∆ΙΑ', 'Ι∆ΙΑΝ', 'Ι∆ΙΑΣ', 'Ι∆ΙΕΣ', 'Ι∆ΙΟ',
                   'Ι∆ΙΟΙ', 'Ι∆ΙΟΝ', 'Ι∆ΙΟΣ', 'Ι∆ΙΟΥ', 'Ι∆ΙΟΥΣ', 'Ι∆ΙΩΝ', 'Ι∆ΙΩΣ', 'ΙΙ', 'ΙΙΙ', 'ΙΣΑΜΕ', 'ΙΣΙΑ', 'ΙΣΩΣ', 'ΚΑΘΕ', 'ΚΑΘΕΜΙΑ',
                   'ΚΑΘΕΜΙΑΣ', 'ΚΑΘΕΝΑ', 'ΚΑΘΕΝΑΣ', 'ΚΑΘΕΝΟΣ', 'ΚΑΘΕΤΙ', 'ΚΑΘΟΛΟΥ', 'ΚΑΘΩΣ', 'ΚΑΙ', 'ΚΑΚΑ', 'ΚΑΚΩΣ', 'ΚΑΛΑ', 'ΚΑΛΩΣ', 'ΚΑΜΙΑ',
                   'ΚΑΜΙΑΝ', 'ΚΑΜΙΑΣ', 'ΚΑΜΠΟΣΑ', 'ΚΑΜΠΟΣΕΣ', 'ΚΑΜΠΟΣΗ', 'ΚΑΜΠΟΣΗΝ', 'ΚΑΜΠΟΣΗΣ', 'ΚΑΜΠΟΣΟ', 'ΚΑΜΠΟΣΟΙ', 'ΚΑΜΠΟΣΟΝ', 'ΚΑΜΠΟΣΟΣ',
                   'ΚΑΜΠΟΣΟΥ', 'ΚΑΜΠΟΣΟΥΣ', 'ΚΑΜΠΟΣΩΝ', 'ΚΑΝΕΙΣ', 'ΚΑΝΕΝ', 'ΚΑΝΕΝΑ', 'ΚΑΝΕΝΑΝ', 'ΚΑΝΕΝΑΣ', 'ΚΑΝΕΝΟΣ', 'ΚΑΠΟΙΑ', 'ΚΑΠΟΙΑΝ', 'ΚΑΠΟΙΑΣ',
                   'ΚΑΠΟΙΕΣ', 'ΚΑΠΟΙΟ', 'ΚΑΠΟΙΟΙ', 'ΚΑΠΟΙΟΝ', 'ΚΑΠΟΙΟΣ', 'ΚΑΠΟΙΟΥ', 'ΚΑΠΟΙΟΥΣ', 'ΚΑΠΟΙΩΝ', 'ΚΑΠΟΤΕ', 'ΚΑΠΟΥ', 'ΚΑΠΩΣ', 'ΚΑΤ', 'ΚΑΤΑ',
                   'ΚΑΤΙ', 'ΚΑΤΙΤΙ', 'ΚΑΤΟΠΙΝ', 'ΚΑΤΩ', 'ΚΙΟΛΑΣ', 'ΚΛΠ', 'ΚΟΝΤΑ', 'ΚΤΛ', 'ΚΥΡΙΩΣ', 'ΛΙΓΑΚΙ', 'ΛΙΓΟ', 'ΛΙΓΩΤΕΡΟ', 'ΛΟΓΩ', 'ΛΟΙΠΑ',
                   'ΛΟΙΠΟΝ', 'ΜΑ', 'ΜΑΖΙ', 'ΜΑΚΑΡΙ', 'ΜΑΚΡΥΑ', 'ΜΑΛΙΣΤΑ', 'ΜΑΛΛΟΝ', 'ΜΑΣ', 'ΜΕ', 'ΜΕΘΑΥΡΙΟ', 'ΜΕΙΟΝ', 'ΜΕΛΕΙ', 'ΜΕΛΛΕΤΑΙ', 'ΜΕΜΙΑΣ',
                   'ΜΕΝ', 'ΜΕΡΙΚΑ', 'ΜΕΡΙΚΕΣ', 'ΜΕΡΙΚΟΙ', 'ΜΕΡΙΚΟΥΣ', 'ΜΕΡΙΚΩΝ', 'ΜΕΣΑ', 'ΜΕΤ', 'ΜΕΤΑ', 'ΜΕΤΑΞΥ', 'ΜΕΧΡΙ', 'ΜΗ', 'ΜΗ∆Ε', 'ΜΗΝ',
                   'ΜΗΠΩΣ', 'ΜΗΤΕ', 'ΜΙΑ', 'ΜΙΑΝ', 'ΜΙΑΣ', 'ΜΟΛΙΣ', 'ΜΟΛΟΝΟΤΙ', 'ΜΟΝΑΧΑ', 'ΜΟΝΕΣ', 'ΜΟΝΗ', 'ΜΟΝΗΝ', 'ΜΟΝΗΣ', 'ΜΟΝΟ', 'ΜΟΝΟΙ',
                   'ΜΟΝΟΜΙΑΣ', 'ΜΟΝΟΣ', 'ΜΟΝΟΥ', 'ΜΟΝΟΥΣ', 'ΜΟΝΩΝ', 'ΜΟΥ', 'ΜΠΟΡΕΙ', 'ΜΠΟΡΟΥΝ', 'ΜΠΡΑΒΟ', 'ΜΠΡΟΣ', 'ΝΑ', 'ΝΑΙ', 'ΝΩΡΙΣ', 'ΞΑΝΑ',
                   'ΞΑΦΝΙΚΑ', 'Ο', 'ΟΙ', 'ΟΛΑ', 'ΟΛΕΣ', 'ΟΛΗ', 'ΟΛΗΝ', 'ΟΛΗΣ', 'ΟΛΟ', 'ΟΛΟΓΥΡΑ', 'ΟΛΟΙ', 'ΟΛΟΝ', 'ΟΛΟΝΕΝ', 'ΟΛΟΣ', 'ΟΛΟΤΕΛΑ', 'ΟΛΟΥ',
                   'ΟΛΟΥΣ', 'ΟΛΩΝ', 'ΟΛΩΣ', 'ΟΛΩΣ∆ΙΟΛΟΥ', 'ΟΜΩΣ', 'ΟΠΟΙΑ', 'ΟΠΟΙΑ∆ΗΠΟΤΕ', 'ΟΠΟΙΑΝ', 'ΟΠΟΙΑΝ∆ΗΠΟΤΕ', 'ΟΠΟΙΑΣ', 'ΟΠΟΙΑΣ∆ΗΠΟΤΕ',
                   'ΟΠΟΙ∆ΗΠΟΤΕ', 'ΟΠΟΙΕΣ', 'ΟΠΟΙΕΣ∆ΗΠΟΤΕ', 'ΟΠΟΙΟ', 'ΟΠΟΙΟ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΙ', 'ΟΠΟΙΟΝ', 'ΟΠΟΙΟΝ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΣ', 'ΟΠΟΙΟΣ∆ΗΠΟΤΕ',
                   'ΟΠΟΙΟΥ', 'ΟΠΟΙΟΥ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΥΣ', 'ΟΠΟΙΟΥΣ∆ΗΠΟΤΕ', 'ΟΠΟΙΩΝ', 'ΟΠΟΙΩΝ∆ΗΠΟΤΕ', 'ΟΠΟΤΕ', 'ΟΠΟΤΕ∆ΗΠΟΤΕ', 'ΟΠΟΥ', 'ΟΠΟΥ∆ΗΠΟΤΕ',
                   'ΟΠΩΣ', 'ΟΡΙΣΜΕΝΑ', 'ΟΡΙΣΜΕΝΕΣ', 'ΟΡΙΣΜΕΝΩΝ', 'ΟΡΙΣΜΕΝΩΣ', 'ΟΣΑ', 'ΟΣΑ∆ΗΠΟΤΕ', 'ΟΣΕΣ', 'ΟΣΕΣ∆ΗΠΟΤΕ', 'ΟΣΗ', 'ΟΣΗ∆ΗΠΟΤΕ', 'ΟΣΗΝ',
                   'ΟΣΗΝ∆ΗΠΟΤΕ', 'ΟΣΗΣ', 'ΟΣΗΣ∆ΗΠΟΤΕ', 'ΟΣΟ', 'ΟΣΟ∆ΗΠΟΤΕ', 'ΟΣΟΙ', 'ΟΣΟΙ∆ΗΠΟΤΕ', 'ΟΣΟΝ', 'ΟΣΟΝ∆ΗΠΟΤΕ', 'ΟΣΟΣ', 'ΟΣΟΣ∆ΗΠΟΤΕ', 'ΟΣΟΥ',
                   'ΟΣΟΥ∆ΗΠΟΤΕ', 'ΟΣΟΥΣ', 'ΟΣΟΥΣ∆ΗΠΟΤΕ', 'ΟΣΩΝ', 'ΟΣΩΝ∆ΗΠΟΤΕ', 'ΟΤΑΝ', 'ΟΤΙ', 'ΟΤΙ∆ΗΠΟΤΕ', 'ΟΤΟΥ', 'ΟΥ', 'ΟΥ∆Ε', 'ΟΥΤΕ', 'ΟΧΙ',
                   'ΠΑΛΙ', 'ΠΑΝΤΟΤΕ', 'ΠΑΝΤΟΥ', 'ΠΑΝΤΩΣ', 'ΠΑΡΑ', 'ΠΕΡΑ', 'ΠΕΡΙ', 'ΠΕΡΙΠΟΥ', 'ΠΕΡΙΣΣΟΤΕΡΟ', 'ΠΕΡΣΙ', 'ΠΕΡΥΣΙ', 'ΠΙΑ', 'ΠΙΘΑΝΟΝ',
                   'ΠΙΟ', 'ΠΙΣΩ', 'ΠΛΑΙ', 'ΠΛΕΟΝ', 'ΠΛΗΝ', 'ΠΟΙΑ', 'ΠΟΙΑΝ', 'ΠΟΙΑΣ', 'ΠΟΙΕΣ', 'ΠΟΙΟ', 'ΠΟΙΟΙ', 'ΠΟΙΟΝ', 'ΠΟΙΟΣ', 'ΠΟΙΟΥ', 'ΠΟΙΟΥΣ',
                   'ΠΟΙΩΝ', 'ΠΟΛΥ', 'ΠΟΣΕΣ', 'ΠΟΣΗ', 'ΠΟΣΗΝ', 'ΠΟΣΗΣ', 'ΠΟΣΟΙ', 'ΠΟΣΟΣ', 'ΠΟΣΟΥΣ', 'ΠΟΤΕ', 'ΠΟΥ', 'ΠΟΥΘΕ', 'ΠΟΥΘΕΝΑ', 'ΠΡΕΠΕΙ',
                   'ΠΡΙΝ', 'ΠΡΟ', 'ΠΡΟΚΕΙΜΕΝΟΥ', 'ΠΡΟΚΕΙΤΑΙ', 'ΠΡΟΠΕΡΣΙ', 'ΠΡΟΣ', 'ΠΡΟΤΟΥ', 'ΠΡΟΧΘΕΣ', 'ΠΡΟΧΤΕΣ', 'ΠΡΩΤΥΤΕΡΑ', 'ΠΩΣ', 'ΣΑΝ', 'ΣΑΣ',
                   'ΣΕ', 'ΣΕΙΣ', 'ΣΗΜΕΡΑ', 'ΣΙΓΑ', 'ΣΟΥ', 'ΣΤΑ', 'ΣΤΗ', 'ΣΤΗΝ', 'ΣΤΗΣ', 'ΣΤΙΣ', 'ΣΤΟ', 'ΣΤΟΝ', 'ΣΤΟΥ', 'ΣΤΟΥΣ', 'ΣΤΩΝ', 'ΣΥΓΧΡΟΝΩΣ',
                   'ΣΥΝ', 'ΣΥΝΑΜΑ', 'ΣΥΝΕΠΩΣ', 'ΣΥΝΗΘΩΣ', 'ΣΥΧΝΑ', 'ΣΥΧΝΑΣ', 'ΣΥΧΝΕΣ', 'ΣΥΧΝΗ', 'ΣΥΧΝΗΝ', 'ΣΥΧΝΗΣ', 'ΣΥΧΝΟ', 'ΣΥΧΝΟΙ', 'ΣΥΧΝΟΝ',
                   'ΣΥΧΝΟΣ', 'ΣΥΧΝΟΥ', 'ΣΥΧΝΟΥ', 'ΣΥΧΝΟΥΣ', 'ΣΥΧΝΩΝ', 'ΣΥΧΝΩΣ', 'ΣΧΕ∆ΟΝ', 'ΣΩΣΤΑ', 'ΤΑ', 'ΤΑ∆Ε', 'ΤΑΥΤΑ', 'ΤΑΥΤΕΣ', 'ΤΑΥΤΗ', 'ΤΑΥΤΗΝ',
                   'ΤΑΥΤΗΣ', 'ΤΑΥΤΟ,ΤΑΥΤΟΝ', 'ΤΑΥΤΟΣ', 'ΤΑΥΤΟΥ', 'ΤΑΥΤΩΝ', 'ΤΑΧΑ', 'ΤΑΧΑΤΕ', 'ΤΕΛΙΚΑ', 'ΤΕΛΙΚΩΣ', 'ΤΕΣ', 'ΤΕΤΟΙΑ', 'ΤΕΤΟΙΑΝ',
                   'ΤΕΤΟΙΑΣ', 'ΤΕΤΟΙΕΣ', 'ΤΕΤΟΙΟ', 'ΤΕΤΟΙΟΙ', 'ΤΕΤΟΙΟΝ', 'ΤΕΤΟΙΟΣ', 'ΤΕΤΟΙΟΥ', 'ΤΕΤΟΙΟΥΣ', 'ΤΕΤΟΙΩΝ', 'ΤΗ', 'ΤΗΝ', 'ΤΗΣ', 'ΤΙ',
                   'ΤΙΠΟΤΑ', 'ΤΙΠΟΤΕ', 'ΤΙΣ', 'ΤΟ', 'ΤΟΙ', 'ΤΟΝ', 'ΤΟΣ', 'ΤΟΣΑ', 'ΤΟΣΕΣ', 'ΤΟΣΗ', 'ΤΟΣΗΝ', 'ΤΟΣΗΣ', 'ΤΟΣΟ', 'ΤΟΣΟΙ', 'ΤΟΣΟΝ', 'ΤΟΣΟΣ',
                   'ΤΟΣΟΥ', 'ΤΟΣΟΥΣ', 'ΤΟΣΩΝ', 'ΤΟΤΕ', 'ΤΟΥ', 'ΤΟΥΛΑΧΙΣΤΟ', 'ΤΟΥΛΑΧΙΣΤΟΝ', 'ΤΟΥΣ', 'ΤΟΥΤΑ', 'ΤΟΥΤΕΣ', 'ΤΟΥΤΗ', 'ΤΟΥΤΗΝ', 'ΤΟΥΤΗΣ',
                   'ΤΟΥΤΟ', 'ΤΟΥΤΟΙ', 'ΤΟΥΤΟΙΣ', 'ΤΟΥΤΟΝ', 'ΤΟΥΤΟΣ', 'ΤΟΥΤΟΥ', 'ΤΟΥΤΟΥΣ', 'ΤΟΥΤΩΝ', 'ΤΥΧΟΝ', 'ΤΩΝ', 'ΤΩΡΑ', 'ΥΠ', 'ΥΠΕΡ', 'ΥΠΟ',
                   'ΥΠΟΨΗ', 'ΥΠΟΨΙΝ', 'ΥΣΤΕΡΑ', 'ΦΕΤΟΣ', 'ΧΑΜΗΛΑ', 'ΧΘΕΣ', 'ΧΤΕΣ', 'ΧΩΡΙΣ', 'ΧΩΡΙΣΤΑ', 'ΨΗΛΑ', 'Ω', 'ΩΡΑΙΑ', 'ΩΣ', 'ΩΣΑΝ', 'ΩΣΟΤΟΥ',
                   'ΩΣΠΟΥ', 'ΩΣΤΕ', 'ΩΣΤΟΣΟ', 'ΩΧ']

# Text Preprocessing

def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# Prepare Data

X_train = train_s["text"]
y_train = train_s["label"]
X_val = val_s["text"]
y_val = val_s["label"]
X_test = test_s["text"]
y_test = test_s["label"]

for df in [train_s]:
    df["text"] = df["text"].astype(str).apply(clean_text)


from collections import Counter

# Count των labels
label_counts = Counter(y_train)
under_5_classes = [label for label, count in label_counts.items() if count < 5]

# Fit Pipeline

vectorizer = CountVectorizer(
    stop_words=greek_stopwords,
    lowercase=True,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95
)
# Fit & transform τα κείμενα
X_train_vec = vectorizer.fit_transform(X_train)


sampling_strategy = {label: 10 for label in under_5_classes}

# Oversampler
ros = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train_vec, y_train)
print(X_resampled.shape[0])
print(y_resampled.shape[0])
# Μοντέλο
clf = LinearSVC()

# Fit στο resampled data
clf.fit(X_resampled, y_resampled)

# Validation
X_val_vec = vectorizer.transform(X_val)
y_pred_val = clf.predict(X_val_vec)

# Αποτελέσματα
print(classification_report(y_val, y_pred_val, digits=4))

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'εαν', 'εαυτο', 'εαυτον', 'εαυτου', 'εαυτους', 'εαυτων', 'εγκαιρα', 'εγκαιρως', 'εγω', 'ει', 'ειθε', 'ειμαι', 'ειμαστε', 'εινα', 'ειναι', 'εις', 

35697
35697


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


              precision    recall  f1-score   support

           0     0.1667    0.2727    0.2069        11
           3     0.0000    0.0000    0.0000         1
           4     0.0000    0.0000    0.0000         1
           5     0.0000    0.0000    0.0000         2
           8     0.8182    0.7500    0.7826        12
           9     0.4545    0.7143    0.5556         7
          11     0.5000    0.5000    0.5000         2
          12     0.0000    0.0000    0.0000         1
          13     0.8750    0.7000    0.7778        10
          14     1.0000    1.0000    1.0000         1
          15     0.0000    0.0000    0.0000         2
          16     1.0000    0.5000    0.6667         2
          17     1.0000    0.8889    0.9412         9
          19     0.2500    0.2500    0.2500         4
          20     1.0000    0.6667    0.8000         3
          21     0.0000    0.0000    0.0000         1
          23     0.0000    0.0000    0.0000         3
          24     1.0000    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [ ]:
import pandas as pd
import re
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
greek_stopwords = ['Α∆ΙΑΚΟΠΑ', 'ΑΙ', 'ΑΚΟΜΑ', 'ΑΚΟΜΗ', 'ΑΚΡΙΒΩΣ', 'ΑΛΗΘΕΙΑ', 'ΑΛΗΘΙΝΑ', 'ΑΛΛΑ', 'ΑΛΛΑΧΟΥ', 'ΑΛΛΕΣ', 'ΑΛΛΗ', 'ΑΛΛΗΝ', 'ΑΛΛΗΣ',
                   'ΑΛΛΙΩΣ', 'ΑΛΛΙΩΤΙΚΑ', 'ΑΛΛΟ', 'ΑΛΛΟΙ', 'ΑΛΛΟΙΩΣ', 'ΑΛΛΟΙΩΤΙΚΑ', 'ΑΛΛΟΝ', 'ΑΛΛΟΣ', 'ΑΛΛΟΤΕ', 'ΑΛΛΟΥ', 'ΑΛΛΟΥΣ', 'ΑΛΛΩΝ', 'ΑΜΑ',
                   'ΑΜΕΣΑ', 'ΑΜΕΣΩΣ', 'ΑΝ', 'ΑΝΑ', 'ΑΝΑΜΕΣΑ', 'ΑΝΑΜΕΤΑΞΥ', 'ΑΝΕΥ', 'ΑΝΤΙ', 'ΑΝΤΙΠΕΡΑ', 'ΑΝΤΙΣ', 'ΑΝΩ', 'ΑΝΩΤΕΡΩ', 'ΑΞΑΦΝΑ', 'ΑΠ',
                   'ΑΠΕΝΑΝΤΙ', 'ΑΠΟ', 'ΑΠΟΨΕ', 'ΑΡΑ', 'ΑΡΑΓΕ', 'ΑΡΓΑ', 'ΑΡΓΟΤΕΡΟ', 'ΑΡΙΣΤΕΡΑ', 'ΑΡΚΕΤΑ', 'ΑΡΧΙΚΑ', 'ΑΣ', 'ΑΥΡΙΟ', 'ΑΥΤΑ', 'ΑΥΤΕΣ',
                   'ΑΥΤΗ', 'ΑΥΤΗΝ', 'ΑΥΤΗΣ', 'ΑΥΤΟ', 'ΑΥΤΟΙ', 'ΑΥΤΟΝ', 'ΑΥΤΟΣ', 'ΑΥΤΟΥ', 'ΑΥΤΟΥΣ', 'ΑΥΤΩΝ', 'ΑΦΟΤΟΥ', 'ΑΦΟΥ', 'ΒΕΒΑΙΑ', 'ΒΕΒΑΙΟΤΑΤΑ',
                   'ΓΙ', 'ΓΙΑ', 'ΓΡΗΓΟΡΑ', 'ΓΥΡΩ', '∆Α', '∆Ε', '∆ΕΙΝΑ', '∆ΕΝ', '∆ΕΞΙΑ', '∆ΗΘΕΝ', '∆ΗΛΑ∆Η', '∆Ι', '∆ΙΑ', '∆ΙΑΡΚΩΣ', '∆ΙΚΑ', '∆ΙΚΟ',
                   '∆ΙΚΟΙ', '∆ΙΚΟΣ', '∆ΙΚΟΥ', '∆ΙΚΟΥΣ', '∆ΙΟΛΟΥ', '∆ΙΠΛΑ', '∆ΙΧΩΣ', 'ΕΑΝ', 'ΕΑΥΤΟ', 'ΕΑΥΤΟΝ', 'ΕΑΥΤΟΥ', 'ΕΑΥΤΟΥΣ', 'ΕΑΥΤΩΝ',
                   'ΕΓΚΑΙΡΑ', 'ΕΓΚΑΙΡΩΣ', 'ΕΓΩ', 'Ε∆Ω', 'ΕΙ∆ΕΜΗ', 'ΕΙΘΕ', 'ΕΙΜΑΙ', 'ΕΙΜΑΣΤΕ', 'ΕΙΝΑΙ', 'ΕΙΣ', 'ΕΙΣΑΙ', 'ΕΙΣΑΣΤΕ', 'ΕΙΣΤΕ', 'ΕΙΤΕ',
                   'ΕΙΧΑ', 'ΕΙΧΑΜΕ', 'ΕΙΧΑΝ', 'ΕΙΧΑΤΕ', 'ΕΙΧΕ', 'ΕΙΧΕΣ', 'ΕΚΑΣΤΑ', 'ΕΚΑΣΤΕΣ', 'ΕΚΑΣΤΗ', 'ΕΚΑΣΤΗΝ', 'ΕΚΑΣΤΗΣ', 'ΕΚΑΣΤΟ', 'ΕΚΑΣΤΟΙ',
                   'ΕΚΑΣΤΟΝ', 'ΕΚΑΣΤΟΣ', 'ΕΚΑΣΤΟΥ', 'ΕΚΑΣΤΟΥΣ', 'ΕΚΑΣΤΩΝ', 'ΕΚΕΙ', 'ΕΚΕΙΝΑ', 'ΕΚΕΙΝΕΣ', 'ΕΚΕΙΝΗ', 'ΕΚΕΙΝΗΝ', 'ΕΚΕΙΝΗΣ', 'ΕΚΕΙΝΟ',
                   'ΕΚΕΙΝΟΙ', 'ΕΚΕΙΝΟΝ', 'ΕΚΕΙΝΟΣ', 'ΕΚΕΙΝΟΥ', 'ΕΚΕΙΝΟΥΣ', 'ΕΚΕΙΝΩΝ', 'ΕΚΤΟΣ', 'ΕΜΑΣ', 'ΕΜΕΙΣ', 'ΕΜΕΝΑ', 'ΕΜΠΡΟΣ', 'ΕΝ', 'ΕΝΑ',
                   'ΕΝΑΝ', 'ΕΝΑΣ', 'ΕΝΟΣ', 'ΕΝΤΕΛΩΣ', 'ΕΝΤΟΣ', 'ΕΝΤΩΜΕΤΑΞΥ', 'ΕΝΩ', 'ΕΞ', 'ΕΞΑΦΝΑ', 'ΕΞΗΣ', 'ΕΞΙΣΟΥ', 'ΕΞΩ', 'ΕΠΑΝΩ', 'ΕΠΕΙ∆Η',
                   'ΕΠΕΙΤΑ', 'ΕΠΙ', 'ΕΠΙΣΗΣ', 'ΕΠΟΜΕΝΩΣ', 'ΕΣΑΣ', 'ΕΣΕΙΣ', 'ΕΣΕΝΑ', 'ΕΣΤΩ', 'ΕΣΥ', 'ΕΤΕΡΑ', 'ΕΤΕΡΑΙ', 'ΕΤΕΡΑΣ', 'ΕΤΕΡΕΣ', 'ΕΤΕΡΗ',
                   'ΕΤΕΡΗΣ', 'ΕΤΕΡΟ', 'ΕΤΕΡΟΙ', 'ΕΤΕΡΟΝ', 'ΕΤΕΡΟΣ', 'ΕΤΕΡΟΥ', 'ΕΤΕΡΟΥΣ', 'ΕΤΕΡΩΝ', 'ΕΤΟΥΤΑ', 'ΕΤΟΥΤΕΣ', 'ΕΤΟΥΤΗ', 'ΕΤΟΥΤΗΝ',
                   'ΕΤΟΥΤΗΣ', 'ΕΤΟΥΤΟ', 'ΕΤΟΥΤΟΙ', 'ΕΤΟΥΤΟΝ', 'ΕΤΟΥΤΟΣ', 'ΕΤΟΥΤΟΥ', 'ΕΤΟΥΤΟΥΣ', 'ΕΤΟΥΤΩΝ', 'ΕΤΣΙ', 'ΕΥΓΕ', 'ΕΥΘΥΣ', 'ΕΥΤΥΧΩΣ',
                   'ΕΦΕΞΗΣ', 'ΕΧΕΙ', 'ΕΧΕΙΣ', 'ΕΧΕΤΕ', 'ΕΧΘΕΣ', 'ΕΧΟΜΕ', 'ΕΧΟΥΜΕ', 'ΕΧΟΥΝ', 'ΕΧΤΕΣ', 'ΕΧΩ', 'ΕΩΣ', 'Η', 'Η∆Η', 'ΗΜΑΣΤΑΝ', 'ΗΜΑΣΤΕ',
                   'ΗΜΟΥΝ', 'ΗΣΑΣΤΑΝ', 'ΗΣΑΣΤΕ', 'ΗΣΟΥΝ', 'ΗΤΑΝ', 'ΗΤΑΝΕ', 'ΗΤΟΙ', 'ΗΤΤΟΝ', 'ΘΑ', 'Ι', 'Ι∆ΙΑ', 'Ι∆ΙΑΝ', 'Ι∆ΙΑΣ', 'Ι∆ΙΕΣ', 'Ι∆ΙΟ',
                   'Ι∆ΙΟΙ', 'Ι∆ΙΟΝ', 'Ι∆ΙΟΣ', 'Ι∆ΙΟΥ', 'Ι∆ΙΟΥΣ', 'Ι∆ΙΩΝ', 'Ι∆ΙΩΣ', 'ΙΙ', 'ΙΙΙ', 'ΙΣΑΜΕ', 'ΙΣΙΑ', 'ΙΣΩΣ', 'ΚΑΘΕ', 'ΚΑΘΕΜΙΑ',
                   'ΚΑΘΕΜΙΑΣ', 'ΚΑΘΕΝΑ', 'ΚΑΘΕΝΑΣ', 'ΚΑΘΕΝΟΣ', 'ΚΑΘΕΤΙ', 'ΚΑΘΟΛΟΥ', 'ΚΑΘΩΣ', 'ΚΑΙ', 'ΚΑΚΑ', 'ΚΑΚΩΣ', 'ΚΑΛΑ', 'ΚΑΛΩΣ', 'ΚΑΜΙΑ',
                   'ΚΑΜΙΑΝ', 'ΚΑΜΙΑΣ', 'ΚΑΜΠΟΣΑ', 'ΚΑΜΠΟΣΕΣ', 'ΚΑΜΠΟΣΗ', 'ΚΑΜΠΟΣΗΝ', 'ΚΑΜΠΟΣΗΣ', 'ΚΑΜΠΟΣΟ', 'ΚΑΜΠΟΣΟΙ', 'ΚΑΜΠΟΣΟΝ', 'ΚΑΜΠΟΣΟΣ',
                   'ΚΑΜΠΟΣΟΥ', 'ΚΑΜΠΟΣΟΥΣ', 'ΚΑΜΠΟΣΩΝ', 'ΚΑΝΕΙΣ', 'ΚΑΝΕΝ', 'ΚΑΝΕΝΑ', 'ΚΑΝΕΝΑΝ', 'ΚΑΝΕΝΑΣ', 'ΚΑΝΕΝΟΣ', 'ΚΑΠΟΙΑ', 'ΚΑΠΟΙΑΝ', 'ΚΑΠΟΙΑΣ',
                   'ΚΑΠΟΙΕΣ', 'ΚΑΠΟΙΟ', 'ΚΑΠΟΙΟΙ', 'ΚΑΠΟΙΟΝ', 'ΚΑΠΟΙΟΣ', 'ΚΑΠΟΙΟΥ', 'ΚΑΠΟΙΟΥΣ', 'ΚΑΠΟΙΩΝ', 'ΚΑΠΟΤΕ', 'ΚΑΠΟΥ', 'ΚΑΠΩΣ', 'ΚΑΤ', 'ΚΑΤΑ',
                   'ΚΑΤΙ', 'ΚΑΤΙΤΙ', 'ΚΑΤΟΠΙΝ', 'ΚΑΤΩ', 'ΚΙΟΛΑΣ', 'ΚΛΠ', 'ΚΟΝΤΑ', 'ΚΤΛ', 'ΚΥΡΙΩΣ', 'ΛΙΓΑΚΙ', 'ΛΙΓΟ', 'ΛΙΓΩΤΕΡΟ', 'ΛΟΓΩ', 'ΛΟΙΠΑ',
                   'ΛΟΙΠΟΝ', 'ΜΑ', 'ΜΑΖΙ', 'ΜΑΚΑΡΙ', 'ΜΑΚΡΥΑ', 'ΜΑΛΙΣΤΑ', 'ΜΑΛΛΟΝ', 'ΜΑΣ', 'ΜΕ', 'ΜΕΘΑΥΡΙΟ', 'ΜΕΙΟΝ', 'ΜΕΛΕΙ', 'ΜΕΛΛΕΤΑΙ', 'ΜΕΜΙΑΣ',
                   'ΜΕΝ', 'ΜΕΡΙΚΑ', 'ΜΕΡΙΚΕΣ', 'ΜΕΡΙΚΟΙ', 'ΜΕΡΙΚΟΥΣ', 'ΜΕΡΙΚΩΝ', 'ΜΕΣΑ', 'ΜΕΤ', 'ΜΕΤΑ', 'ΜΕΤΑΞΥ', 'ΜΕΧΡΙ', 'ΜΗ', 'ΜΗ∆Ε', 'ΜΗΝ',
                   'ΜΗΠΩΣ', 'ΜΗΤΕ', 'ΜΙΑ', 'ΜΙΑΝ', 'ΜΙΑΣ', 'ΜΟΛΙΣ', 'ΜΟΛΟΝΟΤΙ', 'ΜΟΝΑΧΑ', 'ΜΟΝΕΣ', 'ΜΟΝΗ', 'ΜΟΝΗΝ', 'ΜΟΝΗΣ', 'ΜΟΝΟ', 'ΜΟΝΟΙ',
                   'ΜΟΝΟΜΙΑΣ', 'ΜΟΝΟΣ', 'ΜΟΝΟΥ', 'ΜΟΝΟΥΣ', 'ΜΟΝΩΝ', 'ΜΟΥ', 'ΜΠΟΡΕΙ', 'ΜΠΟΡΟΥΝ', 'ΜΠΡΑΒΟ', 'ΜΠΡΟΣ', 'ΝΑ', 'ΝΑΙ', 'ΝΩΡΙΣ', 'ΞΑΝΑ',
                   'ΞΑΦΝΙΚΑ', 'Ο', 'ΟΙ', 'ΟΛΑ', 'ΟΛΕΣ', 'ΟΛΗ', 'ΟΛΗΝ', 'ΟΛΗΣ', 'ΟΛΟ', 'ΟΛΟΓΥΡΑ', 'ΟΛΟΙ', 'ΟΛΟΝ', 'ΟΛΟΝΕΝ', 'ΟΛΟΣ', 'ΟΛΟΤΕΛΑ', 'ΟΛΟΥ',
                   'ΟΛΟΥΣ', 'ΟΛΩΝ', 'ΟΛΩΣ', 'ΟΛΩΣ∆ΙΟΛΟΥ', 'ΟΜΩΣ', 'ΟΠΟΙΑ', 'ΟΠΟΙΑ∆ΗΠΟΤΕ', 'ΟΠΟΙΑΝ', 'ΟΠΟΙΑΝ∆ΗΠΟΤΕ', 'ΟΠΟΙΑΣ', 'ΟΠΟΙΑΣ∆ΗΠΟΤΕ',
                   'ΟΠΟΙ∆ΗΠΟΤΕ', 'ΟΠΟΙΕΣ', 'ΟΠΟΙΕΣ∆ΗΠΟΤΕ', 'ΟΠΟΙΟ', 'ΟΠΟΙΟ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΙ', 'ΟΠΟΙΟΝ', 'ΟΠΟΙΟΝ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΣ', 'ΟΠΟΙΟΣ∆ΗΠΟΤΕ',
                   'ΟΠΟΙΟΥ', 'ΟΠΟΙΟΥ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΥΣ', 'ΟΠΟΙΟΥΣ∆ΗΠΟΤΕ', 'ΟΠΟΙΩΝ', 'ΟΠΟΙΩΝ∆ΗΠΟΤΕ', 'ΟΠΟΤΕ', 'ΟΠΟΤΕ∆ΗΠΟΤΕ', 'ΟΠΟΥ', 'ΟΠΟΥ∆ΗΠΟΤΕ',
                   'ΟΠΩΣ', 'ΟΡΙΣΜΕΝΑ', 'ΟΡΙΣΜΕΝΕΣ', 'ΟΡΙΣΜΕΝΩΝ', 'ΟΡΙΣΜΕΝΩΣ', 'ΟΣΑ', 'ΟΣΑ∆ΗΠΟΤΕ', 'ΟΣΕΣ', 'ΟΣΕΣ∆ΗΠΟΤΕ', 'ΟΣΗ', 'ΟΣΗ∆ΗΠΟΤΕ', 'ΟΣΗΝ',
                   'ΟΣΗΝ∆ΗΠΟΤΕ', 'ΟΣΗΣ', 'ΟΣΗΣ∆ΗΠΟΤΕ', 'ΟΣΟ', 'ΟΣΟ∆ΗΠΟΤΕ', 'ΟΣΟΙ', 'ΟΣΟΙ∆ΗΠΟΤΕ', 'ΟΣΟΝ', 'ΟΣΟΝ∆ΗΠΟΤΕ', 'ΟΣΟΣ', 'ΟΣΟΣ∆ΗΠΟΤΕ', 'ΟΣΟΥ',
                   'ΟΣΟΥ∆ΗΠΟΤΕ', 'ΟΣΟΥΣ', 'ΟΣΟΥΣ∆ΗΠΟΤΕ', 'ΟΣΩΝ', 'ΟΣΩΝ∆ΗΠΟΤΕ', 'ΟΤΑΝ', 'ΟΤΙ', 'ΟΤΙ∆ΗΠΟΤΕ', 'ΟΤΟΥ', 'ΟΥ', 'ΟΥ∆Ε', 'ΟΥΤΕ', 'ΟΧΙ',
                   'ΠΑΛΙ', 'ΠΑΝΤΟΤΕ', 'ΠΑΝΤΟΥ', 'ΠΑΝΤΩΣ', 'ΠΑΡΑ', 'ΠΕΡΑ', 'ΠΕΡΙ', 'ΠΕΡΙΠΟΥ', 'ΠΕΡΙΣΣΟΤΕΡΟ', 'ΠΕΡΣΙ', 'ΠΕΡΥΣΙ', 'ΠΙΑ', 'ΠΙΘΑΝΟΝ',
                   'ΠΙΟ', 'ΠΙΣΩ', 'ΠΛΑΙ', 'ΠΛΕΟΝ', 'ΠΛΗΝ', 'ΠΟΙΑ', 'ΠΟΙΑΝ', 'ΠΟΙΑΣ', 'ΠΟΙΕΣ', 'ΠΟΙΟ', 'ΠΟΙΟΙ', 'ΠΟΙΟΝ', 'ΠΟΙΟΣ', 'ΠΟΙΟΥ', 'ΠΟΙΟΥΣ',
                   'ΠΟΙΩΝ', 'ΠΟΛΥ', 'ΠΟΣΕΣ', 'ΠΟΣΗ', 'ΠΟΣΗΝ', 'ΠΟΣΗΣ', 'ΠΟΣΟΙ', 'ΠΟΣΟΣ', 'ΠΟΣΟΥΣ', 'ΠΟΤΕ', 'ΠΟΥ', 'ΠΟΥΘΕ', 'ΠΟΥΘΕΝΑ', 'ΠΡΕΠΕΙ',
                   'ΠΡΙΝ', 'ΠΡΟ', 'ΠΡΟΚΕΙΜΕΝΟΥ', 'ΠΡΟΚΕΙΤΑΙ', 'ΠΡΟΠΕΡΣΙ', 'ΠΡΟΣ', 'ΠΡΟΤΟΥ', 'ΠΡΟΧΘΕΣ', 'ΠΡΟΧΤΕΣ', 'ΠΡΩΤΥΤΕΡΑ', 'ΠΩΣ', 'ΣΑΝ', 'ΣΑΣ',
                   'ΣΕ', 'ΣΕΙΣ', 'ΣΗΜΕΡΑ', 'ΣΙΓΑ', 'ΣΟΥ', 'ΣΤΑ', 'ΣΤΗ', 'ΣΤΗΝ', 'ΣΤΗΣ', 'ΣΤΙΣ', 'ΣΤΟ', 'ΣΤΟΝ', 'ΣΤΟΥ', 'ΣΤΟΥΣ', 'ΣΤΩΝ', 'ΣΥΓΧΡΟΝΩΣ',
                   'ΣΥΝ', 'ΣΥΝΑΜΑ', 'ΣΥΝΕΠΩΣ', 'ΣΥΝΗΘΩΣ', 'ΣΥΧΝΑ', 'ΣΥΧΝΑΣ', 'ΣΥΧΝΕΣ', 'ΣΥΧΝΗ', 'ΣΥΧΝΗΝ', 'ΣΥΧΝΗΣ', 'ΣΥΧΝΟ', 'ΣΥΧΝΟΙ', 'ΣΥΧΝΟΝ',
                   'ΣΥΧΝΟΣ', 'ΣΥΧΝΟΥ', 'ΣΥΧΝΟΥ', 'ΣΥΧΝΟΥΣ', 'ΣΥΧΝΩΝ', 'ΣΥΧΝΩΣ', 'ΣΧΕ∆ΟΝ', 'ΣΩΣΤΑ', 'ΤΑ', 'ΤΑ∆Ε', 'ΤΑΥΤΑ', 'ΤΑΥΤΕΣ', 'ΤΑΥΤΗ', 'ΤΑΥΤΗΝ',
                   'ΤΑΥΤΗΣ', 'ΤΑΥΤΟ,ΤΑΥΤΟΝ', 'ΤΑΥΤΟΣ', 'ΤΑΥΤΟΥ', 'ΤΑΥΤΩΝ', 'ΤΑΧΑ', 'ΤΑΧΑΤΕ', 'ΤΕΛΙΚΑ', 'ΤΕΛΙΚΩΣ', 'ΤΕΣ', 'ΤΕΤΟΙΑ', 'ΤΕΤΟΙΑΝ',
                   'ΤΕΤΟΙΑΣ', 'ΤΕΤΟΙΕΣ', 'ΤΕΤΟΙΟ', 'ΤΕΤΟΙΟΙ', 'ΤΕΤΟΙΟΝ', 'ΤΕΤΟΙΟΣ', 'ΤΕΤΟΙΟΥ', 'ΤΕΤΟΙΟΥΣ', 'ΤΕΤΟΙΩΝ', 'ΤΗ', 'ΤΗΝ', 'ΤΗΣ', 'ΤΙ',
                   'ΤΙΠΟΤΑ', 'ΤΙΠΟΤΕ', 'ΤΙΣ', 'ΤΟ', 'ΤΟΙ', 'ΤΟΝ', 'ΤΟΣ', 'ΤΟΣΑ', 'ΤΟΣΕΣ', 'ΤΟΣΗ', 'ΤΟΣΗΝ', 'ΤΟΣΗΣ', 'ΤΟΣΟ', 'ΤΟΣΟΙ', 'ΤΟΣΟΝ', 'ΤΟΣΟΣ',
                   'ΤΟΣΟΥ', 'ΤΟΣΟΥΣ', 'ΤΟΣΩΝ', 'ΤΟΤΕ', 'ΤΟΥ', 'ΤΟΥΛΑΧΙΣΤΟ', 'ΤΟΥΛΑΧΙΣΤΟΝ', 'ΤΟΥΣ', 'ΤΟΥΤΑ', 'ΤΟΥΤΕΣ', 'ΤΟΥΤΗ', 'ΤΟΥΤΗΝ', 'ΤΟΥΤΗΣ',
                   'ΤΟΥΤΟ', 'ΤΟΥΤΟΙ', 'ΤΟΥΤΟΙΣ', 'ΤΟΥΤΟΝ', 'ΤΟΥΤΟΣ', 'ΤΟΥΤΟΥ', 'ΤΟΥΤΟΥΣ', 'ΤΟΥΤΩΝ', 'ΤΥΧΟΝ', 'ΤΩΝ', 'ΤΩΡΑ', 'ΥΠ', 'ΥΠΕΡ', 'ΥΠΟ',
                   'ΥΠΟΨΗ', 'ΥΠΟΨΙΝ', 'ΥΣΤΕΡΑ', 'ΦΕΤΟΣ', 'ΧΑΜΗΛΑ', 'ΧΘΕΣ', 'ΧΤΕΣ', 'ΧΩΡΙΣ', 'ΧΩΡΙΣΤΑ', 'ΨΗΛΑ', 'Ω', 'ΩΡΑΙΑ', 'ΩΣ', 'ΩΣΑΝ', 'ΩΣΟΤΟΥ',
                   'ΩΣΠΟΥ', 'ΩΣΤΕ', 'ΩΣΤΟΣΟ', 'ΩΧ']

# Text Preprocessing

def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


# Prepare Data

X_train = train_s["text"]
y_train = train_s["label"]
X_val = val_s["text"]
y_val = val_s["label"]
X_test = test_s["text"]
y_test = test_s["label"]

for df in [train_s]:
    df["text"] = df["text"].astype(str).apply(clean_text)


# Fit Pipeline

vectorizer = CountVectorizer(
    stop_words=greek_stopwords,
    lowercase=True,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95
)
# Fit & transform τα κείμενα
X_train_vec = vectorizer.fit_transform(X_train)


# Μοντέλο
clf1 = LinearSVC(class_weight='balanced')

# Fit στο resampled data
clf1.fit(X_train_vec, y_train)

# Validation
X_val_vec = vectorizer.transform(X_val)
y_pred_val = clf1.predict(X_val_vec)

# Αποτελέσματα
print(classification_report(y_val, y_pred_val, digits=4))

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'εαν', 'εαυτο', 'εαυτον', 'εαυτου', 'εαυτους', 'εαυτων', 'εγκαιρα', 'εγκαιρως', 'εγω', 'ει', 'ειθε', 'ειμαι', 'ειμαστε', 'εινα', 'ειναι', 'εις', 

              precision    recall  f1-score   support

           0     0.2000    0.3636    0.2581        11
           3     0.0000    0.0000    0.0000         1
           4     0.0000    0.0000    0.0000         1
           5     0.0000    0.0000    0.0000         2
           8     0.9000    0.7500    0.8182        12
           9     0.4167    0.7143    0.5263         7
          11     0.5000    0.5000    0.5000         2
          12     0.0000    0.0000    0.0000         1
          13     0.8750    0.7000    0.7778        10
          14     1.0000    1.0000    1.0000         1
          15     0.0000    0.0000    0.0000         2
          16     1.0000    0.5000    0.6667         2
          17     1.0000    1.0000    1.0000         9
          19     0.2500    0.2500    0.2500         4
          20     1.0000    0.6667    0.8000         3
          21     0.0000    0.0000    0.0000         1
          23     0.0000    0.0000    0.0000         3
          24     1.0000    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

TEST BOW

In [ ]:
# TEST DATA
X_test_vec = vectorizer.transform(X_test)
y_pred_test = clf.predict(X_test_vec)

# Αποτελέσματα
print(classification_report(y_test, y_pred_test, digits=4))

              precision    recall  f1-score   support

           0     0.3333    0.7500    0.4615         8
           4     0.0000    0.0000    0.0000         1
           5     0.0000    0.0000    0.0000         3
           8     0.7333    1.0000    0.8462        11
           9     0.2857    0.5000    0.3636         4
          10     0.0000    0.0000    0.0000         3
          11     1.0000    0.3333    0.5000         3
          12     0.0000    0.0000    0.0000         1
          13     0.6667    0.5000    0.5714         4
          14     1.0000    1.0000    1.0000         1
          17     0.5000    0.5000    0.5000         2
          19     0.0000    0.0000    0.0000         3
          20     1.0000    0.2500    0.4000         4
          22     0.0000    0.0000    0.0000         1
          23     0.0000    0.0000    0.0000         1
          24     0.0000    0.0000    0.0000         2
          25     0.7600    0.7917    0.7755        24
          27     0.7778    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [ ]:
# TEST DATA
X_test_vec = vectorizer.transform(X_test)
y_pred_test = clf1.predict(X_test_vec)

# Αποτελέσματα
print(classification_report(y_test, y_pred_test, digits=4))

              precision    recall  f1-score   support

           0     0.4000    0.7500    0.5217         8
           4     0.0000    0.0000    0.0000         1
           5     0.0000    0.0000    0.0000         3
           8     0.7333    1.0000    0.8462        11
           9     0.3333    0.5000    0.4000         4
          10     0.0000    0.0000    0.0000         3
          11     1.0000    0.3333    0.5000         3
          12     0.0000    0.0000    0.0000         1
          13     0.7500    0.7500    0.7500         4
          14     1.0000    1.0000    1.0000         1
          17     0.5000    0.5000    0.5000         2
          19     0.0000    0.0000    0.0000         3
          20     1.0000    0.2500    0.4000         4
          22     0.0000    0.0000    0.0000         1
          23     0.0000    0.0000    0.0000         1
          24     0.0000    0.0000    0.0000         2
          25     0.7826    0.7500    0.7660        24
          27     0.7778    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

# B1 2.

# Volume

In [ ]:

!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.el.300.vec.gz
!gunzip cc.el.300.vec.gz



--2025-05-23 05:26:36--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.el.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.173.166.74, 18.173.166.31, 18.173.166.48, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.173.166.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1256281783 (1.2G) [binary/octet-stream]
Saving to: ‘cc.el.300.vec.gz’

cc.el.300.vec.gz    100%[===================>]   1.17G  33.7MB/s    in 35s     

2025-05-23 05:27:12 (34.3 MB/s) - ‘cc.el.300.vec.gz’ saved [1256281783/1256281783]



In [ ]:
# pretrained FastText
pretrained_path = "cc.el.300.vec"
print("Loaded pretrained embeddings (FastText Greek)...")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 40.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('el_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Loaded pretrained embeddings (FastText Greek)...


In [ ]:
import spacy
import numpy as np
import re
from gensim.models import Word2Vec, KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from gensim.models import Word2Vec, KeyedVectors

import numpy as np
import re

nlp = spacy.load("el_core_news_sm")

fasttext_model = KeyedVectors.load_word2vec_format(pretrained_path, binary=False)
print("Done loading.")


# Text Preprocessing


greek_stopwords = ['Α∆ΙΑΚΟΠΑ', 'ΑΙ', 'ΑΚΟΜΑ', 'ΑΚΟΜΗ', 'ΑΚΡΙΒΩΣ', 'ΑΛΗΘΕΙΑ', 'ΑΛΗΘΙΝΑ', 'ΑΛΛΑ', 'ΑΛΛΑΧΟΥ', 'ΑΛΛΕΣ', 'ΑΛΛΗ', 'ΑΛΛΗΝ', 'ΑΛΛΗΣ',
                   'ΑΛΛΙΩΣ', 'ΑΛΛΙΩΤΙΚΑ', 'ΑΛΛΟ', 'ΑΛΛΟΙ', 'ΑΛΛΟΙΩΣ', 'ΑΛΛΟΙΩΤΙΚΑ', 'ΑΛΛΟΝ', 'ΑΛΛΟΣ', 'ΑΛΛΟΤΕ', 'ΑΛΛΟΥ', 'ΑΛΛΟΥΣ', 'ΑΛΛΩΝ', 'ΑΜΑ',
                   'ΑΜΕΣΑ', 'ΑΜΕΣΩΣ', 'ΑΝ', 'ΑΝΑ', 'ΑΝΑΜΕΣΑ', 'ΑΝΑΜΕΤΑΞΥ', 'ΑΝΕΥ', 'ΑΝΤΙ', 'ΑΝΤΙΠΕΡΑ', 'ΑΝΤΙΣ', 'ΑΝΩ', 'ΑΝΩΤΕΡΩ', 'ΑΞΑΦΝΑ', 'ΑΠ',
                   'ΑΠΕΝΑΝΤΙ', 'ΑΠΟ', 'ΑΠΟΨΕ', 'ΑΡΑ', 'ΑΡΑΓΕ', 'ΑΡΓΑ', 'ΑΡΓΟΤΕΡΟ', 'ΑΡΙΣΤΕΡΑ', 'ΑΡΚΕΤΑ', 'ΑΡΧΙΚΑ', 'ΑΣ', 'ΑΥΡΙΟ', 'ΑΥΤΑ', 'ΑΥΤΕΣ',
                   'ΑΥΤΗ', 'ΑΥΤΗΝ', 'ΑΥΤΗΣ', 'ΑΥΤΟ', 'ΑΥΤΟΙ', 'ΑΥΤΟΝ', 'ΑΥΤΟΣ', 'ΑΥΤΟΥ', 'ΑΥΤΟΥΣ', 'ΑΥΤΩΝ', 'ΑΦΟΤΟΥ', 'ΑΦΟΥ', 'ΒΕΒΑΙΑ', 'ΒΕΒΑΙΟΤΑΤΑ',
                   'ΓΙ', 'ΓΙΑ', 'ΓΡΗΓΟΡΑ', 'ΓΥΡΩ', '∆Α', '∆Ε', '∆ΕΙΝΑ', '∆ΕΝ', '∆ΕΞΙΑ', '∆ΗΘΕΝ', '∆ΗΛΑ∆Η', '∆Ι', '∆ΙΑ', '∆ΙΑΡΚΩΣ', '∆ΙΚΑ', '∆ΙΚΟ',
                   '∆ΙΚΟΙ', '∆ΙΚΟΣ', '∆ΙΚΟΥ', '∆ΙΚΟΥΣ', '∆ΙΟΛΟΥ', '∆ΙΠΛΑ', '∆ΙΧΩΣ', 'ΕΑΝ', 'ΕΑΥΤΟ', 'ΕΑΥΤΟΝ', 'ΕΑΥΤΟΥ', 'ΕΑΥΤΟΥΣ', 'ΕΑΥΤΩΝ',
                   'ΕΓΚΑΙΡΑ', 'ΕΓΚΑΙΡΩΣ', 'ΕΓΩ', 'Ε∆Ω', 'ΕΙ∆ΕΜΗ', 'ΕΙΘΕ', 'ΕΙΜΑΙ', 'ΕΙΜΑΣΤΕ', 'ΕΙΝΑΙ', 'ΕΙΣ', 'ΕΙΣΑΙ', 'ΕΙΣΑΣΤΕ', 'ΕΙΣΤΕ', 'ΕΙΤΕ',
                   'ΕΙΧΑ', 'ΕΙΧΑΜΕ', 'ΕΙΧΑΝ', 'ΕΙΧΑΤΕ', 'ΕΙΧΕ', 'ΕΙΧΕΣ', 'ΕΚΑΣΤΑ', 'ΕΚΑΣΤΕΣ', 'ΕΚΑΣΤΗ', 'ΕΚΑΣΤΗΝ', 'ΕΚΑΣΤΗΣ', 'ΕΚΑΣΤΟ', 'ΕΚΑΣΤΟΙ',
                   'ΕΚΑΣΤΟΝ', 'ΕΚΑΣΤΟΣ', 'ΕΚΑΣΤΟΥ', 'ΕΚΑΣΤΟΥΣ', 'ΕΚΑΣΤΩΝ', 'ΕΚΕΙ', 'ΕΚΕΙΝΑ', 'ΕΚΕΙΝΕΣ', 'ΕΚΕΙΝΗ', 'ΕΚΕΙΝΗΝ', 'ΕΚΕΙΝΗΣ', 'ΕΚΕΙΝΟ',
                   'ΕΚΕΙΝΟΙ', 'ΕΚΕΙΝΟΝ', 'ΕΚΕΙΝΟΣ', 'ΕΚΕΙΝΟΥ', 'ΕΚΕΙΝΟΥΣ', 'ΕΚΕΙΝΩΝ', 'ΕΚΤΟΣ', 'ΕΜΑΣ', 'ΕΜΕΙΣ', 'ΕΜΕΝΑ', 'ΕΜΠΡΟΣ', 'ΕΝ', 'ΕΝΑ',
                   'ΕΝΑΝ', 'ΕΝΑΣ', 'ΕΝΟΣ', 'ΕΝΤΕΛΩΣ', 'ΕΝΤΟΣ', 'ΕΝΤΩΜΕΤΑΞΥ', 'ΕΝΩ', 'ΕΞ', 'ΕΞΑΦΝΑ', 'ΕΞΗΣ', 'ΕΞΙΣΟΥ', 'ΕΞΩ', 'ΕΠΑΝΩ', 'ΕΠΕΙ∆Η',
                   'ΕΠΕΙΤΑ', 'ΕΠΙ', 'ΕΠΙΣΗΣ', 'ΕΠΟΜΕΝΩΣ', 'ΕΣΑΣ', 'ΕΣΕΙΣ', 'ΕΣΕΝΑ', 'ΕΣΤΩ', 'ΕΣΥ', 'ΕΤΕΡΑ', 'ΕΤΕΡΑΙ', 'ΕΤΕΡΑΣ', 'ΕΤΕΡΕΣ', 'ΕΤΕΡΗ',
                   'ΕΤΕΡΗΣ', 'ΕΤΕΡΟ', 'ΕΤΕΡΟΙ', 'ΕΤΕΡΟΝ', 'ΕΤΕΡΟΣ', 'ΕΤΕΡΟΥ', 'ΕΤΕΡΟΥΣ', 'ΕΤΕΡΩΝ', 'ΕΤΟΥΤΑ', 'ΕΤΟΥΤΕΣ', 'ΕΤΟΥΤΗ', 'ΕΤΟΥΤΗΝ',
                   'ΕΤΟΥΤΗΣ', 'ΕΤΟΥΤΟ', 'ΕΤΟΥΤΟΙ', 'ΕΤΟΥΤΟΝ', 'ΕΤΟΥΤΟΣ', 'ΕΤΟΥΤΟΥ', 'ΕΤΟΥΤΟΥΣ', 'ΕΤΟΥΤΩΝ', 'ΕΤΣΙ', 'ΕΥΓΕ', 'ΕΥΘΥΣ', 'ΕΥΤΥΧΩΣ',
                   'ΕΦΕΞΗΣ', 'ΕΧΕΙ', 'ΕΧΕΙΣ', 'ΕΧΕΤΕ', 'ΕΧΘΕΣ', 'ΕΧΟΜΕ', 'ΕΧΟΥΜΕ', 'ΕΧΟΥΝ', 'ΕΧΤΕΣ', 'ΕΧΩ', 'ΕΩΣ', 'Η', 'Η∆Η', 'ΗΜΑΣΤΑΝ', 'ΗΜΑΣΤΕ',
                   'ΗΜΟΥΝ', 'ΗΣΑΣΤΑΝ', 'ΗΣΑΣΤΕ', 'ΗΣΟΥΝ', 'ΗΤΑΝ', 'ΗΤΑΝΕ', 'ΗΤΟΙ', 'ΗΤΤΟΝ', 'ΘΑ', 'Ι', 'Ι∆ΙΑ', 'Ι∆ΙΑΝ', 'Ι∆ΙΑΣ', 'Ι∆ΙΕΣ', 'Ι∆ΙΟ',
                   'Ι∆ΙΟΙ', 'Ι∆ΙΟΝ', 'Ι∆ΙΟΣ', 'Ι∆ΙΟΥ', 'Ι∆ΙΟΥΣ', 'Ι∆ΙΩΝ', 'Ι∆ΙΩΣ', 'ΙΙ', 'ΙΙΙ', 'ΙΣΑΜΕ', 'ΙΣΙΑ', 'ΙΣΩΣ', 'ΚΑΘΕ', 'ΚΑΘΕΜΙΑ',
                   'ΚΑΘΕΜΙΑΣ', 'ΚΑΘΕΝΑ', 'ΚΑΘΕΝΑΣ', 'ΚΑΘΕΝΟΣ', 'ΚΑΘΕΤΙ', 'ΚΑΘΟΛΟΥ', 'ΚΑΘΩΣ', 'ΚΑΙ', 'ΚΑΚΑ', 'ΚΑΚΩΣ', 'ΚΑΛΑ', 'ΚΑΛΩΣ', 'ΚΑΜΙΑ',
                   'ΚΑΜΙΑΝ', 'ΚΑΜΙΑΣ', 'ΚΑΜΠΟΣΑ', 'ΚΑΜΠΟΣΕΣ', 'ΚΑΜΠΟΣΗ', 'ΚΑΜΠΟΣΗΝ', 'ΚΑΜΠΟΣΗΣ', 'ΚΑΜΠΟΣΟ', 'ΚΑΜΠΟΣΟΙ', 'ΚΑΜΠΟΣΟΝ', 'ΚΑΜΠΟΣΟΣ',
                   'ΚΑΜΠΟΣΟΥ', 'ΚΑΜΠΟΣΟΥΣ', 'ΚΑΜΠΟΣΩΝ', 'ΚΑΝΕΙΣ', 'ΚΑΝΕΝ', 'ΚΑΝΕΝΑ', 'ΚΑΝΕΝΑΝ', 'ΚΑΝΕΝΑΣ', 'ΚΑΝΕΝΟΣ', 'ΚΑΠΟΙΑ', 'ΚΑΠΟΙΑΝ', 'ΚΑΠΟΙΑΣ',
                   'ΚΑΠΟΙΕΣ', 'ΚΑΠΟΙΟ', 'ΚΑΠΟΙΟΙ', 'ΚΑΠΟΙΟΝ', 'ΚΑΠΟΙΟΣ', 'ΚΑΠΟΙΟΥ', 'ΚΑΠΟΙΟΥΣ', 'ΚΑΠΟΙΩΝ', 'ΚΑΠΟΤΕ', 'ΚΑΠΟΥ', 'ΚΑΠΩΣ', 'ΚΑΤ', 'ΚΑΤΑ',
                   'ΚΑΤΙ', 'ΚΑΤΙΤΙ', 'ΚΑΤΟΠΙΝ', 'ΚΑΤΩ', 'ΚΙΟΛΑΣ', 'ΚΛΠ', 'ΚΟΝΤΑ', 'ΚΤΛ', 'ΚΥΡΙΩΣ', 'ΛΙΓΑΚΙ', 'ΛΙΓΟ', 'ΛΙΓΩΤΕΡΟ', 'ΛΟΓΩ', 'ΛΟΙΠΑ',
                   'ΛΟΙΠΟΝ', 'ΜΑ', 'ΜΑΖΙ', 'ΜΑΚΑΡΙ', 'ΜΑΚΡΥΑ', 'ΜΑΛΙΣΤΑ', 'ΜΑΛΛΟΝ', 'ΜΑΣ', 'ΜΕ', 'ΜΕΘΑΥΡΙΟ', 'ΜΕΙΟΝ', 'ΜΕΛΕΙ', 'ΜΕΛΛΕΤΑΙ', 'ΜΕΜΙΑΣ',
                   'ΜΕΝ', 'ΜΕΡΙΚΑ', 'ΜΕΡΙΚΕΣ', 'ΜΕΡΙΚΟΙ', 'ΜΕΡΙΚΟΥΣ', 'ΜΕΡΙΚΩΝ', 'ΜΕΣΑ', 'ΜΕΤ', 'ΜΕΤΑ', 'ΜΕΤΑΞΥ', 'ΜΕΧΡΙ', 'ΜΗ', 'ΜΗ∆Ε', 'ΜΗΝ',
                   'ΜΗΠΩΣ', 'ΜΗΤΕ', 'ΜΙΑ', 'ΜΙΑΝ', 'ΜΙΑΣ', 'ΜΟΛΙΣ', 'ΜΟΛΟΝΟΤΙ', 'ΜΟΝΑΧΑ', 'ΜΟΝΕΣ', 'ΜΟΝΗ', 'ΜΟΝΗΝ', 'ΜΟΝΗΣ', 'ΜΟΝΟ', 'ΜΟΝΟΙ',
                   'ΜΟΝΟΜΙΑΣ', 'ΜΟΝΟΣ', 'ΜΟΝΟΥ', 'ΜΟΝΟΥΣ', 'ΜΟΝΩΝ', 'ΜΟΥ', 'ΜΠΟΡΕΙ', 'ΜΠΟΡΟΥΝ', 'ΜΠΡΑΒΟ', 'ΜΠΡΟΣ', 'ΝΑ', 'ΝΑΙ', 'ΝΩΡΙΣ', 'ΞΑΝΑ',
                   'ΞΑΦΝΙΚΑ', 'Ο', 'ΟΙ', 'ΟΛΑ', 'ΟΛΕΣ', 'ΟΛΗ', 'ΟΛΗΝ', 'ΟΛΗΣ', 'ΟΛΟ', 'ΟΛΟΓΥΡΑ', 'ΟΛΟΙ', 'ΟΛΟΝ', 'ΟΛΟΝΕΝ', 'ΟΛΟΣ', 'ΟΛΟΤΕΛΑ', 'ΟΛΟΥ',
                   'ΟΛΟΥΣ', 'ΟΛΩΝ', 'ΟΛΩΣ', 'ΟΛΩΣ∆ΙΟΛΟΥ', 'ΟΜΩΣ', 'ΟΠΟΙΑ', 'ΟΠΟΙΑ∆ΗΠΟΤΕ', 'ΟΠΟΙΑΝ', 'ΟΠΟΙΑΝ∆ΗΠΟΤΕ', 'ΟΠΟΙΑΣ', 'ΟΠΟΙΑΣ∆ΗΠΟΤΕ',
                   'ΟΠΟΙ∆ΗΠΟΤΕ', 'ΟΠΟΙΕΣ', 'ΟΠΟΙΕΣ∆ΗΠΟΤΕ', 'ΟΠΟΙΟ', 'ΟΠΟΙΟ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΙ', 'ΟΠΟΙΟΝ', 'ΟΠΟΙΟΝ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΣ', 'ΟΠΟΙΟΣ∆ΗΠΟΤΕ',
                   'ΟΠΟΙΟΥ', 'ΟΠΟΙΟΥ∆ΗΠΟΤΕ', 'ΟΠΟΙΟΥΣ', 'ΟΠΟΙΟΥΣ∆ΗΠΟΤΕ', 'ΟΠΟΙΩΝ', 'ΟΠΟΙΩΝ∆ΗΠΟΤΕ', 'ΟΠΟΤΕ', 'ΟΠΟΤΕ∆ΗΠΟΤΕ', 'ΟΠΟΥ', 'ΟΠΟΥ∆ΗΠΟΤΕ',
                   'ΟΠΩΣ', 'ΟΡΙΣΜΕΝΑ', 'ΟΡΙΣΜΕΝΕΣ', 'ΟΡΙΣΜΕΝΩΝ', 'ΟΡΙΣΜΕΝΩΣ', 'ΟΣΑ', 'ΟΣΑ∆ΗΠΟΤΕ', 'ΟΣΕΣ', 'ΟΣΕΣ∆ΗΠΟΤΕ', 'ΟΣΗ', 'ΟΣΗ∆ΗΠΟΤΕ', 'ΟΣΗΝ',
                   'ΟΣΗΝ∆ΗΠΟΤΕ', 'ΟΣΗΣ', 'ΟΣΗΣ∆ΗΠΟΤΕ', 'ΟΣΟ', 'ΟΣΟ∆ΗΠΟΤΕ', 'ΟΣΟΙ', 'ΟΣΟΙ∆ΗΠΟΤΕ', 'ΟΣΟΝ', 'ΟΣΟΝ∆ΗΠΟΤΕ', 'ΟΣΟΣ', 'ΟΣΟΣ∆ΗΠΟΤΕ', 'ΟΣΟΥ',
                   'ΟΣΟΥ∆ΗΠΟΤΕ', 'ΟΣΟΥΣ', 'ΟΣΟΥΣ∆ΗΠΟΤΕ', 'ΟΣΩΝ', 'ΟΣΩΝ∆ΗΠΟΤΕ', 'ΟΤΑΝ', 'ΟΤΙ', 'ΟΤΙ∆ΗΠΟΤΕ', 'ΟΤΟΥ', 'ΟΥ', 'ΟΥ∆Ε', 'ΟΥΤΕ', 'ΟΧΙ',
                   'ΠΑΛΙ', 'ΠΑΝΤΟΤΕ', 'ΠΑΝΤΟΥ', 'ΠΑΝΤΩΣ', 'ΠΑΡΑ', 'ΠΕΡΑ', 'ΠΕΡΙ', 'ΠΕΡΙΠΟΥ', 'ΠΕΡΙΣΣΟΤΕΡΟ', 'ΠΕΡΣΙ', 'ΠΕΡΥΣΙ', 'ΠΙΑ', 'ΠΙΘΑΝΟΝ',
                   'ΠΙΟ', 'ΠΙΣΩ', 'ΠΛΑΙ', 'ΠΛΕΟΝ', 'ΠΛΗΝ', 'ΠΟΙΑ', 'ΠΟΙΑΝ', 'ΠΟΙΑΣ', 'ΠΟΙΕΣ', 'ΠΟΙΟ', 'ΠΟΙΟΙ', 'ΠΟΙΟΝ', 'ΠΟΙΟΣ', 'ΠΟΙΟΥ', 'ΠΟΙΟΥΣ',
                   'ΠΟΙΩΝ', 'ΠΟΛΥ', 'ΠΟΣΕΣ', 'ΠΟΣΗ', 'ΠΟΣΗΝ', 'ΠΟΣΗΣ', 'ΠΟΣΟΙ', 'ΠΟΣΟΣ', 'ΠΟΣΟΥΣ', 'ΠΟΤΕ', 'ΠΟΥ', 'ΠΟΥΘΕ', 'ΠΟΥΘΕΝΑ', 'ΠΡΕΠΕΙ',
                   'ΠΡΙΝ', 'ΠΡΟ', 'ΠΡΟΚΕΙΜΕΝΟΥ', 'ΠΡΟΚΕΙΤΑΙ', 'ΠΡΟΠΕΡΣΙ', 'ΠΡΟΣ', 'ΠΡΟΤΟΥ', 'ΠΡΟΧΘΕΣ', 'ΠΡΟΧΤΕΣ', 'ΠΡΩΤΥΤΕΡΑ', 'ΠΩΣ', 'ΣΑΝ', 'ΣΑΣ',
                   'ΣΕ', 'ΣΕΙΣ', 'ΣΗΜΕΡΑ', 'ΣΙΓΑ', 'ΣΟΥ', 'ΣΤΑ', 'ΣΤΗ', 'ΣΤΗΝ', 'ΣΤΗΣ', 'ΣΤΙΣ', 'ΣΤΟ', 'ΣΤΟΝ', 'ΣΤΟΥ', 'ΣΤΟΥΣ', 'ΣΤΩΝ', 'ΣΥΓΧΡΟΝΩΣ',
                   'ΣΥΝ', 'ΣΥΝΑΜΑ', 'ΣΥΝΕΠΩΣ', 'ΣΥΝΗΘΩΣ', 'ΣΥΧΝΑ', 'ΣΥΧΝΑΣ', 'ΣΥΧΝΕΣ', 'ΣΥΧΝΗ', 'ΣΥΧΝΗΝ', 'ΣΥΧΝΗΣ', 'ΣΥΧΝΟ', 'ΣΥΧΝΟΙ', 'ΣΥΧΝΟΝ',
                   'ΣΥΧΝΟΣ', 'ΣΥΧΝΟΥ', 'ΣΥΧΝΟΥ', 'ΣΥΧΝΟΥΣ', 'ΣΥΧΝΩΝ', 'ΣΥΧΝΩΣ', 'ΣΧΕ∆ΟΝ', 'ΣΩΣΤΑ', 'ΤΑ', 'ΤΑ∆Ε', 'ΤΑΥΤΑ', 'ΤΑΥΤΕΣ', 'ΤΑΥΤΗ', 'ΤΑΥΤΗΝ',
                   'ΤΑΥΤΗΣ', 'ΤΑΥΤΟ,ΤΑΥΤΟΝ', 'ΤΑΥΤΟΣ', 'ΤΑΥΤΟΥ', 'ΤΑΥΤΩΝ', 'ΤΑΧΑ', 'ΤΑΧΑΤΕ', 'ΤΕΛΙΚΑ', 'ΤΕΛΙΚΩΣ', 'ΤΕΣ', 'ΤΕΤΟΙΑ', 'ΤΕΤΟΙΑΝ',
                   'ΤΕΤΟΙΑΣ', 'ΤΕΤΟΙΕΣ', 'ΤΕΤΟΙΟ', 'ΤΕΤΟΙΟΙ', 'ΤΕΤΟΙΟΝ', 'ΤΕΤΟΙΟΣ', 'ΤΕΤΟΙΟΥ', 'ΤΕΤΟΙΟΥΣ', 'ΤΕΤΟΙΩΝ', 'ΤΗ', 'ΤΗΝ', 'ΤΗΣ', 'ΤΙ',
                   'ΤΙΠΟΤΑ', 'ΤΙΠΟΤΕ', 'ΤΙΣ', 'ΤΟ', 'ΤΟΙ', 'ΤΟΝ', 'ΤΟΣ', 'ΤΟΣΑ', 'ΤΟΣΕΣ', 'ΤΟΣΗ', 'ΤΟΣΗΝ', 'ΤΟΣΗΣ', 'ΤΟΣΟ', 'ΤΟΣΟΙ', 'ΤΟΣΟΝ', 'ΤΟΣΟΣ',
                   'ΤΟΣΟΥ', 'ΤΟΣΟΥΣ', 'ΤΟΣΩΝ', 'ΤΟΤΕ', 'ΤΟΥ', 'ΤΟΥΛΑΧΙΣΤΟ', 'ΤΟΥΛΑΧΙΣΤΟΝ', 'ΤΟΥΣ', 'ΤΟΥΤΑ', 'ΤΟΥΤΕΣ', 'ΤΟΥΤΗ', 'ΤΟΥΤΗΝ', 'ΤΟΥΤΗΣ',
                   'ΤΟΥΤΟ', 'ΤΟΥΤΟΙ', 'ΤΟΥΤΟΙΣ', 'ΤΟΥΤΟΝ', 'ΤΟΥΤΟΣ', 'ΤΟΥΤΟΥ', 'ΤΟΥΤΟΥΣ', 'ΤΟΥΤΩΝ', 'ΤΥΧΟΝ', 'ΤΩΝ', 'ΤΩΡΑ', 'ΥΠ', 'ΥΠΕΡ', 'ΥΠΟ',
                   'ΥΠΟΨΗ', 'ΥΠΟΨΙΝ', 'ΥΣΤΕΡΑ', 'ΦΕΤΟΣ', 'ΧΑΜΗΛΑ', 'ΧΘΕΣ', 'ΧΤΕΣ', 'ΧΩΡΙΣ', 'ΧΩΡΙΣΤΑ', 'ΨΗΛΑ', 'Ω', 'ΩΡΑΙΑ', 'ΩΣ', 'ΩΣΑΝ', 'ΩΣΟΤΟΥ',
                   'ΩΣΠΟΥ', 'ΩΣΤΕ', 'ΩΣΤΟΣΟ', 'ΩΧ']


def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# Prepare data
X_train = train_v["text"]
y_train = train_v["label"]
X_val = val_v["text"]
y_val = val_v["label"]
X_test = test_v["text"]
y_test = test_v["label"]

for df in [train_v]:
    df["text"] = df["text"].astype(str).apply(clean_text)


def tokenize(text):
    return [token.text for token in nlp(text)]

train_sentences = [[token.text for token in doc] for doc in nlp.pipe(X_train, batch_size=32)]
val_sentences   = [[token.text for token in doc] for doc in nlp.pipe(X_val, batch_size=32)]
test_sentences  = [[token.text for token in doc] for doc in nlp.pipe(X_test, batch_size=32)]


#Δημιουργία νέου Word2Vec μοντέλου για fine-tuning
w2v_model = Word2Vec(vector_size=300, min_count=1)  # 300 για να ταιριάζει με FastText

#Build vocab από τα train δεδομένα
w2v_model.build_vocab(train_sentences)

#Ενημέρωση του vocab με pretrained vectors

pretrained_vocab = list(fasttext_model.key_to_index.keys())
pretrained_vectors = fasttext_model.vectors

# Προσθήκη pretrained λέξεων στο Word2Vec μοντέλο
print("Adding pretrained vectors to Word2Vec model...")
w2v_model.wv.add_vectors(pretrained_vocab, pretrained_vectors)


w2v_model.wv.vectors_lockf = np.ones(len(w2v_model.wv), dtype=np.float32)

# Fine-tuning με τα train δεδομένα
print("Fine-tuning embeddings with your training data...")
w2v_model.train(train_sentences, total_examples=len(train_sentences), epochs=5)

print("Fine-tuning complete.")

# Δημιουργία document embeddings (μέσος όρος λέξεων)
def document_vector(doc, model):
    doc = [word for word in doc if word in model.wv]
    return np.mean(model.wv[doc], axis=0) if doc else np.zeros(model.vector_size)

X_train_vec = np.array([document_vector(words, w2v_model) for words in train_sentences])
X_val_vec = np.array([document_vector(words, w2v_model) for words in val_sentences])

#classifier
clf1 = LogisticRegression(max_iter=200)
clf1.fit(X_train_vec, y_train)

print("Classifier trained.")

# Για validation

y_pred_val = clf1.predict(X_val_vec)


print(classification_report(y_val, y_pred_val, digits=4))


Done loading.
Adding pretrained vectors to Word2Vec model...
Fine-tuning embeddings with your training data...
Fine-tuning complete.
Classifier trained.
              precision    recall  f1-score   support

           0     0.5511    0.5187    0.5344       187
           1     0.5459    0.5810    0.5629       389
           2     0.5918    0.3671    0.4531        79
           3     0.4646    0.4836    0.4739       244
           4     0.5795    0.5896    0.5845       346
           5     0.6598    0.5926    0.6244       108
           6     0.6786    0.6786    0.6786       224
           7     0.4156    0.2883    0.3404       111
           8     0.5114    0.4945    0.5028        91
           9     0.8333    0.3521    0.4950        71
          10     0.6146    0.3688    0.4609       160
          11     0.5897    0.6250    0.6069       184
          12     0.5893    0.3976    0.4748        83
          13     0.4878    0.5263    0.5063       152
          14     0.7033    0.7574   

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
import spacy
import numpy as np
import re
from gensim.models import KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

nlp = spacy.load("el_core_news_sm")

# Load pretrained FastText model
fasttext_model = KeyedVectors.load_word2vec_format(pretrained_path, binary=False)
print("Done loading.")

def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

for df in [train_v]:
    df["text"] = df["text"].astype(str).apply(clean_text)

# Prepare data
X_train, y_train = train_v["text"], train_v["label"]
X_val, y_val = val_v["text"], val_v["label"]

# Tokenize
train_sentences = [[token.text for token in doc] for doc in nlp.pipe(X_train, batch_size=32)]
val_sentences   = [[token.text for token in doc] for doc in nlp.pipe(X_val, batch_size=32)]

# Create document embeddings (average word vectors)
def document_vector(doc, model):
    vectors = [model[word] for word in doc if word in model.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

X_train_vec = np.array([document_vector(words, fasttext_model) for words in train_sentences])
X_val_vec   = np.array([document_vector(words, fasttext_model) for words in val_sentences])

# Classifier
clf = LogisticRegression(max_iter=200)
clf.fit(X_train_vec, y_train)

# Validation
y_pred_val = clf.predict(X_val_vec)
print(classification_report(y_val, y_pred_val, digits=4))


Done loading.
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       187
           1     0.3674    0.4344    0.3981       389
           2     0.0000    0.0000    0.0000        79
           3     1.0000    0.0082    0.0163       244
           4     0.3198    0.2283    0.2664       346
           5     0.1111    0.0278    0.0444       108
           6     0.6591    0.1295    0.2164       224
           7     0.0000    0.0000    0.0000       111
           8     0.0000    0.0000    0.0000        91
           9     0.0000    0.0000    0.0000        71
          10     0.0000    0.0000    0.0000       160
          11     0.6500    0.0707    0.1275       184
          12     0.0000    0.0000    0.0000        83
          13     0.0000    0.0000    0.0000       152
          14     0.6818    0.0888    0.1571       169
          15     0.4167    0.0323    0.0599       155
          16     0.0000    0.0000    0.0000       102
          17 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import spacy
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


# Prepare data
X_train, y_train = train_v["text"], train_v["label"]
X_val, y_val = val_v["text"], val_v["label"]

# Preprocessing
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


for df in [train_v]:
    df["text"] = df["text"].astype(str).apply(clean_text)

# Tokenize
train_sentences = [[token.text for token in doc] for doc in nlp.pipe(train_v["text"], batch_size=32)]


# Εκπαίδευση Word2Vec μοντέλου
w2v_model = Word2Vec(
    sentences=train_sentences,
    vector_size=100,       # μέγεθος vector
    window=5,              # context window
    min_count=1,           # εμφάνιση λέξης τουλάχιστον min_count φορές
    workers=4,             # parallel threads
    sg=1
)

# Function για μετατροπή document σε embedding
def document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)
# Tokenize
val_sentences   = [[token.text for token in doc] for doc in nlp.pipe(val_v["text"], batch_size=32)]

# Μετατροπή όλων σε embeddings
X_train_vec = np.array([document_vector(words, w2v_model) for words in train_sentences])
X_val_vec   = np.array([document_vector(words, w2v_model) for words in val_sentences])


# Labels
y_train = train_v["label"]
y_val   = val_v["label"]


# Classifier
clf3 = LogisticRegression(max_iter=200)
clf3.fit(X_train_vec, y_train)

# Validation
y_pred_val = clf3.predict(X_val_vec)
print("Validation Results:\n")
print(classification_report(y_val, y_pred_val, digits=4))



Validation Results:

              precision    recall  f1-score   support

           0     0.5987    0.5027    0.5465       187
           1     0.4540    0.6093    0.5203       389
           2     0.6250    0.1266    0.2105        79
           3     0.4669    0.4918    0.4790       244
           4     0.5524    0.6705    0.6057       346
           5     0.6792    0.3333    0.4472       108
           6     0.6906    0.6875    0.6890       224
           7     0.5484    0.1532    0.2394       111
           8     0.5789    0.3626    0.4459        91
           9     0.5000    0.0986    0.1647        71
          10     0.5132    0.2437    0.3305       160
          11     0.5746    0.7120    0.6359       184
          12     0.7647    0.1566    0.2600        83
          13     0.5435    0.4934    0.5172       152
          14     0.6105    0.6213    0.6158       169
          15     0.6930    0.5097    0.5874       155
          16     0.6269    0.4118    0.4970       102
      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import spacy
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


# Prepare data
X_train, y_train = train_v["text"], train_v["label"]
X_val, y_val = val_v["text"], val_v["label"]

# Preprocessing
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


for df in [train_v]:
    df["text"] = df["text"].astype(str).apply(clean_text)

# train sentences από το προηγούμενο

# Εκπαίδευση δικού μου Word2Vec μοντέλου
w2v_model = Word2Vec(
    sentences=train_sentences,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    sg=1
)

# επιπλέον training epochs για καλύτερη σύγκλιση
w2v_model.train(
    train_sentences,
    total_examples=len(train_sentences),
    epochs=10
)

#μετατροπή document σε embedding
def document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)


#val_sentences από πριν

# Μετατροπή όλων σε embeddings
X_train_vec = np.array([document_vector(words, w2v_model) for words in train_sentences])
X_val_vec   = np.array([document_vector(words, w2v_model) for words in val_sentences])

# Labels
y_train = train_v["label"]
y_val   = val_v["label"]

# Classifier
clf4 = LogisticRegression(max_iter=200)
clf4.fit(X_train_vec, y_train)

# Validation
y_pred_val = clf4.predict(X_val_vec)
print("Validation Results:\n")
print(classification_report(y_val, y_pred_val, digits=4))


Validation Results:

              precision    recall  f1-score   support

           0     0.6689    0.5401    0.5976       187
           1     0.5359    0.6530    0.5886       389
           2     0.5750    0.2911    0.3866        79
           3     0.4928    0.5574    0.5231       244
           4     0.6114    0.7139    0.6587       346
           5     0.7925    0.3889    0.5217       108
           6     0.7824    0.7545    0.7682       224
           7     0.6545    0.3243    0.4337       111
           8     0.5833    0.3846    0.4636        91
           9     0.7000    0.1972    0.3077        71
          10     0.5588    0.2375    0.3333       160
          11     0.6027    0.7174    0.6551       184
          12     0.7407    0.2410    0.3636        83
          13     0.5833    0.5526    0.5676       152
          14     0.6536    0.6923    0.6724       169
          15     0.7207    0.5161    0.6015       155
          16     0.6341    0.5098    0.5652       102
      

In [ ]:
import spacy
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


# Prepare data
X_train, y_train = train_v["text"], train_v["label"]
X_val, y_val = val_v["text"], val_v["label"]

# Preprocessing function
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# Καθάρισε τα κείμενα
for df in [train_v]:
    df["text"] = df["text"].astype(str).apply(clean_text)

# train sentences από το προηγούμενο

# Εκπαίδευση δικού μου Word2Vec μοντέλου
w2v_model = Word2Vec(
    sentences=train_sentences,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    sg=1
)

# επιπλέον training epochs για καλύτερη σύγκλιση
w2v_model.train(
    train_sentences,
    total_examples=len(train_sentences),
    epochs=20
)

# Function για μετατροπή document σε embedding
def document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)


#val_sentences από πριν

# Μετατροπή όλων σε embeddings
X_train_vec = np.array([document_vector(words, w2v_model) for words in train_sentences])
X_val_vec   = np.array([document_vector(words, w2v_model) for words in val_sentences])


# Labels
y_train = train_v["label"]
y_val   = val_v["label"]


# Classifier
clf5 = LogisticRegression(max_iter=200)
clf5.fit(X_train_vec, y_train)

# Validation
y_pred_val = clf5.predict(X_val_vec)
print("Validation Results:\n")
print(classification_report(y_val, y_pred_val, digits=4))

Validation Results:

              precision    recall  f1-score   support

           0     0.6500    0.5561    0.5994       187
           1     0.5667    0.6658    0.6123       389
           2     0.6111    0.2785    0.3826        79
           3     0.5000    0.5656    0.5308       244
           4     0.6000    0.7110    0.6508       346
           5     0.7407    0.3704    0.4938       108
           6     0.7857    0.7857    0.7857       224
           7     0.6909    0.3423    0.4578       111
           8     0.6667    0.3736    0.4789        91
           9     0.8571    0.1690    0.2824        71
          10     0.5926    0.3000    0.3983       160
          11     0.6062    0.7446    0.6683       184
          12     0.7647    0.3133    0.4444        83
          13     0.5887    0.5461    0.5666       152
          14     0.6824    0.6864    0.6844       169
          15     0.7297    0.5226    0.6090       155
          16     0.5882    0.4902    0.5348       102
      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import spacy
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


# Prepare data
X_train, y_train = train_v["text"], train_v["label"]
X_val, y_val = val_v["text"], val_v["label"]


stopwords = nlp.Defaults.stop_words
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    tokens = [word for word in text.strip().split() if word not in stopwords]
    return " ".join(tokens)


# Καθάρισε τα κείμενα
for df in [train_v]:
    df["text"] = df["text"].astype(str).apply(clean_text)

# train sentences από το προηγούμενο

# Εκπαίδευση δικού μου Word2Vec μοντέλου
w2v_model = Word2Vec(
    sentences=train_sentences,
    vector_size=300,
    window=8,
    min_count=4,
    workers=4,
    sg=1
)

# επιπλέον training epochs για καλύτερη σύγκλιση
w2v_model.train(
    train_sentences,
    total_examples=len(train_sentences),
    epochs=30
)

# Function για μετατροπή document σε embedding
def document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)


#val_sentences από πριν

# Μετατροπή όλων σε embeddings
X_train_vec = np.array([document_vector(words, w2v_model) for words in train_sentences])
X_val_vec   = np.array([document_vector(words, w2v_model) for words in val_sentences])


# Labels
y_train = train_v["label"]
y_val   = val_v["label"]


# Classifier
clf6 = LogisticRegression(max_iter=500)
clf6.fit(X_train_vec, y_train)

# Validation
y_pred_val = clf6.predict(X_val_vec)
print("Validation Results:\n")
print(classification_report(y_val, y_pred_val, digits=4))

Validation Results:

              precision    recall  f1-score   support

           0     0.7815    0.6310    0.6982       187
           1     0.6770    0.7866    0.7277       389
           2     0.8085    0.4810    0.6032        79
           3     0.6101    0.6926    0.6488       244
           4     0.7128    0.8179    0.7618       346
           5     0.7917    0.5278    0.6333       108
           6     0.9073    0.8304    0.8671       224
           7     0.7910    0.4775    0.5955       111
           8     0.7857    0.4835    0.5986        91
           9     0.9091    0.2817    0.4301        71
          10     0.6957    0.4000    0.5079       160
          11     0.6791    0.7935    0.7318       184
          12     0.8182    0.5422    0.6522        83
          13     0.7021    0.6513    0.6758       152
          14     0.7683    0.7456    0.7568       169
          15     0.7557    0.6387    0.6923       155
          16     0.7586    0.6471    0.6984       102
      

In [ ]:
import spacy
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


# Prepare data
X_train, y_train = train_v["text"], train_v["label"]
X_val, y_val = val_v["text"], val_v["label"]


stopwords = nlp.Defaults.stop_words
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    tokens = [word for word in text.strip().split() if word not in stopwords]
    return " ".join(tokens)


# Καθάρισε τα κείμενα
for df in [train_v, val_v]:
    df["text"] = df["text"].astype(str).apply(clean_text)

# train sentences από το προηγούμενο

# Εκπαίδευση δικού μου Word2Vec μοντέλου
w2v_model_1 = Word2Vec(
    sentences=train_sentences,
    vector_size=300,
    window=8,
    min_count=5,
    workers=4,
    sg=1
)

# επιπλέον training epochs για καλύτερη σύγκλιση
w2v_model_1.train(
    train_sentences,
    total_examples=len(train_sentences),
    epochs=50
)

# Function για μετατροπή document σε embedding
def document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)


#val_sentences από πριν

# Μετατροπή όλων σε embeddings
X_train_vec = np.array([document_vector(words, w2v_model_1) for words in train_sentences])
X_val_vec   = np.array([document_vector(words, w2v_model_1) for words in val_sentences])


# Labels
y_train = train_v["label"]
y_val   = val_v["label"]


# Classifier
clf7 = LogisticRegression(max_iter=500)
clf7.fit(X_train_vec, y_train)

# Validation
y_pred_val = clf7.predict(X_val_vec)
print("Validation Results:\n")
print(classification_report(y_val, y_pred_val, digits=4))

Validation Results:

              precision    recall  f1-score   support

           0     0.8000    0.6417    0.7122       187
           1     0.6883    0.7892    0.7353       389
           2     0.7826    0.4557    0.5760        79
           3     0.6125    0.6803    0.6447       244
           4     0.7284    0.8295    0.7757       346
           5     0.8286    0.5370    0.6517       108
           6     0.8932    0.8214    0.8558       224
           7     0.7703    0.5135    0.6162       111
           8     0.7818    0.4725    0.5890        91
           9     0.8000    0.2817    0.4167        71
          10     0.6804    0.4125    0.5136       160
          11     0.6835    0.8098    0.7413       184
          12     0.8182    0.5422    0.6522        83
          13     0.7034    0.6711    0.6869       152
          14     0.7605    0.7515    0.7560       169
          15     0.7891    0.6516    0.7138       155
          16     0.7614    0.6569    0.7053       102
      

In [ ]:
# Prepare data
X_test, y_test = test_v["text"], test_v["label"]

# Tokenize
test_sentences = [[token.text for token in doc] for doc in nlp.pipe(test_v["text"], batch_size=32)]

# Function για μετατροπή document σε embedding
def document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)


#val_sentences από πριν

# Μετατροπή όλων σε embeddings
X_test_vec = np.array([document_vector(words, w2v_model_1) for words in test_sentences])


# Validation
y_pred = clf7.predict(X_test_vec)
print("Validation Results:\n")
print(classification_report(y_test, y_pred, digits=4))

Validation Results:

              precision    recall  f1-score   support

           0     0.6778    0.6524    0.6649       187
           1     0.7242    0.7763    0.7494       389
           2     0.8222    0.4744    0.6016        78
           3     0.5862    0.6296    0.6071       243
           4     0.6609    0.7717    0.7120       346
           5     0.7586    0.6055    0.6735       109
           6     0.8850    0.7867    0.8329       225
           7     0.7284    0.5364    0.6178       110
           8     0.8542    0.4457    0.5857        92
           9     0.7500    0.3380    0.4660        71
          10     0.6972    0.4750    0.5651       160
          11     0.6976    0.7814    0.7371       183
          12     0.8772    0.6024    0.7143        83
          13     0.6849    0.6536    0.6689       153
          14     0.7483    0.6471    0.6940       170
          15     0.7829    0.6516    0.7113       155
          16     0.7419    0.6765    0.7077       102
      

In [ ]:
import spacy
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


# Prepare data
X_train, y_train = train_v["text"], train_v["label"]
X_val, y_val = val_v["text"], val_v["label"]


stopwords = nlp.Defaults.stop_words
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    tokens = [word for word in text.strip().split() if word not in stopwords]
    return " ".join(tokens)


# Καθάρισε τα κείμενα
for df in [train_v]:
    df["text"] = df["text"].astype(str).apply(clean_text)

# train sentences από το προηγούμενο

# Εκπαίδευση δικού μου Word2Vec μοντέλου
w2v_model_2 = Word2Vec(
    sentences=train_sentences,
    vector_size=300,
    window=10,
    min_count=5,
    workers=4,
    sg=1
)

# επιπλέον training epochs για καλύτερη σύγκλιση
w2v_model_2.train(
    train_sentences,
    total_examples=len(train_sentences),
    epochs=50
)

# Function για μετατροπή document σε embedding
def document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)


#val_sentences από πριν

# Μετατροπή όλων σε embeddings
X_train_vec = np.array([document_vector(words, w2v_model_2) for words in train_sentences])
X_val_vec   = np.array([document_vector(words, w2v_model_2) for words in val_sentences])


# Labels
y_train = train_v["label"]
y_val   = val_v["label"]


# Classifier
clf8 = LogisticRegression(max_iter=500)
clf8.fit(X_train_vec, y_train)

# Validation
y_pred_val = clf8.predict(X_val_vec)
print("Validation Results:\n")
print(classification_report(y_val, y_pred_val, digits=4))

Validation Results:

              precision    recall  f1-score   support

           0     0.7792    0.6417    0.7038       187
           1     0.6783    0.7969    0.7329       389
           2     0.8298    0.4937    0.6190        79
           3     0.6165    0.6721    0.6431       244
           4     0.7225    0.8353    0.7748       346
           5     0.8358    0.5185    0.6400       108
           6     0.9122    0.8348    0.8718       224
           7     0.8194    0.5315    0.6448       111
           8     0.8148    0.4835    0.6069        91
           9     0.8462    0.3099    0.4536        71
          10     0.7263    0.4313    0.5412       160
          11     0.6818    0.8152    0.7426       184
          12     0.8214    0.5542    0.6619        83
          13     0.7050    0.6447    0.6735       152
          14     0.7455    0.7278    0.7365       169
          15     0.7752    0.6452    0.7042       155
          16     0.7614    0.6569    0.7053       102
      



*   TEST VOLUME



In [ ]:
# Prepare data
X_test, y_test = test_v["text"], test_v["label"]

# Tokenize
test_sentences = [[token.text for token in doc] for doc in nlp.pipe(test_v["text"], batch_size=32)]

# Function για μετατροπή document σε embedding
def document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Μετατροπή όλων σε embeddings
X_test_vec = np.array([document_vector(words, w2v_model_2) for words in test_sentences])


# Test Data
y_pred = clf8.predict(X_test_vec)
print("Validation Results:\n") #εκ παραδρομής έμεινε validation αντί για Τest
print(classification_report(y_test, y_pred, digits=4))

Validation Results:

              precision    recall  f1-score   support

           0     0.7000    0.6364    0.6667       187
           1     0.7160    0.7841    0.7485       389
           2     0.8444    0.4872    0.6179        78
           3     0.5896    0.6502    0.6184       243
           4     0.6626    0.7775    0.7154       346
           5     0.7831    0.5963    0.6771       109
           6     0.8960    0.8044    0.8478       225
           7     0.7407    0.5455    0.6283       110
           8     0.8125    0.4239    0.5571        92
           9     0.7742    0.3380    0.4706        71
          10     0.6923    0.4500    0.5455       160
          11     0.6980    0.7705    0.7325       183
          12     0.9074    0.5904    0.7153        83
          13     0.7273    0.6797    0.7027       153
          14     0.7534    0.6471    0.6962       170
          15     0.7969    0.6581    0.7208       155
          16     0.7363    0.6569    0.6943       102
      

# Chapter

In [ ]:
import spacy
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


# Prepare data
X_train, y_train = train_c["text"], train_c["label"]
X_val, y_val = val_c["text"], val_c["label"]


stopwords = nlp.Defaults.stop_words
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    tokens = [word for word in text.strip().split() if word not in stopwords]
    return " ".join(tokens)


# Καθάρισε τα κείμενα
for df in [train_c]:
    df["text"] = df["text"].astype(str).apply(clean_text)


# Tokenize
train_sentences = [[token.text for token in doc] for doc in nlp.pipe(train_c["text"], batch_size=32)]
val_sentences = [[token.text for token in doc] for doc in nlp.pipe(val_c["text"], batch_size=32)]

# Εκπαίδευση Word2Vec μοντέλου
w2v_model_2 = Word2Vec(
    sentences=train_sentences,
    vector_size=300,
    window=10,
    min_count=4,
    workers=4,
    sg=1
)

# επιπλέον training epochs για καλύτερη σύγκλιση
w2v_model_2.train(
    train_sentences,
    total_examples=len(train_sentences),
    epochs=50
)

# Function για μετατροπή document σε embedding
def document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)


# Μετατροπή όλων σε embeddings
X_train_vec = np.array([document_vector(words, w2v_model_2) for words in train_sentences])
X_val_vec   = np.array([document_vector(words, w2v_model_2) for words in val_sentences])


# Labels
y_train = train_c["label"]
y_val   = val_c["label"]


# Classifier
clf9 = LogisticRegression(class_weight='balanced',max_iter=500)
clf9.fit(X_train_vec, y_train)

# Validation
y_pred_val = clf9.predict(X_val_vec)
print("Validation Results:\n")
print(classification_report(y_val, y_pred_val, digits=4))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_val, y_pred_val))


Validation Results:

              precision    recall  f1-score   support

           0     0.9773    0.8776    0.9247        49
           1     0.6667    0.7619    0.7111        21
           2     0.8462    0.9167    0.8800        12
           3     0.4062    0.6842    0.5098        19
           4     1.0000    1.0000    1.0000         1
           5     0.5227    0.8214    0.6389        28
           6     0.6538    0.5484    0.5965        31
           7     0.5000    0.4545    0.4762        22
           8     0.6885    0.6269    0.6562        67
           9     0.6667    0.6667    0.6667         3
          10     0.8333    0.6250    0.7143         8
          11     0.8519    0.7419    0.7931        31
          12     0.0000    0.0000    0.0000         1
          13     0.0000    0.0000    0.0000         0
          14     0.5833    0.7778    0.6667        27
          15     0.5714    0.6667    0.6154         6
          16     0.1333    0.6667    0.2222         3
      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [ ]:
import spacy
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


# Εκπαίδευση Word2Vec μοντέλου
w2v_model_3 = Word2Vec(
    sentences=train_sentences,
    vector_size=400,
    window=10,
    min_count=4,
    workers=4,
    sg=1
)

# επιπλέον training epochs για καλύτερη σύγκλιση
w2v_model_3.train(
    train_sentences,
    total_examples=len(train_sentences),
    epochs=50
)

# Function για μετατροπή document σε embedding
def document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)


# Μετατροπή όλων σε embeddings
X_train_vec = np.array([document_vector(words, w2v_model_3) for words in train_sentences])
X_val_vec   = np.array([document_vector(words, w2v_model_3) for words in val_sentences])


# Labels
y_train = train_c["label"]
y_val   = val_c["label"]


# Classifier
clf10 = LogisticRegression(class_weight='balanced',max_iter=500)
clf10.fit(X_train_vec, y_train)

# Validation
y_pred_val = clf10.predict(X_val_vec)
print("Validation Results:\n")
print(classification_report(y_val, y_pred_val, digits=4))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_val, y_pred_val))


Validation Results:

              precision    recall  f1-score   support

           0     0.9556    0.8776    0.9149        49
           1     0.5926    0.7619    0.6667        21
           2     0.8462    0.9167    0.8800        12
           3     0.4062    0.6842    0.5098        19
           4     1.0000    1.0000    1.0000         1
           5     0.5455    0.8571    0.6667        28
           6     0.8182    0.5806    0.6792        31
           7     0.5500    0.5000    0.5238        22
           8     0.6935    0.6418    0.6667        67
           9     0.5000    0.6667    0.5714         3
          10     0.8333    0.6250    0.7143         8
          11     0.8519    0.7419    0.7931        31
          12     0.0000    0.0000    0.0000         1
          13     0.0000    0.0000    0.0000         0
          14     0.6176    0.7778    0.6885        27
          15     0.5000    0.6667    0.5714         6
          16     0.1429    0.6667    0.2353         3
      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [ ]:
# Prepare data
X_test, y_test = test_c["text"], test_c["label"]

# Tokenize
test_sentences = [[token.text for token in doc] for doc in nlp.pipe(test_c["text"], batch_size=32)]

# Function για μετατροπή document σε embedding
def document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Μετατροπή όλων σε embeddings
X_test_vec = np.array([document_vector(words, w2v_model_3) for words in test_sentences])

# Validation
y_pred = clf10.predict(X_test_vec)
print("Validation Results:\n")
print(classification_report(y_test, y_pred, digits=4))

Validation Results:

              precision    recall  f1-score   support

           0     0.8000    0.6512    0.7179        43
           1     0.2917    0.3182    0.3043        22
           2     0.1538    0.6667    0.2500         6
           3     0.0667    0.0625    0.0645        16
           4     0.0000    0.0000    0.0000         2
           5     0.3256    0.5600    0.4118        25
           6     0.2000    0.3846    0.2632        13
           7     0.4762    0.3333    0.3922        30
           8     0.6531    0.5614    0.6038        57
           9     0.1667    1.0000    0.2857         2
          10     0.7500    0.8571    0.8000         7
          11     0.7561    0.7045    0.7294        44
          12     0.1333    1.0000    0.2353         2
          13     0.1250    0.5000    0.2000         2
          14     0.4528    0.7059    0.5517        34
          15     0.5000    0.5000    0.5000         2
          16     0.1429    1.0000    0.2500         3
      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [ ]:
import spacy
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Εκπαίδευση Word2Vec μοντέλου
w2v_model_4 = Word2Vec(
    sentences=train_sentences,
    vector_size=400,
    window=10,
    min_count=4,
    workers=4,
    sg=0
)

# επιπλέον training epochs για καλύτερη σύγκλιση
w2v_model_4.train(
    train_sentences,
    total_examples=len(train_sentences),
    epochs=50
)

# Function για μετατροπή document σε embedding
def document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)


# Μετατροπή όλων σε embeddings
X_train_vec = np.array([document_vector(words, w2v_model_4) for words in train_sentences])
X_val_vec   = np.array([document_vector(words, w2v_model_4) for words in val_sentences])


# Labels
y_train = train_c["label"]
y_val   = val_c["label"]


# Classifier
clf11 = LogisticRegression(class_weight='balanced',max_iter=500)
clf11.fit(X_train_vec, y_train)

# Validation
y_pred_val = clf11.predict(X_val_vec)
print("Validation Results:\n")
print(classification_report(y_val, y_pred_val, digits=4))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_val, y_pred_val))


Validation Results:

              precision    recall  f1-score   support

           0     0.9333    0.8571    0.8936        49
           1     0.5556    0.7143    0.6250        21
           2     0.8571    1.0000    0.9231        12
           3     0.6500    0.6842    0.6667        19
           4     0.0000    0.0000    0.0000         1
           5     0.6286    0.7857    0.6984        28
           6     0.7308    0.6129    0.6667        31
           7     0.7059    0.5455    0.6154        22
           8     0.7143    0.7463    0.7299        67
           9     1.0000    0.6667    0.8000         3
          10     1.0000    0.5000    0.6667         8
          11     0.8929    0.8065    0.8475        31
          12     0.0000    0.0000    0.0000         1
          13     0.0000    0.0000    0.0000         0
          14     0.6897    0.7407    0.7143        27
          15     0.8000    0.6667    0.7273         6
          16     0.2000    0.3333    0.2500         3
      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [ ]:
# Prepare data
X_test, y_test = test_c["text"], test_c["label"]

# Tokenize
test_sentences = [[token.text for token in doc] for doc in nlp.pipe(test_c["text"], batch_size=32)]

# Function για μετατροπή document σε embedding
def document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Μετατροπή όλων σε embeddings
X_test_vec = np.array([document_vector(words, w2v_model_4) for words in test_sentences])

# Validation
y_pred = clf11.predict(X_test_vec)
print("Validation Results:\n")
print(classification_report(y_test, y_pred, digits=4))

Validation Results:

              precision    recall  f1-score   support

           0     0.8387    0.6047    0.7027        43
           1     0.3077    0.1818    0.2286        22
           2     0.1579    0.5000    0.2400         6
           3     0.1667    0.1250    0.1429        16
           4     0.0000    0.0000    0.0000         2
           5     0.3913    0.3600    0.3750        25
           6     0.3214    0.6923    0.4390        13
           7     0.5000    0.3000    0.3750        30
           8     0.4706    0.5614    0.5120        57
           9     0.4000    1.0000    0.5714         2
          10     0.1500    0.8571    0.2553         7
          11     0.8049    0.7500    0.7765        44
          12     0.0588    0.5000    0.1053         2
          13     0.0000    0.0000    0.0000         2
          14     0.4667    0.8235    0.5957        34
          15     0.1667    0.5000    0.2500         2
          16     0.1875    1.0000    0.3158         3
      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [ ]:
import spacy
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


# Εκπαίδευση Word2Vec μοντέλου
w2v_model_5 = Word2Vec(
    sentences=train_sentences,
    vector_size=500,       # μέγεθος vector
    window=10,              # context window
    min_count=4,           # εμφάνιση λέξης τουλάχιστον min_count φορές
    workers=4,             # parallel threads
    sg=0                  # skip-gram (αντί CBOW)
)

# επιπλέον training epochs για καλύτερη σύγκλιση
w2v_model_5.train(
    train_sentences,
    total_examples=len(train_sentences),
    epochs=60
)

# Function για μετατροπή document σε embedding
def document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)


# Μετατροπή όλων σε embeddings
X_train_vec = np.array([document_vector(words, w2v_model_5) for words in train_sentences])
X_val_vec   = np.array([document_vector(words, w2v_model_5) for words in val_sentences])


# Labels
y_train = train_c["label"]
y_val   = val_c["label"]


# Classifier
clf12 = LogisticRegression(class_weight='balanced',max_iter=500)
clf12.fit(X_train_vec, y_train)

# Validation
y_pred_val = clf12.predict(X_val_vec)
print("Validation Results:\n")
print(classification_report(y_val, y_pred_val, digits=4))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_val, y_pred_val))


Validation Results:

              precision    recall  f1-score   support

           0     0.9348    0.8776    0.9053        49
           1     0.6522    0.7143    0.6818        21
           2     0.9167    0.9167    0.9167        12
           3     0.6875    0.5789    0.6286        19
           4     0.0000    0.0000    0.0000         1
           5     0.6316    0.8571    0.7273        28
           6     0.7692    0.6452    0.7018        31
           7     0.7059    0.5455    0.6154        22
           8     0.7246    0.7463    0.7353        67
           9     1.0000    0.6667    0.8000         3
          10     0.8000    0.5000    0.6154         8
          11     0.8387    0.8387    0.8387        31
          12     0.0000    0.0000    0.0000         1
          13     0.0000    0.0000    0.0000         0
          14     0.6897    0.7407    0.7143        27
          15     0.6667    0.6667    0.6667         6
          16     0.2000    0.3333    0.2500         3
      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [ ]:

# Function για μετατροπή document σε embedding
def document_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Μετατροπή όλων σε embeddings
X_test_vec = np.array([document_vector(words, w2v_model_5) for words in test_sentences])


# Validation
y_pred = clf12.predict(X_test_vec)
print("Validation Results:\n")
print(classification_report(y_test, y_pred, digits=4))

Validation Results:

              precision    recall  f1-score   support

           0     0.8125    0.6047    0.6933        43
           1     0.2500    0.1818    0.2105        22
           2     0.1765    0.5000    0.2609         6
           3     0.1250    0.1250    0.1250        16
           4     0.0000    0.0000    0.0000         2
           5     0.3500    0.2800    0.3111        25
           6     0.2222    0.6154    0.3265        13
           7     0.5000    0.2333    0.3182        30
           8     0.5147    0.6140    0.5600        57
           9     0.2500    1.0000    0.4000         2
          10     0.1395    0.8571    0.2400         7
          11     0.8333    0.7955    0.8140        44
          12     0.0500    0.5000    0.0909         2
          13     0.0000    0.0000    0.0000         2
          14     0.4737    0.7941    0.5934        34
          15     0.1667    0.5000    0.2500         2
          16     0.2143    1.0000    0.3529         3
      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

* ΔΙΝΩ ΒΑΡΟΣ TFIDF και προχωράω σε περαιτέρω ανάλυση για να δικαιολογήσω τα χαμηλά ποσοστά που παίρνω

In [ ]:
import spacy
import numpy as np
import pandas as pd
import re

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

# Prepare data
X_train, y_train = train_c["text"], train_c["label"]
X_val, y_val = val_c["text"], val_c["label"]

stopwords = nlp.Defaults.stop_words
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    tokens = [word for word in text.strip().split() if word not in stopwords]
    return " ".join(tokens)

# Καθάρισε τα κείμενα
train_c["text"] =train_c["text"].astype(str).apply(clean_text)

# Φόρτωση ελληνικού tokenizer
nlp = spacy.load("el_core_news_sm")

# Προεπεξεργασία / Tokenization
train_sentences = [[token.text for token in doc] for doc in nlp.pipe(train_c["text"], batch_size=32)]
val_sentences   = [[token.text for token in doc] for doc in nlp.pipe(val_c["text"], batch_size=32)]
test_sentences  = [[token.text for token in doc] for doc in nlp.pipe(test_c["text"], batch_size=32)]

In [ ]:
from collections import Counter

token_counts = Counter(word for sent in train_sentences for word in sent)
top_20_words = set(word for word, _ in token_counts.most_common(20))

print("Top 20 λέξεις:", top_20_words)
train_sentences = [
    [word for word in sent if word not in top_20_words]
    for sent in train_sentences
]



Top 20 λέξεις: {'υπουργού', 'επιτροπής', 'αποφάσεως', 'υπηρεσίας', 'άρθρο', 'υπουργείου', 'υπηρεσιών', 'εφ', 'σύμφωνα', 'προσωπικού', 'ταμείου', 'άρθρου', 'απόφαση', 'τ', 'εργασίας', 'τω', 'δύναται', 'περίπτωση', 'κ', 'διατάξεων'}


ΤΕΛΙΚΟ ΜΟΝΤΕΛΟ

OOV Ratio: 79.73% λόγω oversampling 70.23% πριν από αυτό

In [ ]:

from gensim.models import Word2Vec, FastText


In [ ]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
import pandas as pd

# Αρχικά κείμενα και ετικέτες
X_train_texts, y_train = train_c["text"], train_c["label"]

# Oversampling strategy: φτάνει minority classes στο 30% του max
counter = Counter(y_train)
max_count = max(counter.values())
target_ratio = 0.3
sampling_strategy = {
    label: min(int(max_count * target_ratio), max_count)
    for label, count in counter.items()
    if count < int(max_count * target_ratio)
}

# Εφαρμογή oversampling
ros = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
X_train_texts, y_train = ros.fit_resample(X_train_texts.to_frame(), y_train)

# === Tokenization ===
train_sentences = [text.split() for text in X_train_texts["text"]]
val_sentences = [text.split() for text in val_c["text"]]
test_sentences = [text.split() for text in test_c["text"]]

# === FastText φορτωμένο μοντέλο ===
from gensim.models.fasttext import load_facebook_model
fasttext_model = load_facebook_model("cc.el.300.bin")
fasttext_model.build_vocab(train_sentences, update=True)
fasttext_model.train(train_sentences, total_examples=len(train_sentences), epochs=30)

# === TF-IDF ===
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf.fit([" ".join(doc) for doc in train_sentences])
tfidf_vocab = tfidf.vocabulary_

def get_tfidf_weight(word, vocab, model):
    if word in vocab:
        return model.idf_[vocab[word]]
    return 0.0

def document_vector_tfidf(doc, model, tfidf_vocab, tfidf_model):
    weights, vectors = [], []
    for word in doc:
        if word in model.wv.key_to_index and word in tfidf_vocab:
            vec = model.wv[word]
            weight = get_tfidf_weight(word, tfidf_vocab, tfidf_model)
            vectors.append(vec * weight)
            weights.append(weight)
    return np.sum(vectors, axis=0) / np.sum(weights) if weights else np.zeros(model.vector_size)

# === Embedding representation ===
X_train_vec = np.array([document_vector_tfidf(doc, fasttext_model, tfidf_vocab, tfidf) for doc in train_sentences])
X_val_vec   = np.array([document_vector_tfidf(doc, fasttext_model, tfidf_vocab, tfidf) for doc in val_sentences])
X_test_vec  = np.array([document_vector_tfidf(doc, fasttext_model, tfidf_vocab, tfidf) for doc in test_sentences])

# === Classification ===
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

clf = LogisticRegression(class_weight='balanced', max_iter=800)
clf.fit(X_train_vec, y_train)

# === Evaluation ===
print("\nValidation:")
print(classification_report(y_val, clf.predict(X_val_vec), digits=4))
print("\nTest:")
print(classification_report(y_test, clf.predict(X_test_vec), digits=4))

# === OOV ===
vocab_train = set(w for doc in train_sentences for w in doc)
vocab_test  = set(w for doc in test_sentences for w in doc)
oov = vocab_test - vocab_train
print(f"\n🔍 OOV Ratio: {len(oov) / len(vocab_test):.2%}")



Validation:
              precision    recall  f1-score   support

           0     0.7647    0.5306    0.6265        49
           1     0.2308    0.1429    0.1765        21
           2     0.5556    0.8333    0.6667        12
           3     0.0962    0.2632    0.1408        19
           4     0.0000    0.0000    0.0000         1
           5     0.1887    0.3571    0.2469        28
           6     0.4286    0.4839    0.4545        31
           7     0.4615    0.2727    0.3429        22
           8     0.4394    0.4328    0.4361        67
           9     0.0435    0.3333    0.0769         3
          10     1.0000    0.5000    0.6667         8
          11     0.8095    0.5484    0.6538        31
          12     0.0000    0.0000    0.0000         1
          13     0.0000    0.0000    0.0000         0
          14     0.3469    0.6296    0.4474        27
          15     0.7500    0.5000    0.6000         6
          16     0.1250    0.3333    0.1818         3
          17  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_


🔍 OOV Ratio: 79.73%


In [ ]:
#! pip install numpy==1.24.4 scipy==1.10.1 gensim==4.3.0
#import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 113.5 MB/s eta 0:00:00
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20431 sha256=a14a6bccfcddb0e15c1252b2ed4e7bfb82f51286f7ec870d362cdc559ee1df80
  Stored in directory: /root/.cache/pip/wheels/69/f5/e5/18ad53fe1ed6b2af9fad05ec052e4acbac8e92441df44bad2e
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3506 sha256=6888c4695811b12a545ac4bc395b5eface8197ef5c22bd541ece587553068b62
  Stored in directory: /root/.ca

#Subject

εχω δοκιμάσει πολλά μοντέλα αλλά το OOV Ratio ειναι τεράστιο και εδώ

In [ ]:
nlp = spacy.load("el_core_news_sm")

# Αρχικά κείμενα και ετικέτες
X_train, y_train = train_s["text"], train_s["label"]

stopwords = nlp.Defaults.stop_words
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    tokens = [word for word in text.strip().split() if word not in stopwords]
    return " ".join(tokens)


# Καθάρισε τα κείμενα
for df in [train_s]:
    df["text"] = df["text"].astype(str).apply(clean_text)


from tqdm.notebook import tqdm

# === Tokenization με tqdm ===
from tqdm.notebook import tqdm

train_sentences = [
    [token.text for token in doc]
    for doc in tqdm(nlp.pipe(X_train, batch_size=32), total=len(X_train), desc="Tokenizing train")
]

val_sentences = [
    [token.text for token in doc]
    for doc in tqdm(nlp.pipe(val_s["text"], batch_size=32), total=len(val_s), desc="Tokenizing val")
]

test_sentences = [
    [token.text for token in doc]
    for doc in tqdm(nlp.pipe(test_s["text"], batch_size=32), total=len(test_s), desc="Tokenizing test")
]


Tokenizing train:   0%|          | 0/28536 [00:00<?, ?it/s]

Tokenizing val:   0%|          | 0/9511 [00:00<?, ?it/s]

Tokenizing test:   0%|          | 0/9516 [00:00<?, ?it/s]

In [ ]:
total_tokens = sum(len(sent) for sent in train_sentences)
print(f"🔢 Total tokens in train set: {total_tokens}")


🔢 Total tokens in train set: 10158231


In [ ]:
all_tokens = [token for sent in train_sentences for token in sent]
unique_tokens = set(all_tokens)

print(f"🧾 Αριθμός προτάσεων: {len(train_sentences):,}")
print(f"🔢 Συνολικά tokens: {len(all_tokens):,}")
print(f"🔠 Μοναδικά tokens: {len(unique_tokens):,}")
#print(f"📚 FastText λεξικό (μετά το build_vocab): {len(fasttext_model.wv.key_to_index):,}")



🧾 Αριθμός προτάσεων: 28,536
🔢 Συνολικά tokens: 10,158,231
🔠 Μοναδικά tokens: 205,085


In [ ]:
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.el.300.bin.gz
! gunzip cc.el.300.bin.gz


--2025-05-25 12:37:24--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.el.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.249.39.82, 13.249.39.25, 13.249.39.89, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.249.39.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4507428610 (4.2G) [application/octet-stream]
Saving to: ‘cc.el.300.bin.gz’

cc.el.300.bin.gz    100%[===================>]   4.20G   196MB/s    in 21s     

2025-05-25 12:37:45 (204 MB/s) - ‘cc.el.300.bin.gz’ saved [4507428610/4507428610]



ΤΟ ΚΑΛΥΤΕΡΟ ΑΠΟ ΤΑ ΥΠΟΛΟΙΠΑ ΜΟΝΤΕΛΑ

In [ ]:
from gensim.models import Word2Vec

# Εκπαίδευση Word2Vec
word2vec_model = Word2Vec(
    sentences=train_sentences,
    vector_size=500,
    window=10,
    min_count=1,
    workers=4,
    sg=0
)



#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf.fit([" ".join(doc) for doc in train_sentences])
tfidf_vocab = tfidf.vocabulary_

def get_tfidf_weight(word, vocab, model):
    if word in vocab:
        return model.idf_[vocab[word]]
    return 0.0

def document_vector_tfidf(doc, model, tfidf_vocab, tfidf_model):
    weights, vectors = [], []
    for word in doc:
        if word in model.wv.key_to_index and word in tfidf_vocab:
            vec = model.wv[word]
            weight = get_tfidf_weight(word, tfidf_vocab, tfidf_model)
            vectors.append(vec * weight)
            weights.append(weight)
    return np.sum(vectors, axis=0) / np.sum(weights) if weights else np.zeros(model.vector_size)


#Embedding
X_train_vec = np.array([document_vector_tfidf(doc, word2vec_model, tfidf_vocab, tfidf) for doc in train_sentences])
X_val_vec   = np.array([document_vector_tfidf(doc, word2vec_model, tfidf_vocab, tfidf) for doc in val_sentences])
X_test_vec  = np.array([document_vector_tfidf(doc, word2vec_model, tfidf_vocab, tfidf) for doc in test_sentences])

#Classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

clf = LogisticRegression(class_weight='balanced')
clf.fit(X_train_vec, y_train)


y_val = val_s["label"]
y_test = test_s["label"]

# Evaluation
print("\nValidation:")
print(classification_report(y_val, clf.predict(X_val_vec), digits=4))
print("\nTest:")
print(classification_report(y_test, clf.predict(X_test_vec), digits=4))
from sklearn.metrics import f1_score
print("Macro F1:", f1_score(y_test, clf.predict(X_test_vec), average="macro"))

word2vec_vocab = set(word2vec_model.wv.key_to_index)
vocab_test = set(w for doc in test_sentences for w in doc)

oov_test = vocab_test - word2vec_vocab
print(f"🔍 Word2Vec OOV στο test set: {len(oov_test)} / {len(vocab_test)} = {len(oov_test)/len(vocab_test):.2%}")




Validation:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        11
           1     0.0000    0.0000    0.0000         0
           3     0.3333    1.0000    0.5000         1
           4     0.0000    0.0000    0.0000         1
           5     0.0000    0.0000    0.0000         2
           6     0.0000    0.0000    0.0000         0
           8     0.0000    0.0000    0.0000        12
           9     0.0000    0.0000    0.0000         7
          10     0.0000    0.0000    0.0000         0
          11     0.0000    0.0000    0.0000         2
          12     0.0000    0.0000    0.0000         1
          13     0.5000    0.2000    0.2857        10
          14     1.0000    1.0000    1.0000         1
          15     0.0000    0.0000    0.0000         2
          16     0.0000    0.0000    0.0000         2
          17     0.5000    0.1111    0.1818         9
          18     0.0000    0.0000    0.0000         0
          19     0.0000    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

              precision    recall  f1-score   support

           0     0.6667    0.2500    0.3636         8
           1     0.0000    0.0000    0.0000         0
           3     0.0000    0.0000    0.0000         0
           4     0.0000    0.0000    0.0000         1
           5     0.3333    0.3333    0.3333         3
           6     0.0000    0.0000    0.0000         0
           8     0.0000    0.0000    0.0000        11
           9     0.0000    0.0000    0.0000         4
          10     0.0000    0.0000    0.0000         3
          11     0.0000    0.0000    0.0000         3
          12     0.0000    0.0000    0.0000         1
          13     0.7500    0.7500    0.7500         4
          14     0.0000    0.0000    0.0000         1
          16     0.0000    0.0000    0.0000         0
          17     0.0000    0.0000    0.0000         2
          18     0.0000    0.0000    0.0000         0
          19     0.0000    0.0000    0.0000         3
          20     0.0000    

In [ ]:
from gensim.models import Word2Vec

# Εκπαίδευση Word2Vec
word2vec_model = Word2Vec(
    sentences=train_sentences,
    vector_size=500,
    window=10,
    min_count=1,
    workers=4,
    sg=0
)



#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf.fit([" ".join(doc) for doc in train_sentences])
tfidf_vocab = tfidf.vocabulary_



def document_vector_mean(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv.key_to_index]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Embedding representation
X_train_vec = np.array([document_vector_mean(doc, word2vec_model) for doc in train_sentences])
X_val_vec   = np.array([document_vector_mean(doc, word2vec_model) for doc in val_sentences])
X_test_vec  = np.array([document_vector_mean(doc, word2vec_model) for doc in test_sentences])



# Classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

clf = LogisticRegression(class_weight='balanced')
clf.fit(X_train_vec, y_train)


y_val = val_s["label"]
y_test = test_s["label"]
# === Evaluation ===
print("\nValidation:")
print(classification_report(y_val, clf.predict(X_val_vec), digits=4))
print("\nTest:")
print(classification_report(y_test, clf.predict(X_test_vec), digits=4))
from sklearn.metrics import f1_score
print("Macro F1:", f1_score(y_test, clf.predict(X_test_vec), average="macro"))

word2vec_vocab = set(word2vec_model.wv.key_to_index)
vocab_test = set(w for doc in test_sentences for w in doc)

oov_test = vocab_test - word2vec_vocab
print(f"🔍 Word2Vec OOV στο test set: {len(oov_test)} / {len(vocab_test)} = {len(oov_test)/len(vocab_test):.2%}")




Validation:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        11
           1     0.0000    0.0000    0.0000         0
           3     0.2000    1.0000    0.3333         1
           4     0.0000    0.0000    0.0000         1
           5     0.0000    0.0000    0.0000         2
           6     0.0000    0.0000    0.0000         0
           8     0.0000    0.0000    0.0000        12
           9     0.5000    0.1429    0.2222         7
          10     0.0000    0.0000    0.0000         0
          11     0.0000    0.0000    0.0000         2
          12     0.0000    0.0000    0.0000         1
          13     0.5000    0.1000    0.1667        10
          14     1.0000    1.0000    1.0000         1
          15     0.0000    0.0000    0.0000         2
          16     0.0000    0.0000    0.0000         2
          17     0.5000    0.1111    0.1818         9
          18     0.0000    0.0000    0.0000         0
          19     0.0000    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

              precision    recall  f1-score   support

           0     1.0000    0.1250    0.2222         8
           1     0.0000    0.0000    0.0000         0
           3     0.0000    0.0000    0.0000         0
           4     0.0000    0.0000    0.0000         1
           5     0.1667    0.3333    0.2222         3
           6     0.0000    0.0000    0.0000         0
           8     0.0000    0.0000    0.0000        11
           9     0.0000    0.0000    0.0000         4
          10     0.0000    0.0000    0.0000         3
          11     0.0000    0.0000    0.0000         3
          12     0.0000    0.0000    0.0000         1
          13     0.5000    0.2500    0.3333         4
          14     0.0000    0.0000    0.0000         1
          16     0.0000    0.0000    0.0000         0
          17     0.0000    0.0000    0.0000         2
          18     0.0000    0.0000    0.0000         0
          19     0.0000    0.0000    0.0000         3
          20     0.0000    

# B1.3

# Volume

In [ ]:
# Prepare data
X_train = train_v["text"]
y_train = train_v["label"]
X_val = val_v["text"]
y_val = val_v["label"]
X_test = test_v["text"]
y_test = test_v["label"]
stopwords = nlp.Defaults.stop_words
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    tokens = [word for word in text.strip().split() if word not in stopwords]
    return " ".join(tokens)


train_v["text"] = train_v["text"].astype(str).apply(clean_text)

#Tokenization
def tokenize_sentences(texts, desc="Tokenizing"):
    return [[token.text for token in doc] for doc in tqdm(nlp.pipe(texts, batch_size=32), total=len(texts), desc=desc)]

train_sentences = tokenize_sentences(train_v["text"], "Train")
val_sentences   = tokenize_sentences(val_v["text"], "Validation")
test_sentences  = tokenize_sentences(test_v["text"], "Test")

Train:   0%|          | 0/28536 [00:00<?, ?it/s]

Validation:   0%|          | 0/9511 [00:00<?, ?it/s]

Test:   0%|          | 0/9516 [00:00<?, ?it/s]

ΥΛΟΠΟΙΩ 2 MLPS
* 1 MLP με embeddings
* 1 MLP με TF-IDF αναπαράσταση

In [ ]:
import numpy as np
import pandas as pd
import spacy
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.fasttext import load_facebook_model

#SpaCy για Ελληνικά
nlp = spacy.load("el_core_news_sm")

# Φόρτωση και Fine-tune FastText
fasttext_model = load_facebook_model("cc.el.300.bin")
fasttext_model.build_vocab(train_sentences, update=True)
fasttext_model.train(train_sentences, total_examples=len(train_sentences), epochs=10)

# TF-IDF για weighted embeddings
tfidf = TfidfVectorizer()
tfidf.fit([" ".join(doc) for doc in train_sentences])
tfidf_vocab = tfidf.vocabulary_

def get_tfidf_weight(word, vocab, model):
    return model.idf_[vocab[word]] if word in vocab else 0.0

def document_vector_tfidf(doc, model, tfidf_vocab, tfidf_model):
    vectors, weights = [], []
    for word in doc:
        if word in model.wv.key_to_index and word in tfidf_vocab:
            vec = model.wv[word]
            weight = get_tfidf_weight(word, tfidf_vocab, tfidf_model)
            vectors.append(vec * weight)
            weights.append(weight)
    return np.sum(vectors, axis=0) / np.sum(weights) if weights else np.zeros(model.vector_size)

# Δημιουργία Embedding Vectors
X_train_vec = np.array([document_vector_tfidf(doc, fasttext_model, tfidf_vocab, tfidf) for doc in train_sentences])
X_val_vec   = np.array([document_vector_tfidf(doc, fasttext_model, tfidf_vocab, tfidf) for doc in val_sentences])
X_test_vec  = np.array([document_vector_tfidf(doc, fasttext_model, tfidf_vocab, tfidf) for doc in test_sentences])

# Labels
y_train = train_v["label"]
y_val   = val_v["label"]
y_test  = test_v["label"]

# MLP με embeddings
mlp_emb_pipeline = make_pipeline(
    StandardScaler(),
    MLPClassifier(
        hidden_layer_sizes=(256, 128),
        activation='relu',
        solver='adam',
        early_stopping=True,
        max_iter=300,
        random_state=42
    )
)
mlp_emb_pipeline.fit(X_train_vec, y_train)

# Αξιολόγηση
def evaluate_model(y_true, y_pred, split_name="Set"):
    print(f"\n📊 {split_name} Results:")
    print(classification_report(y_true, y_pred, digits=4))
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision (macro): {precision_score(y_true, y_pred, average='macro'):.4f}")
    print(f"Recall (macro): {recall_score(y_true, y_pred, average='macro'):.4f}")
    print(f"F1-score (macro): {f1_score(y_true, y_pred, average='macro'):.4f}")

evaluate_model(y_val, mlp_emb_pipeline.predict(X_val_vec), "Validation (Embeddings)")
evaluate_model(y_test, mlp_emb_pipeline.predict(X_test_vec), "Test (Embeddings)")

# MLP με TF-IDF

mlp_tfidf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words=greek_stopwords,
        lowercase=True,
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.95
    )),
    ('clf', MLPClassifier(
        hidden_layer_sizes=(256, 128),
        activation='relu',
        solver='adam',
        early_stopping=True,
        max_iter=300,
        random_state=42
    ))
])

mlp_tfidf_pipeline.fit(train_v["text"], y_train)
y_val_pred_tfidf = mlp_tfidf_pipeline.predict(val_v["text"])
evaluate_model(y_val, y_val_pred_tfidf, "Validation (TF-IDF)")

#Υπολογισμός OOV
vocab_train = set(word for sentence in train_sentences for word in sentence)
vocab_val   = set(word for sentence in val_sentences for word in sentence)
vocab_test  = set(word for sentence in test_sentences for word in sentence)

oov_val_words  = vocab_val - vocab_train
oov_test_words = vocab_test - vocab_train

print(f"\n🔍 OOV Ratio (Validation): {len(oov_val_words) / len(vocab_val):.2%} ({len(oov_val_words)} / {len(vocab_val)} λέξεις)")
print(f"🔍 OOV Ratio (Test): {len(oov_test_words) / len(vocab_test):.2%} ({len(oov_test_words)} / {len(vocab_test)} λέξεις)")



📊 Validation (Embeddings) Results:
              precision    recall  f1-score   support

           0     0.3720    0.3262    0.3476       187
           1     0.5536    0.4113    0.4720       389
           2     0.3000    0.2278    0.2590        79
           3     0.5116    0.3607    0.4231       244
           4     0.5294    0.4682    0.4969       346
           5     0.1959    0.1759    0.1854       108
           6     0.4348    0.4464    0.4405       224
           7     0.5446    0.4955    0.5189       111
           8     0.3717    0.4615    0.4118        91
           9     0.2500    0.0141    0.0267        71
          10     0.1449    0.0625    0.0873       160
          11     0.6111    0.3587    0.4521       184
          12     0.5769    0.3614    0.4444        83
          13     0.3612    0.6250    0.4578       152
          14     0.2222    0.7101    0.3385       169
          15     0.4099    0.4258    0.4177       155
          16     0.2609    0.1176    0.1622  

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")



📊 Validation (TF-IDF) Results:
              precision    recall  f1-score   support

           0     0.8814    0.8342    0.8571       187
           1     0.8895    0.8895    0.8895       389
           2     0.7647    0.8228    0.7927        79
           3     0.7862    0.8893    0.8346       244
           4     0.8486    0.9393    0.8916       346
           5     0.8602    0.7407    0.7960       108
           6     0.9528    0.9018    0.9266       224
           7     0.8602    0.7207    0.7843       111
           8     0.8125    0.7143    0.7602        91
           9     0.8095    0.7183    0.7612        71
          10     0.9485    0.8063    0.8716       160
          11     0.8218    0.9022    0.8601       184
          12     0.9211    0.8434    0.8805        83
          13     0.8819    0.8355    0.8581       152
          14     0.7824    0.8935    0.8343       169
          15     0.8493    0.8000    0.8239       155
          16     0.7257    0.8039    0.7628      

* Διαλέγω σίγουρα την υλοποιήση με τα TF-IDF γιατί δίνει πολύ καλύτερα αποτελέσματα

TEST EVAL VOLUME WITH MLP AND TF IDF REPRESENTATION

In [ ]:
# Test dataset
y_test_pred_tfidf = mlp_tfidf_pipeline.predict(test_v["text"])
evaluate_model(y_test, y_test_pred_tfidf, "Test (TF-IDF)")


📊 Test (TF-IDF) Results:
              precision    recall  f1-score   support

           0     0.8232    0.7968    0.8098       187
           1     0.8776    0.8663    0.8719       389
           2     0.7500    0.7692    0.7595        78
           3     0.7572    0.8601    0.8054       243
           4     0.8163    0.8988    0.8556       346
           5     0.8557    0.7615    0.8058       109
           6     0.9550    0.8489    0.8988       225
           7     0.8842    0.7636    0.8195       110
           8     0.7821    0.6630    0.7176        92
           9     0.8154    0.7465    0.7794        71
          10     0.9209    0.8000    0.8562       160
          11     0.7364    0.8852    0.8040       183
          12     0.9324    0.8313    0.8790        83
          13     0.8526    0.8693    0.8608       153
          14     0.8286    0.8529    0.8406       170
          15     0.8456    0.8129    0.8289       155
          16     0.8000    0.8235    0.8116       102
 

# Chapter

In [ ]:
# Prepare data
X_train = train_c["text"]
y_train = train_c["label"]
X_val = val_c["text"]
y_val = val_c["label"]
X_test = test_c["text"]
y_test = test_c["label"]
stopwords = nlp.Defaults.stop_words
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    tokens = [word for word in text.strip().split() if word not in stopwords]
    return " ".join(tokens)



train_c["text"] = train_c["text"].astype(str).apply(clean_text)
#Tokenization
def tokenize_sentences(texts, desc="Tokenizing"):
    return [[token.text for token in doc] for doc in tqdm(nlp.pipe(texts, batch_size=32), total=len(texts), desc=desc)]

train_sentences = tokenize_sentences(train_c["text"], "Train")
val_sentences   = tokenize_sentences(val_c["text"], "Validation")
test_sentences  = tokenize_sentences(test_c["text"], "Test")

Test: 100%|██████████| 9516/9516 [14:11<00:00, 11.17it/s]


In [ ]:
#Φόρτωση SpaCy για Ελληνικά
nlp = spacy.load("el_core_news_sm")

#Φόρτωση και Fine-tune FastText
fasttext_model = load_facebook_model("cc.el.300.bin")
fasttext_model.build_vocab(train_sentences, update=True)
fasttext_model.train(train_sentences, total_examples=len(train_sentences), epochs=10)

#TF-IDF για weighted embeddings
tfidf = TfidfVectorizer()
tfidf.fit([" ".join(doc) for doc in train_sentences])
tfidf_vocab = tfidf.vocabulary_

def get_tfidf_weight(word, vocab, model):
    return model.idf_[vocab[word]] if word in vocab else 0.0

def document_vector_tfidf(doc, model, tfidf_vocab, tfidf_model):
    vectors, weights = [], []
    for word in doc:
        if word in model.wv.key_to_index and word in tfidf_vocab:
            vec = model.wv[word]
            weight = get_tfidf_weight(word, tfidf_vocab, tfidf_model)
            vectors.append(vec * weight)
            weights.append(weight)
    return np.sum(vectors, axis=0) / np.sum(weights) if weights else np.zeros(model.vector_size)

#Δημιουργία Embedding Vectors
X_train_vec = np.array([document_vector_tfidf(doc, fasttext_model, tfidf_vocab, tfidf) for doc in train_sentences])
X_val_vec   = np.array([document_vector_tfidf(doc, fasttext_model, tfidf_vocab, tfidf) for doc in val_sentences])
X_test_vec  = np.array([document_vector_tfidf(doc, fasttext_model, tfidf_vocab, tfidf) for doc in test_sentences])


In [ ]:
# === MLP με embeddings ===
mlp_emb_pipeline = make_pipeline(
    StandardScaler(),
    MLPClassifier(
        hidden_layer_sizes=(256, 128),
        activation='relu',
        solver='adam',
        early_stopping=True,
        max_iter=300,
        random_state=42
    )
)
mlp_emb_pipeline.fit(X_train_vec, y_train)

# === Αξιολόγηση ===
def evaluate_model(y_true, y_pred, split_name="Set"):
    print(f"\n📊 {split_name} Results:")
    print(classification_report(y_true, y_pred, digits=4))
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision (macro): {precision_score(y_true, y_pred, average='macro'):.4f}")
    print(f"Recall (macro): {recall_score(y_true, y_pred, average='macro'):.4f}")
    print(f"F1-score (macro): {f1_score(y_true, y_pred, average='macro'):.4f}")

evaluate_model(y_val, mlp_emb_pipeline.predict(X_val_vec), "Validation (Embeddings)")
evaluate_model(y_test, mlp_emb_pipeline.predict(X_test_vec), "Test (Embeddings)")


📊 Validation (Embeddings) Results:
              precision    recall  f1-score   support

           0     0.4898    0.4898    0.4898        49
           1     0.0000    0.0000    0.0000        21
           2     0.5263    0.8333    0.6452        12
           3     0.0000    0.0000    0.0000        19
           4     0.0000    0.0000    0.0000         1
           5     0.3077    0.2857    0.2963        28
           6     0.6429    0.2903    0.4000        31
           7     0.1143    0.1818    0.1404        22
           8     0.3219    0.7015    0.4413        67
           9     0.0909    0.3333    0.1429         3
          10     0.6667    0.2500    0.3636         8
          11     0.7273    0.2581    0.3810        31
          12     0.0000    0.0000    0.0000         1
          13     0.0000    0.0000    0.0000         0
          14     0.2105    0.4444    0.2857        27
          15     0.7500    0.5000    0.6000         6
          16     1.0000    0.3333    0.5000  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_


📊 Test (Embeddings) Results:
              precision    recall  f1-score   support

           0     0.4194    0.6047    0.4952        43
           1     0.0000    0.0000    0.0000        22
           2     0.2632    0.8333    0.4000         6
           3     0.0000    0.0000    0.0000        16
           4     0.0000    0.0000    0.0000         2
           5     0.3913    0.3600    0.3750        25
           6     0.2222    0.1538    0.1818        13
           7     0.1818    0.2000    0.1905        30
           8     0.3148    0.5965    0.4121        57
           9     0.1667    1.0000    0.2857         2
          10     0.7500    0.8571    0.8000         7
          11     1.0000    0.5227    0.6866        44
          12     0.0000    0.0000    0.0000         2
          13     0.0000    0.0000    0.0000         2
          14     0.2687    0.5294    0.3564        34
          15     0.5000    0.5000    0.5000         2
          16     1.0000    1.0000    1.0000        

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

* ΤΕΛΙΚΑ ΚΡΑΤΑΩ ΑΥΤΟ ΤΟ ΜΟΝΤΕΛΟ

In [ ]:
import numpy as np
import pandas as pd
import spacy
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.fasttext import load_facebook_model
import spacy
import re


nlp = spacy.load("el_core_news_sm")

# Prepare data
X_train = train_c["text"]
y_train = train_c["label"]
X_val = val_c["text"]
y_val = val_c["label"]
X_test = test_c["text"]
y_test = test_c["label"]
stopwords = nlp.Defaults.stop_words
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    tokens = [word for word in text.strip().split() if word not in stopwords]
    return " ".join(tokens)

train_c["text"] = train_c["text"].astype(str).apply(clean_text)

# Αξιολόγηση
def evaluate_model(y_true, y_pred, split_name="Set"):
    print(f"\n📊 {split_name} Results:")
    print(classification_report(y_true, y_pred, digits=4))
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision (macro): {precision_score(y_true, y_pred, average='macro'):.4f}")
    print(f"Recall (macro): {recall_score(y_true, y_pred, average='macro'):.4f}")
    print(f"F1-score (macro): {f1_score(y_true, y_pred, average='macro'):.4f}")

# MLP με TF-IDF

mlp_tfidf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        lowercase=True,
        ngram_range=(1, 2),
        min_df=3,
        max_df=0.95
    )),
    ('clf', MLPClassifier(
        hidden_layer_sizes=(256, 128),
        early_stopping=True,
        activation='relu',
        solver='adam',
        max_iter=300,
        random_state=42
    ))
])


# Fit pipeline
mlp_tfidf_pipeline.fit(train_c["text"], y_train)

y_val_pred_tfidf = mlp_tfidf_pipeline.predict(val_c["text"])
evaluate_model(y_val, y_val_pred_tfidf, "Validation (TF-IDF)")
# Test dataset
y_test_pred_tfidf = mlp_tfidf_pipeline.predict(test_c["text"])
evaluate_model(y_test, y_test_pred_tfidf, "Test (TF-IDF)")


📊 Validation (TF-IDF) Results:
              precision    recall  f1-score   support

           0     0.9778    0.8980    0.9362        49
           1     0.6667    0.7619    0.7111        21
           2     0.9231    1.0000    0.9600        12
           3     0.6875    0.5789    0.6286        19
           4     1.0000    1.0000    1.0000         1
           5     0.7500    0.7500    0.7500        28
           6     0.7742    0.7742    0.7742        31
           7     0.7500    0.5455    0.6316        22
           8     0.7297    0.8060    0.7660        67
           9     0.6667    0.6667    0.6667         3
          10     0.5714    0.5000    0.5333         8
          11     0.8966    0.8387    0.8667        31
          12     0.0000    0.0000    0.0000         1
          13     0.0000    0.0000    0.0000         0
          14     0.7308    0.7037    0.7170        27
          15     1.0000    0.6667    0.8000         6
          16     1.0000    0.3333    0.5000      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_


📊 Test (TF-IDF) Results:
              precision    recall  f1-score   support

           0     0.8537    0.8140    0.8333        43
           1     0.6923    0.8182    0.7500        22
           2     0.7143    0.8333    0.7692         6
           3     0.7857    0.6875    0.7333        16
           4     0.0000    0.0000    0.0000         2
           5     0.8000    0.6400    0.7111        25
           6     0.4444    0.9231    0.6000        13
           7     0.7273    0.5333    0.6154        30
           8     0.7541    0.8070    0.7797        57
           9     1.0000    1.0000    1.0000         2
          10     0.5556    0.7143    0.6250         7
          11     0.9750    0.8864    0.9286        44
          12     0.0000    0.0000    0.0000         2
          13     0.0000    0.0000    0.0000         2
          14     0.8182    0.7941    0.8060        34
          15     0.5000    1.0000    0.6667         2
          16     1.0000    1.0000    1.0000         3
 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [ ]:
import pandas as pd
X_train = train_s["text"]
y_train = train_s["label"]
X_val = val_s["text"]
y_val = val_s["label"]
X_test = test_s["text"]
y_test = test_s["label"]
# Αν το πλήρες set labels είναι από 0 έως 2284:
all_labels = set(range(2285))

# Υπάρχοντα labels στο training set:
train_labels = set(y_train.unique())

# Labels που λείπουν από το training set:
missing_labels = all_labels - train_labels

print(f"Labels missing from training set: {len(missing_labels)}")
val_labels = set(y_val.unique())
test_labels = set(y_test.unique())

missing_in_val = all_labels - val_labels
missing_in_test = all_labels - test_labels

print(f"Missing from validation: {len(missing_in_val)}")
print(f"Missing from test: {len(missing_in_test)}")
print(y_train.value_counts())


Labels missing from training set: 142
Missing from validation: 606
Missing from test: 600
331     793
988     377
1496    186
1019    186
864     148
       ... 
1476      1
502       1
1630      1
716       1
1691      1
Name: label, Length: 2143, dtype: int64


# Subject

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import spacy
!python -m spacy download el_core_news_sm


# Φόρτωση ελληνικού μοντέλου
nlp = spacy.load("el_core_news_sm")

# === Data ===

stopwords = nlp.Defaults.stop_words
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)
    tokens = [word for word in text.strip().split() if word not in stopwords]
    return " ".join(tokens)


# Καθάρισε τα κείμενα
train_s["text"] = train_s["text"].astype(str).apply(clean_text)

train_texts = train_s["text"].astype(str).tolist()
val_texts = val_s["text"].astype(str).tolist()
test_texts = test_s["text"].astype(str).tolist()




# === Label Encoding ===
# === LabelEncoder fit ONLY on train labels
le = LabelEncoder()
train_labels = le.fit_transform(train_s["label"])
known_labels = set(le.classes_)

# === Validation: κρατάμε μόνο δείγματα με κλάσεις που υπήρχαν στο training
val_mask = val_s["label"].isin(known_labels)
val_labels = le.transform(val_s.loc[val_mask, "label"])
val_texts_filtered = [val_texts[i] for i, keep in enumerate(val_mask) if keep]

# === Test set αντίστοιχα
test_mask = test_s["label"].isin(known_labels)
test_labels = le.transform(test_s.loc[test_mask, "label"])
test_texts_filtered = [test_texts[i] for i, keep in enumerate(test_mask) if keep]


num_classes = len(le.classes_)

# === TF-IDF ===
vectorizer = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    max_features=50000
)

X_train = vectorizer.fit_transform(train_texts).toarray()
X_val = vectorizer.transform(val_texts_filtered).toarray()
X_test = vectorizer.transform(test_texts_filtered).toarray()


# === Torch Dataset ===
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TextDataset(X_train, train_labels)
val_dataset = TextDataset(X_val, val_labels)
test_dataset = TextDataset(X_test, test_labels)


train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)
test_loader = DataLoader(test_dataset, batch_size=128)

# === MLP Model ===
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLPClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.net(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLPClassifier(input_dim=X_train.shape[1], num_classes=num_classes).to(device)

# === Loss & Optimizer ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# === Training Loop ===
def train_model(model, loader):
    model.train()
    total_loss = 0
    for X_batch, y_batch in tqdm(loader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

# === Evaluation Function ===
def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(y_batch.numpy())

    return np.array(all_labels), np.array(all_preds)

# === Εκπαίδευση ===
epochs = 20
for epoch in range(epochs):
    train_loss = train_model(model, train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}")

    y_val_true, y_val_pred = evaluate_model(model, val_loader)
    acc = (y_val_true == y_val_pred).mean()
    print(f"Validation Accuracy: {acc:.4f}")

# === Τελική Αξιολόγηση ===
y_test_true, y_test_pred = evaluate_model(model, test_loader)

print(classification_report(y_test_true, y_test_pred, digits=4, zero_division=0))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 128.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('el_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


100%|██████████| 223/223 [00:03<00:00, 58.73it/s]


Epoch 1/20, Train Loss: 6.8157
Validation Accuracy: 0.0685


100%|██████████| 223/223 [00:03<00:00, 69.56it/s]


Epoch 2/20, Train Loss: 5.2801
Validation Accuracy: 0.3066


...

Epoch 18/20, Train Loss: 0.0528
Validation Accuracy: 0.6241


100%|██████████| 223/223 [00:03<00:00, 72.12it/s]


Epoch 19/20, Train Loss: 0.0459
Validation Accuracy: 0.6232


100%|██████████| 223/223 [00:03<00:00, 70.94it/s]


Epoch 20/20, Train Loss: 0.0397
Validation Accuracy: 0.6270
              precision    recall  f1-score   support

           0     0.8750    0.8750    0.8750         8
           4     0.0000    0.0000    0.0000         1
           5     0.0000    0.0000    0.0000         3
           8     0.9167    1.0000    0.9565        11
           9     0.5000    0.2500    0.3333         4
          10     0.0000    0.0000    0.0000         3
          11     1.0000    0.3333    0.5000         3
          12     0.0000    0.0000    0.0000         1
          13     0.3333    1.0000    0.5000         4
          14     1.0000    1.0000    1.0000         1
          15     0.0000    0.0000    0.0000         0
          16     0.3333    0.5000    0.4000         2
          18     0.5000    0.3333    0.4000         3
          19     0.6667    0.5000    0.5714         4
          20     0.0000    0.0000    0.0000         1
          21     0.0000    0.0000    0.0000         1
          22     0.00

In [ ]:
set_test = set(test_s["label"])
set_train = set(train_s["label"])
unknown_labels = set_test - set_train
print(f"⚠️ Μοναδικές άγνωστες ετικέτες στο test set: {unknown_labels}")
num_unknown = test_s["label"].isin(unknown_labels).sum()
print(f"📊 Αριθμός αγνώστων δειγμάτων στο test set: {num_unknown}")

⚠️ Μοναδικές άγνωστες ετικέτες στο test set: {1538, 526, 530, 1563, 1564, 543, 2080, 547, 1059, 38, 1576, 559, 2098, 2102, 1093, 584, 73, 585, 591, 1108, 95, 2144, 99, 2150, 2157, 1136, 118, 1654, 2167, 2169, 125, 2174, 2177, 2178, 1162, 1680, 1171, 1684, 1175, 2200, 158, 670, 2207, 163, 172, 2225, 2230, 2248, 220, 223, 226, 1270, 1805, 271, 789, 280, 1308, 1821, 291, 821, 827, 332, 1364, 1367, 857, 859, 1376, 1387, 1390, 376, 891, 893, 1941, 409, 1946, 935, 939, 1965, 450, 966, 463, 468, 1495, 2008, 989, 481, 997, 2022, 1526, 1529, 508, 510}
📊 Αριθμός αγνώστων δειγμάτων στο test set: 113


In [ ]:
unknown_mask = test_s["label"].isin(unknown_labels)
unknown_texts = [text for i, text in enumerate(test_texts) if unknown_mask.iloc[i]]
X_unknown = vectorizer.transform(unknown_texts).toarray()

# Run model
model.eval()
with torch.no_grad():
    inputs = torch.tensor(X_unknown, dtype=torch.float32).to(device)
    outputs = model(inputs)
    preds = torch.argmax(outputs, dim=1).cpu().numpy()

# Προβλέψεις σε ετικέτες που το μοντέλο δεν έμαθε ποτέ
print(f"🔍 Προβλέψεις για άγνωστες ετικέτες: {preds[:10]}")

🔍 Προβλέψεις για άγνωστες ετικέτες: [ 504 1022 1916 1731 1077  431 1924  985  854 1492]




*  Τελική Αξιολόγηση: Περιλαμβάνει και άγνωστες ετικέτες του test_s



In [ ]:


from sklearn.metrics import classification_report

#LabelEncoder για ΟΛΕΣ τις ετικέτες του test_s
full_le = LabelEncoder()
full_le.fit(test_s["label"])
all_test_classes = full_le.classes_

#Μετατροπή των πραγματικών test labels με τον πλήρη encoder
y_test_true_full = full_le.transform(test_s["label"])

#Μετατροπή όλων των test texts με το TF-IDF vectorizer
X_test_full = vectorizer.transform(test_texts).toarray()

# μοντέλο που έχει μάθει μόνο train-γνωστές κλάσεις
model.eval()
with torch.no_grad():
    inputs = torch.tensor(X_test_full, dtype=torch.float32).to(device)
    outputs = model(inputs)
    preds = torch.argmax(outputs, dim=1).cpu().numpy()  # class indices του training encoder

#Μετατροπή των predicted indices → original labels (π.χ. 401, 1526, κλπ)
y_test_pred_labels = le.inverse_transform(preds)

#Mapping αυτών των labels στο encoding του full test set encoder
# Κρατάμε μόνο όσες προβλέψεις υπάρχουν στο full_le
pred_mask = np.isin(y_test_pred_labels, full_le.classes_)
y_test_pred_filtered = [label if keep else full_le.classes_[0]  # ή label="UNKNOWN"
                        for label, keep in zip(y_test_pred_labels, pred_mask)]


y_test_pred_full = full_le.transform(y_test_pred_filtered)


# Classification report για ΟΛΕΣ τις κλάσεις του test_s
print(classification_report(
    y_test_true_full,
    y_test_pred_full,
    labels=np.arange(len(all_test_classes)),
    target_names=[str(lbl) for lbl in all_test_classes],
    digits=4,
    zero_division=0
))


              precision    recall  f1-score   support

           0     0.0355    0.8750    0.0683         8
           4     0.0000    0.0000    0.0000         1
           5     0.0000    0.0000    0.0000         3
           8     0.9167    1.0000    0.9565        11
           9     0.5000    0.2500    0.3333         4
          10     0.0000    0.0000    0.0000         3
          11     1.0000    0.3333    0.5000         3
          12     0.0000    0.0000    0.0000         1
          13     0.3333    1.0000    0.5000         4
          14     1.0000    1.0000    1.0000         1
          17     0.3333    0.5000    0.4000         2
          19     0.3333    0.3333    0.3333         3
          20     0.6667    0.5000    0.5714         4
          22     0.0000    0.0000    0.0000         1
          23     0.0000    0.0000    0.0000         1
          24     0.0000    0.0000    0.0000         2
          25     0.7500    0.2500    0.3750        24
          27     1.0000    